# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import sys 
import glob
import ray
import logging 
import gc
import cv2
import time
import itertools
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

import xarray as xr
import matplotlib.gridspec as gridspec

from tqdm.notebook import tqdm
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import medfilt
from pathlib import Path
from scipy.optimize import minimize_scalar
from scipy.stats import binned_statistic

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset, Subset
from kornia.geometry.transform import Affine
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sys.path.append(str(Path('.').absolute()))
from utils import *
import io_dict_to_hdf5 as ioh5
from format_data import *
from models import *
from fit_GLM import *
# pd.set_option('display.max_rows', None)

ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)


tsp python fit_GLM.py --date_ani='070921/J553RT' --ModRun=1 --Nepochs=10000 --NoShifter=True
tsp python fit_GLM.py --date_ani='110421/J569LT' --ModRun=1 --Nepochs=10000 --NoShifter=True
tsp python fit_GLM.py --date_ani='102821/J570LT' --ModRun=1 --Nepochs=10000 --NoShifter=True
tsp python fit_GLM.py --date_ani='101521/J559NC' --ModRun=1 --Nepochs=10000 --NoShifter=True


In [ ]:
args = arg_parser(jupyter=True)
MovModel = 1
Kfold = 0
args['free_move'] = True
if args['free_move']:
    stim_type = 'fm1'
else:
    stim_type = 'hf1_wn'  # 'fm1' #

dates_all = ['070921/J553RT' ,'101521/J559NC','102821/J570LT','110421/J569LT'] # '102621/J558NC' '062921/G6HCK1ALTRN',
args['date_ani'] = dates_all[3]
# args['date_ani'] = '070921/J553RT' 
# args['date_ani'] = '101521/J559NC'   
# args['date_ani'] = '102621/J558NC'
# args['date_ani'] = '110421/J569LT'
# yaml_path = sorted(list((Path(args['save_dir']).expanduser() / date_ani / stim_type / 'GLM_Network_NL/').rglob('*.yaml')))
# args['shiftn']=7
args['train_shifter']=False
args['NoL1'] = False
args['Nepochs'] = 5000
params,file_dict,exp = load_params(MovModel,Kfold,args,debug=True)

data, train_idx_list, test_idx_list = load_train_test(file_dict, **params)


if args['NoL1'] == True: 
    L1 = 'NoL1'
    mod_name = '*{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
else: 
    L1 = 'withL1'
    mod_name = '*{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
ModelList = sorted(list(exp.save_dir.rglob(mod_name+'.h5')))
model_info = params['date_ani2']+'_'+mod_name.replace('*', '')

##### Set Train Test Splits #####
Kfold = 0
train_idx = train_idx_list[Kfold]
test_idx = test_idx_list[Kfold]
data = load_Kfold_data(data,train_idx,test_idx,params)
locals().update(data)

# Initialize movement combinations
titles = np.array(['Theta', 'Phi', 'Roll', 'Pitch'])

params, xtr, xtrm, xte, xtem, ytr, yte, shift_in_tr, shift_in_te, input_size, output_size, model_type, meanbias, model_move = load_GLM_data(data, params, train_idx, test_idx)
print('Model: {}, LinMix: {}, move_features: {}, Ncells: {}, train_shifter: {}'.format(params['MovModel'],params['LinMix'],params['move_features'],params['Ncells'],params['train_shifter']))



# Single Model Plot

# Scatter Plot

In [ ]:
mod_titles = ['Mot','Vis','Add','Mul']
mod_clrs = ["#136f63","#e0ca3c","#f34213","#80a4ed"]
move_clrs = ['blue','orange','green','red']
Nepochs = 5000
NKfold=1

vloss_Mot = np.zeros((NKfold,output_size))
vloss_Vis = np.zeros((NKfold,output_size))
vloss_Add = np.zeros((NKfold,output_size))
vloss_Mul = np.zeros((NKfold,output_size))
r2_Mot = np.zeros((NKfold,output_size))
r2_Vis = np.zeros((NKfold,output_size))
r2_Add = np.zeros((NKfold,output_size))
r2_Mul = np.zeros((NKfold,output_size))
for Kfold in range(NKfold):
    params_m0,_,_ = load_params(0,Kfold,args,debug=True)
    params_m1,_,_ = load_params(1,Kfold,args,debug=True)
    params_m2,_,_ = load_params(2,Kfold,args,debug=True)
    params_m3,_,_ = load_params(3,Kfold,args,debug=True)
    data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params_m1, Kfold=Kfold, dataset_type='test')
    GLM_Mot = ioh5.load(ModelList[0])
    GLM_Vis = ioh5.load(ModelList[1])
    GLM_Add = ioh5.load(ModelList[2])
    GLM_Mul = ioh5.load(ModelList[3])
    r2_Mot[Kfold] = GLM_Mot['r2_test'].copy()
    r2_Vis[Kfold] = GLM_Vis['r2_test'].copy()
    r2_Add[Kfold] = GLM_Add['r2_test'].copy()
    r2_Mul[Kfold] = GLM_Mul['r2_test'].copy()
    vloss_Mot[Kfold] = np.mean(GLM_Mot['vloss_trace_all'][:,-10:],axis=1)/np.var(data['test_nsp'],axis=0)
    vloss_Vis[Kfold] = np.mean(GLM_Vis['vloss_trace_all'][:,-10:],axis=1)/np.var(data['test_nsp'],axis=0)
    vloss_Add[Kfold] = np.mean(GLM_Add['vloss_trace_all'][:,-10:],axis=1)/np.var(data['test_nsp'],axis=0)
    vloss_Mul[Kfold] = np.mean(GLM_Mul['vloss_trace_all'][:,-10:],axis=1)/np.var(data['test_nsp'],axis=0)


In [ ]:
celltype_dir = params['data_dir'].parent.parent.parent.parent / 'batch_files/120221_hffm'
celltype_file = celltype_dir / 'pooled_ephys_population_update_120621.pickle'
celltype = pickle.load(open(celltype_file,'rb'))
date_exp = [n for n,da in enumerate(celltype['session'].unique()) if params['date_ani2'] in da][0]
celltype2 = celltype[celltype['session']==celltype['session'].unique()[date_exp]].reset_index(drop=True)
inh = celltype2['exc_or_inh'] == 'inh'
exc = celltype2['exc_or_inh'] == 'exc'
depth = celltype2['hf1_wn_depth_from_layer5']
clust, clabel = pd.factorize(celltype2['movement_psth_type_simple'])
print(celltype['session'].unique()[date_exp])
#[2,4,7,9]

In [ ]:
dates_all = ['070921/J553RT' ,'101521/J559NC','102821/J570LT','110421/J569LT'] # '102621/J558NC' 

celltype['session'].unique() 

In [ ]:
a = 0; l = 0
thresh_fr = 1
tuning_thresh = .2
fontsize=24
bin_length=40

vals_Mot = np.nanmean(r2_Mot,axis=0)[tuning_sig2]
vals_Vis = np.nanmean(r2_Vis,axis=0)[tuning_sig2]
vals_Add = np.nanmean(r2_Add,axis=0)[tuning_sig2]
vals_Mul = np.nanmean(r2_Mul,axis=0)[tuning_sig2]
tuning_idx = np.where(tuning_sig2)[0]
lims = (0, .6)

fig, axs = plt.subplots(2,2,figsize=(16,16))
##### Vis vs Mot #####
ax = axs[0,0]
ax.scatter(vals_Vis,vals_Mot,30,c='k', label='Mul')
ax.plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
ax.set_xlabel('Visual: $r^2$',fontsize=fontsize)
ax.set_ylabel('Position: $r^2$',fontsize=fontsize)
ax.set_yticks(np.arange(0,lims[1],.1))
ax.set_yticklabels(np.round(np.arange(0,lims[1],.1),decimals=1))
for i, txt in enumerate(np.arange(len(vals_Vis))):
    ax.annotate(tuning_idx[i], (vals_Vis[i], vals_Mot[i]),fontsize=10)

##### Vis vs Add/Mul #####
ax = axs[0,1]
ax.scatter(vals_Vis,vals_Add,30,c=mod_clrs[2], label='Add')
ax.scatter(vals_Vis,vals_Mul,30,c=mod_clrs[3], label='Mul')
ax.plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
ax.set_xlabel('Visual: $r^2$',fontsize=fontsize)
ax.set_ylabel('Visual & Position: $r^2$',fontsize=fontsize)
ax.set_yticks(np.arange(0,lims[1],.1))
ax.set_yticklabels(np.round(np.arange(0,lims[1],.1),decimals=1))
leg = ax.legend()
for handle, text in zip(leg.legendHandles, leg.get_texts()):
    text.set_color(handle.get_facecolor()[0])
for i, txt in enumerate(np.arange(len(vals_Vis))):
    ax.annotate(tuning_idx[i], (vals_Vis[i], vals_Add[i]),fontsize=10,c=mod_clrs[2])
    ax.annotate(tuning_idx[i], (vals_Vis[i], vals_Mul[i]),fontsize=10,c=mod_clrs[3])

##### MSE #####
diff = vloss_Add[Kfold][tuning_sig2]-vloss_Mul[Kfold][tuning_sig2]
increase = 1-vloss_Vis[Kfold][tuning_sig2]
th = increase>-10

NMul = np.sum(diff[th]>0)
NAdd = np.sum(diff[th]<0)
ax=axs[1,0]
ax.scatter(diff[th],increase[th],c='b',s=20)
ax.axvline(x=0,c='k')
ax.set_ylabel('1-(NLVis_MSE/SD)')
ax.set_xlabel('MSE: Add-Mul')
ax.set_title('NAdd:{}, NMul:{}'.format(NAdd,NMul))
ax.set_ylim(-.05,.25)
ax.set_xlim(-.02,.02)
for i, txt in enumerate(np.arange(len(tuning_idx[th]))):
    ax.annotate(tuning_idx[th][i], (diff[th][i], increase[th][i]),fontsize=10)

##### R2 Explained Variance #####
diff = r2_Mul[Kfold][tuning_sig2]-r2_Add[Kfold][tuning_sig2]
increase = 1-vloss_Vis[Kfold][tuning_sig2]
th = increase>-10

NMul = np.sum(diff[th]>0)
NAdd = np.sum(diff[th]<0)
ax=axs[1,1]
ax.scatter(diff[th],increase[th],c='b',s=20)
ax.axvline(x=0,c='k')
ax.set_ylabel('1-(NLVis_MSE/SD)')
ax.set_xlabel('R2: Mul-Add')
ax.set_title('NAdd:{}, NMul:{}'.format(NAdd,NMul))
ax.set_ylim(-.05,.25)
for i, txt in enumerate(np.arange(len(tuning_idx[th]))):
    ax.annotate(tuning_idx[th][i], (diff[th][i], increase[th][i]),fontsize=10)

plt.tight_layout() 
fig.savefig(params_m1['fig_dir']/('ScatterComparison_'+model_info+'.png'), facecolor='white', transparent=True, bbox_inches='tight')

# Comparison Plots

In [ ]:
save_pdf = True
ncells = data['model_nsp'].shape[-1]
colors = plt.cm.cool(np.linspace(0,1,4))
hbins = .05

quartiles = np.arange(0,1.25,.25)
spk_percentile2 = np.arange(.125,1.125,.25)

fontsize = 24
params_m0,_,_ = load_params(0,Kfold,args,debug=True)
params_m1,_,_ = load_params(1,Kfold,args,debug=True)
params_m2,_,_ = load_params(2,Kfold,args,debug=True)
params_m3,_,_ = load_params(3,Kfold,args,debug=True)
data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params_m1, Kfold=Kfold, dataset_type='test')
GLM_Mot = ioh5.load(ModelList[0])
GLM_Vis = ioh5.load(ModelList[1])
GLM_Add = ioh5.load(ModelList[2])
GLM_Mul = ioh5.load(ModelList[3])
locals().update(params)
GLM_Vis['sta_all'] = GLM_Vis['Cell_NN.0.weight'].reshape((ncells,params['nt_glm_lag'],)+params['nks'])
sp_smooth = np.apply_along_axis(lambda m: np.convolve(m, np.ones(bin_length), mode='same')/(bin_length * params['model_dt']), axis=0, arr=data['test_nsp'])[bin_length:-bin_length]
pred_smooth_Mot = np.apply_along_axis(lambda m: np.convolve(m, np.ones(bin_length), mode='same')/(bin_length * params['model_dt']), axis=1, arr=GLM_Mot['pred_cv'])[:,bin_length:-bin_length].T
pred_smooth_Vis = np.apply_along_axis(lambda m: np.convolve(m, np.ones(bin_length), mode='same')/(bin_length * params['model_dt']), axis=1, arr=GLM_Vis['pred_cv'])[:,bin_length:-bin_length].T
pred_smooth_Add = np.apply_along_axis(lambda m: np.convolve(m, np.ones(bin_length), mode='same')/(bin_length * params['model_dt']), axis=1, arr=GLM_Add['pred_cv'])[:,bin_length:-bin_length].T
pred_smooth_Mul = np.apply_along_axis(lambda m: np.convolve(m, np.ones(bin_length), mode='same')/(bin_length * params['model_dt']), axis=1, arr=GLM_Mul['pred_cv'])[:,bin_length:-bin_length].T

r2_Mot = np.array([(np.corrcoef(sp_smooth[:,celln],pred_smooth_Mot[:,celln])[0, 1])**2 for celln in range(params['Ncells'])])
r2_Vis = np.array([(np.corrcoef(sp_smooth[:,celln],pred_smooth_Vis[:,celln])[0, 1])**2 for celln in range(params['Ncells'])])
r2_Add = np.array([(np.corrcoef(sp_smooth[:,celln],pred_smooth_Add[:,celln])[0, 1])**2 for celln in range(params['Ncells'])])
r2_Mul = np.array([(np.corrcoef(sp_smooth[:,celln],pred_smooth_Mul[:,celln])[0, 1])**2 for celln in range(params['Ncells'])])
r2_all = np.stack((r2_Mot,r2_Vis,r2_Add,r2_Mul))

# pbar = tqdm([21,25,49,51,61,62,88,117,121,126]) # [12,13,20,25,49,51]) # 
pbar = tqdm(np.arange(ncells))
pdf_name = params_m1['fig_dir']/ ('MoViAdMu_'+model_info+'_CellSummary.pdf')
if save_pdf:
    pdf = PdfPages(pdf_name)
    fig1, axs = plt.subplots(1,r2_all.shape[0],figsize=(20,5))
    for n in range(r2_all.shape[0]):
        ax = axs[n]
        count,edges = np.histogram(r2_all[n],bins=np.arange(0,1,hbins))
        edges_mid = np.array([(edges[i]+edges[i+1])/2 for i in range(len(edges)-1)])
        ax.bar(edges_mid, count/len(r2_all[n]),color=mod_clrs[n],width=hbins, alpha=1)
        ax.set_xticks(np.arange(0,1.2,.2))
        ax.set_xticklabels(np.round(np.arange(0,1.2,.2),decimals=1))
        ax.set_title(mod_titles[n])
        ax.set_xlabel(r'$r^2$')
    plt.tight_layout()
    pdf.savefig()
    plt.close()
for n, celln in enumerate(pbar): 
    fig2 = plt.figure(constrained_layout=False, figsize=(20,10))
    spec2 = gridspec.GridSpec(ncols=5, nrows=3, figure=fig2)
    axs = np.array([fig2.add_subplot(spec2[0, n]) for n in range(params['nt_glm_lag'])])
    axs2b = fig2.add_subplot(spec2[0,-1])
    axs3a = np.array([fig2.add_subplot(spec2[2, n]) for n in range(len(titles))])
    f2_ax6 = fig2.add_subplot(spec2[1, :5//2+1])
    f2_ax7 = fig2.add_subplot(spec2[1, (5//2+1):-1])
    f2_ax8 = fig2.add_subplot(spec2[1,-1])
    f2_ax8b = fig2.add_subplot(spec2[2,-1])
    if MovModel != 0:
        crange = np.max(np.abs(GLM_Vis['sta_all'][celln]))
        for m,ax in enumerate(axs):
            im = ax.imshow(GLM_Vis['sta_all'][celln,m],'RdBu_r',vmin=-crange,vmax=crange)
            cbar = add_colorbar(im)
            ax.axis('off')

    axs2b.plot(GLM_Vis['loss_regcv'][0,:,celln])
    axs2b.set_ylabel('MSE')
    axs2b.set_xlabel(r'$\lambda_n$')
    axs2b.set_title('Best lam:{}'.format(np.nanargmin(GLM_Vis['loss_regcv'][0,:,celln],axis=0)))

    pred_time = np.arange(len(pred_smooth_Vis[:,celln]))*params_m1['model_dt']
    f2_ax6.plot(pred_time,sp_smooth[:,celln], 'k', lw=2, zorder=0)
    f2_ax6.plot(pred_time,pred_smooth_Mot[:,celln],c=mod_clrs[0], lw=2,label='Mot', zorder=4)
    f2_ax6.plot(pred_time,pred_smooth_Vis[:,celln],c=mod_clrs[1], lw=2,label='Vis', zorder=3)
    f2_ax6.plot(pred_time,pred_smooth_Add[:,celln],c=mod_clrs[2], lw=2, label='Vis+Mov', zorder=2)
    f2_ax6.plot(pred_time,pred_smooth_Mul[:,celln],c=mod_clrs[3], lw=2, label='Vis*Mov', zorder=1)
    f2_ax6.set_xlabel('Time (s)')
    f2_ax6.set_ylabel('Spike Rate')
    f2_ax6.legend(labelcolor='linecolor', fontsize=10, handlelength=0, handletextpad=0)


    f2_ax7.plot(GLM_Vis['vloss_trace_all'][celln], label='Vis Te')
    f2_ax7.plot(GLM_Add['vloss_trace_all'][celln], label='Vis+Mov Te')
    f2_ax7.plot(GLM_Mul['vloss_trace_all'][celln], label='Vis*Mov Te')
    f2_ax7.set_xlabel('Batch #')
    f2_ax7.set_ylabel('Loss')
    f2_ax7.legend(labelcolor='linecolor', fontsize=10, handlelength=0, handletextpad=0, fancybox=True)

    w_move_Add = GLM_Add['posNN.Layer0.weight']
    w_move_Mov = GLM_Mul['posNN.Layer0.weight']
    for modeln in range(len(titles)):
        f2_ax8.bar(modeln, w_move_Add[celln,modeln], color=move_clrs[modeln])
        f2_ax8.set_xticks(np.arange(0,len(titles)))
        f2_ax8.set_xticklabels(titles)
        f2_ax8.set_ylabel('GLM Weight')
        f2_ax8.set_title('Add Weights')
        f2_ax8b.bar(modeln, w_move_Mov[celln,modeln], color=move_clrs[modeln])
        f2_ax8b.set_xticks(np.arange(0,len(titles)))
        f2_ax8b.set_xticklabels(titles)
        f2_ax8b.set_ylabel('GLM Weight')
        f2_ax8b.set_title('Mul Weights')

    top_yaxs = np.max(ax_ylims[celln])+2*np.max(tuning_stds[celln])
    for i,modeln in enumerate(range(len(titles))):
        metric = move_data[:,modeln]
        nranges = np.quantile(metric,quartiles)
        stat_range, edges, _ = binned_statistic(metric,nsp_raw[:,celln],statistic='mean',bins=nranges)
        edge_mids = np.quantile(metric,spk_percentile2)#
        # for m in range(len(nranges)-1):
        #     axs[0,modeln].axvspan(nranges[m], nranges[m+1],ymin=0,ymax=1,alpha=0.8, color=colors[m],zorder=0)
        axs3a[modeln].plot(edge_mids,stat_range/model_dt,'.-', ms=20, lw=4,c=move_clrs[modeln])

        axs3a[modeln].set_ylim(bottom=0,top=np.max(ax_ylims,axis=1)[celln]+2*np.nanmax(tuning_stds,axis=(1,2))[celln])
        axs3a[modeln].set_xlim(-30,30)
        axs3a[modeln].set_xlabel('Angle (deg)',fontsize=fontsize)
        axs3a[modeln].set_ylabel('sp/sec',fontsize=fontsize)
        axs3a[modeln].set_title(titles[modeln],fontsize=fontsize)

    plt.suptitle(' celln: {}\n Mot $r^2$:{:.03f}, Vis $r^2$:{:.03f}, Add $r^2$:{:.03f}, Mult $r^2$:{:.03f}'.format(celln,r2_all[0,celln],r2_all[1,celln],r2_all[2,celln],r2_all[3,celln]))
    plt.tight_layout()
    if save_pdf:
        pdf.savefig()
        plt.close()
if save_pdf:
    pdf.close()



In [ ]:
date_ani,params['save_dir'].parent.parent.parent/date_ani

## Tuning Histograms

In [ ]:
clrs = ['blue','orange','green','red']
titles = np.array(['theta', 'phi', 'rho', 'omega'])
fontsize=24
fig, ax = plt.subplots(1,4,figsize=(20,5))
ax = ax.flatten()
dwidth = .1
for modeln in np.arange(len(titles)):
    count,edges = np.histogram(tuning_sig[:,modeln], bins=np.arange(0,1.1,dwidth))
    edges_mid = np.array([(edges[i]+edges[i+1])/2 for i in range(len(edges)-1)])
    ax[modeln].bar(edges_mid, count/len(tuning_sig[:,modeln]),color=clrs[modeln],width=dwidth)
    ax[modeln].set_title('$\{}$'.format(titles[modeln].lower()), fontsize=fontsize)
    ax[modeln].set_xlabel('Modulation Index', fontsize=fontsize)
    ax[modeln].set_ylabel('Proportion', fontsize=fontsize)
    ax[modeln].set_xlim(0,1)
#     ax[modeln].set_ylim(0,.2)
plt.tight_layout()
fig.savefig(fig_dir/'Modulation_hist.pdf',facecolor='white', transparent=True, bbox_inches='tight')

In [ ]:
celln = 126
fontsize = 24
fig, axs = plt.subplots(1,4, figsize=(25,5))
# Tuning Curves
for i,modeln in enumerate(range(len(titles))):
    metric = move_data[:,modeln]
    axs[modeln].plot(edge_mids,tuning_curves[celln,modeln],'.-', ms=25, lw=7,c=clrs[modeln])
    axs[modeln].set_xlim(-30,30)
    axs[modeln].set_xlabel('angle (deg)',fontsize=fontsize)
    axs[modeln].set_ylabel('sp/sec',fontsize=fontsize)
    axs[modeln].set_title(r'$\{}$'.format(titles[modeln]),fontsize=fontsize)
    axs[modeln].set_ylim(bottom=0,top=np.nanmax(ax_ylims,axis=1)[celln]+2*np.nanmax(tuning_stds,axis=(1,2))[celln])
plt.tight_layout()
fig.savefig(fig_dir/'Turning_Curve_cell{}.pdf'.format(celln),facecolor='white', transparent=True, bbox_inches='tight')

# HF Vs FM RFs

In [ ]:
args = arg_parser(jupyter=True)
MovModel = 1
Kfold = 0

args['free_move'] = True
fm_dir = 'fm1' if args['prey_cap']==False else 'fm1_prey'
if args['free_move']:
    stim_type = fm_dir
else:
    stim_type = 'hf1_wn' 
dates_all = ['062921/G6HCK1ALTRN', '070921/J553RT' ,'101521/J559NC','102821/J570LT','110421/J569LT'] # '102621/J558NC' 
args['date_ani'] = dates_all[1]
# args['date_ani'] = dates_all[3]

# args['date_ani'] = '110421/J569LT'# args['date_ani']
# args['date_ani'] = '101521/J559NC'
# args['date_ani'] = '102621/J558NC'
params_FM,file_dict,exp = load_params(MovModel,Kfold,args,debug=True)

data, train_idx_list, test_idx_list = load_train_test(file_dict, **params_FM)

##### Set Train Test Splits #####
Kfold = 0
train_idx = train_idx_list[Kfold]
test_idx = test_idx_list[Kfold]
data = load_Kfold_data(data,train_idx,test_idx,params_FM)

args['NoL1'] = False
if args['NoL1'] == True: 
    L1 = 'NoL1'
    mod_name = '*Vis_{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_all'.format(L1,int(params_FM['model_dt']*1000), params_FM['nt_glm_lag'], params['Nepochs'],Kfold)
else: 
    L1 = 'withL1'
    mod_name = '*Vis_{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_all'.format(L1,int(params_FM['model_dt']*1000), params_FM['nt_glm_lag'], params['Nepochs'],Kfold)
ModelList_FM = sorted(list(exp.save_dir.rglob(mod_name+'.h5')))


model_vid_sm_shift = ioh5.load(params_FM['save_dir']/'ModelWC_shifted_dt{:03d}_MovModel{:d}.h5'.format(int(params['model_dt']*1000), 1))['model_vid_sm_shift']  # [:,5:-5,5:-5]
rolled_vid = np.hstack([np.roll(model_vid_sm_shift, nframes, axis=0) for nframes in params_FM['lag_list']])  
model_vid = rolled_vid.reshape(model_vid_sm_shift.shape[0],-1)
model_nsp = data['model_nsp']
Ncells = model_nsp.shape[-1]
sta = (model_vid.T @ model_nsp)/np.sum(model_nsp,0,keepdims=True)
FM_RF = sta.T.reshape((Ncells,len(params_FM['lag_list'])) + model_vid_sm_shift.shape[1:])

# Initialize movement combinations
titles = np.array(['Theta', 'Phi', 'Roll', 'Pitch'])
test_nsp_FM = data['test_nsp']
params_FM, xtr, xtrm, xte, xtem, ytr, yte, shift_in_tr, shift_in_te, input_size, output_size, model_type, meanbias, model_move = load_GLM_data(data, params_FM, train_idx, test_idx)

##### Head Fixed Data #####
args['free_move'] = False
fm_dir = 'fm1' if args['prey_cap']==False else 'fm1_prey'
if args['free_move']:
    stim_type = fm_dir
else:
    stim_type = 'hf1_wn' 
# args['date_ani'] = '110421/J569LT'# args['date_ani']
# args['date_ani'] = '102621/J558NC'
params_HF,file_dict,exp = load_params(MovModel,Kfold,args,debug=True)

data, train_idx_list, test_idx_list = load_train_test(file_dict, **params_HF)

##### Set Train Test Splits #####
Kfold = 0
train_idx = train_idx_list[Kfold]
test_idx = test_idx_list[Kfold]
data = load_Kfold_data(data,train_idx,test_idx,params_HF)

args['NoL1'] = False
if args['NoL1'] == True: 
    L1 = 'NoL1'
    mod_name = '*Vis_{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_all'.format(L1,int(params_FM['model_dt']*1000), params_FM['nt_glm_lag'], params['Nepochs'],Kfold)
else: 
    L1 = 'withL1'
    mod_name = '*Vis_{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_all'.format(L1,int(params_FM['model_dt']*1000), params_FM['nt_glm_lag'], params['Nepochs'],Kfold)
ModelList_HF = sorted(list(exp.save_dir.rglob(mod_name+'.h5')))

model_vid_sm_shift = ioh5.load(params_HF['save_dir']/'ModelWC_shifted_dt{:03d}_MovModel{:d}.h5'.format(int(params['model_dt']*1000), 1))['model_vid_sm_shift']  # [:,5:-5,5:-5]
rolled_vid = np.hstack([np.roll(model_vid_sm_shift, nframes, axis=0) for nframes in params_HF['lag_list']])  
model_vid = rolled_vid.reshape(model_vid_sm_shift.shape[0],-1)
model_nsp = data['model_nsp']
Ncells = model_nsp.shape[-1]
sta = (model_vid.T @ model_nsp)/np.sum(model_nsp,0,keepdims=True)
HF_RF = sta.T.reshape((Ncells,len(params_HF['lag_list'])) + model_vid_sm_shift.shape[1:])

# Initialize movement combinations
titles = np.array(['Theta', 'Phi', 'Roll', 'Pitch'])
test_nsp_HF = data['test_nsp']
params_HF['Ncells'] = params_FM['Ncells']
print('Model: {}, LinMix: {}, move_features: {}, Ncells: {}, train_shifter: {}'.format(params_FM['MovModel'],params_FM['LinMix'],params_FM['move_features'],params_FM['Ncells'],params_FM['train_shifter']))


In [ ]:
Nepochs = 5000
Kfold = 0
save_dir_fm = params_FM['save_dir_fm'] / 'GLM_Network' / exp.name / 'version_{}'.format(Kfold)
save_dir_hf = params_HF['save_dir_hf'] / 'GLM_Network' / exp.name / 'version_{}'.format(Kfold)
GLM_FM = ioh5.load(ModelList_FM[0])
GLM_HF = ioh5.load(ModelList_HF[0])

GLM_FM['r2_test'][np.isnan(GLM_FM['r2_test'])]=0
GLM_HF['r2_test'][np.isnan(GLM_HF['r2_test'])]=0

malph = 0*np.ones(params_FM['Ncells'],dtype=int)
mlam_FM = np.nanargmax(GLM_FM['r2_test'][0],axis=0) #22*np.ones(params['Ncells'],dtype=int)# 
mlam_HF = np.nanargmax(GLM_HF['r2_test'][0],axis=0) #22*np.ones(params['Ncells'],dtype=int)# 
mlam_FM_mse = np.nanargmin(GLM_FM['loss_regcv'][0],axis=0) #11*np.ones(params['Ncells'],dtype=int)# 
mlam_HF_mse = np.nanargmin(GLM_HF['loss_regcv'][0],axis=0) #11*np.ones(params['Ncells'],dtype=int)# 
cellnum = np.arange(params_FM['Ncells'])

sta_FM = GLM_FM['Cell_NN.0.weight'][malph,mlam_FM_mse,cellnum].reshape((params_FM['Ncells'],params_FM['nt_glm_lag'],)+params_FM['nks'])[:,0,5:-5,5:-5]
sta_HF = GLM_HF['Cell_NN.0.weight'][malph,mlam_HF_mse,cellnum].reshape((params_FM['Ncells'],params_FM['nt_glm_lag'],)+params_FM['nks'])[:,0,5:-5,5:-5]

GLM_FM['sta_all'] = GLM_FM['Cell_NN.0.weight'][malph,mlam_FM_mse,cellnum].reshape((params_FM['Ncells'],params_FM['nt_glm_lag'],)+params_FM['nks'])#[:,:,5:-5,5:-5]
GLM_HF['sta_all'] = GLM_HF['Cell_NN.0.weight'][malph,mlam_HF_mse,cellnum].reshape((params_FM['Ncells'],params_FM['nt_glm_lag'],)+params_FM['nks'])#[:,:,5:-5,5:-5]


In [ ]:
sf = 4
FM_sta_up = np.zeros((GLM_FM['sta_all'].shape[0], GLM_FM['sta_all'].shape[1], sf*(GLM_FM['sta_all'].shape[-2]), sf*(GLM_FM['sta_all'].shape[-1])))
HF_sta_up = np.zeros((GLM_HF['sta_all'].shape[0], GLM_HF['sta_all'].shape[1], sf*(GLM_HF['sta_all'].shape[-2]), sf*(GLM_HF['sta_all'].shape[-1])))
FM_RF_up = np.zeros((FM_RF.shape[0], FM_RF.shape[1], sf*(FM_RF.shape[-2]), sf*(FM_RF.shape[-1])))
HF_RF_up = np.zeros((HF_RF.shape[0], HF_RF.shape[1], sf*(HF_RF.shape[-2]), sf*(HF_RF.shape[-1])))

for n in range(GLM_HF['sta_all'].shape[0]):
    for t in range(GLM_HF['sta_all'].shape[1]):
        HF_sta_up[n, t] = cv2.resize(GLM_HF['sta_all'][n, t], (sf*(GLM_HF['sta_all'].shape[-1]), sf*(GLM_HF['sta_all'].shape[-2])))
for n in range(GLM_FM['sta_all'].shape[0]):
    for t in range(GLM_FM['sta_all'].shape[1]):
        FM_sta_up[n, t] = cv2.resize(GLM_FM['sta_all'][n, t], (sf*GLM_FM['sta_all'].shape[-1], sf*GLM_FM['sta_all'].shape[-2]))

for n in range(FM_RF_up.shape[0]):
    for t in range(FM_RF_up.shape[1]):
        FM_RF_up[n, t] = cv2.resize(FM_RF[n, t], (sf*(FM_RF.shape[-1]), sf*(FM_RF.shape[-2])))
for n in range(HF_RF_up.shape[0]):
    for t in range(HF_RF_up.shape[1]):
        HF_RF_up[n, t] = cv2.resize(HF_RF[n, t], (sf*(HF_RF.shape[-1]), sf*(HF_RF.shape[-2])))

r2_HFFM_sta = np.zeros(params_FM['Ncells'])
for celln in np.arange(params_FM['Ncells']):
    r2_HFFM_sta[celln] = np.corrcoef(sta_HF[celln].flatten(),sta_FM[celln].flatten())[0,1]

In [ ]:
sp_smooth_FM = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params_FM['bin_length']), mode='same')/(params_FM['bin_length'] * params_FM['model_dt']), axis=0, arr=test_nsp_FM)[params_FM['bin_length']:-params_FM['bin_length']]
sp_smooth_HF = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params_FM['bin_length']), mode='same')/(params_FM['bin_length'] * params_FM['model_dt']), axis=0, arr=test_nsp_HF)[params_FM['bin_length']:-params_FM['bin_length']]
pred_smooth_FM = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params_FM['bin_length']), mode='same')/(params_FM['bin_length'] * params_FM['model_dt']), axis=1, arr=GLM_FM['pred_cv'][malph,mlam_FM_mse,cellnum])[:,params_FM['bin_length']:-params_FM['bin_length']].T
pred_smooth_HF = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params_FM['bin_length']), mode='same')/(params_FM['bin_length'] * params_FM['model_dt']), axis=1, arr=GLM_HF['pred_cv'][malph,mlam_HF_mse,cellnum])[:,params_FM['bin_length']:-params_FM['bin_length']].T
r2_FM= np.array([(np.corrcoef(sp_smooth_FM[:,celln],pred_smooth_FM[:,celln])[0, 1])**2 for celln in range(params_FM['Ncells'])])
r2_HF= np.array([(np.corrcoef(sp_smooth_HF[:,celln],pred_smooth_HF[:,celln])[0, 1])**2 for celln in range(params_FM['Ncells'])])
r2_all = np.stack((r2_FM,r2_HF,r2_HFFM_sta))

In [ ]:
save_pdf=True
hbins=.05
mod_titles = ['Mot','Vis','Add','Mul']
mod_clrs = ["#136f63","#e0ca3c","#f34213","#80a4ed"]
move_clrs = ['blue','orange','green','red']
sta_titles = ['FM','HF','FMHF']
# pbar = tqdm([21,25,49,51,61,62,88,117,121,126]) # [12,13,20,25,49,51]) # 
pbar = tqdm(np.arange(params_FM['Ncells']))
pdf_name = params_FM['fig_dir']/ ('FM_HF_RF_MSE_'+model_info+'2.pdf')
if save_pdf:
    pdf = PdfPages(pdf_name)
    fig1, axs = plt.subplots(1,r2_all.shape[0],figsize=(15,5))
    for n in range(r2_all.shape[0]):
        ax = axs[n]
        if n < 2:
            lim0 = 0
            dlim = .2
            xlab = r'$r^2$'
        else: 
            lim0=0
            dlim = .2
            xlab = 'cc'
        count,edges = np.histogram(np.abs(r2_all[n]),bins=np.arange(lim0,1,hbins))
        edges_mid = np.array([(edges[i]+edges[i+1])/2 for i in range(len(edges)-1)])
        ax.bar(edges_mid, count/len(r2_all[n]),color=mod_clrs[n],width=hbins, alpha=1)
        ax.set_xticks(np.arange(lim0,1.2,dlim))
        ax.set_xticklabels(np.round(np.arange(lim0,1.2,dlim),decimals=1))
        ax.set_title(sta_titles[n])
        ax.set_xlabel(xlab)
    plt.tight_layout()
    pdf.savefig()
    plt.close()
for n, celln in enumerate(pbar): 
    fig = plt.figure(constrained_layout=False, figsize=(25,8))
    spec = gridspec.GridSpec(ncols=5, nrows=2, figure=fig)
    axs0 = np.array([fig.add_subplot(spec[0, n]) for n in range(spec.ncols)])
    axs1 = np.array([fig.add_subplot(spec[1, n]) for n in range(spec.ncols)])

    ax = axs0[0]
    crange = np.max(np.abs(FM_RF_up[celln]))
    im = ax.imshow(FM_RF_up[celln,0],'RdBu_r',vmin=-crange,vmax=crange)
    cbar = add_colorbar(im)
    ax.set_title('FM STA')
    
    ax = axs0[1]
    crange = np.max(np.abs(FM_sta_up[celln]))
    im = ax.imshow(FM_sta_up[celln,0],'RdBu_r',vmin=-crange,vmax=crange)
    cbar = add_colorbar(im)
    ax.set_title('FM RF')


    ax = axs0[2]
    ax.plot(sp_smooth_FM[:,celln])
    ax.plot(pred_smooth_FM[:,celln])
    ax.set_title('$r^2$: {:.03f}'.format(r2_FM[celln]))

    ax = axs0[3]
    ax.plot(GLM_FM['loss_regcv'][0,:,celln])
    ax.set_ylabel('MSE')
    ax.set_xlabel(r'$\lambda_n$')
    ax.set_title('Best lam:{}'.format(mlam_FM_mse[celln]))

    ax = axs0[4]
    ax.plot(GLM_FM['r2_test'][0,:,celln])
    ax.set_ylabel('r2')
    ax.set_xlabel(r'$\lambda_n$')
    ax.set_title('Best lam:{}'.format(mlam_FM[celln]))

    ########### Head Fixed plots ###########
    ax = axs1[0]
    crange = np.max(np.abs(HF_RF_up[celln]))
    im = ax.imshow(HF_RF_up[celln,0],'RdBu_r',vmin=-crange,vmax=crange)
    cbar = add_colorbar(im)
    ax.set_title('HF STA')

    ax = axs1[1]
    crange = np.max(np.abs(HF_sta_up[celln]))
    im = ax.imshow(HF_sta_up[celln,0],'RdBu_r',vmin=-crange,vmax=crange)
    cbar = add_colorbar(im)
    ax.set_title('HF RF')

    ax = axs1[2]
    ax.plot(sp_smooth_HF[:,celln])
    ax.plot(pred_smooth_HF[:,celln])
    ax.set_title('$r^2$: {:.03f}'.format(r2_HF[celln]))

    ax = axs1[3]
    ax.plot(GLM_HF['loss_regcv'][0,:,celln])
    ax.set_ylabel('MSE')
    ax.set_xlabel(r'$\lambda_n$')
    ax.set_title('Best lam:{}'.format(mlam_HF_mse[celln]))

    ax = axs1[4]
    ax.plot(GLM_HF['r2_test'][0,:,celln])
    ax.set_ylabel('r2')
    ax.set_xlabel(r'$\lambda_n$')
    ax.set_title('Best lam:{}'.format(mlam_HF[celln]))

    plt.suptitle(' celln: {} FM_HF_RF_CC:{:.03f}'.format(celln,r2_HFFM_sta[celln]))
    plt.tight_layout()

    if save_pdf:
        pdf.savefig()
        plt.close()
if save_pdf:
    pdf.close()


In [ ]:
cells = [25,45,70,117] # 
# cells = [12,20,25] #[43,48,49,51,61,68,70,73,79,86,90,93,99] #np.arange(100)
num_cells = len(cells)
fontsize=24
# fig_dir_presentations = check_path(fig_dir,'PaperFigs') #UT_Austin_Talk

# fig1, axs1 = plt.subplots(2,2,figsize=(10,5),)
fig1 = plt.figure(constrained_layout=False, figsize=(int(5*num_cells),5))
spec2 = gridspec.GridSpec(ncols=num_cells, nrows=2, figure=fig1, wspace=.1,hspace=.05)
axs1 = np.array([fig1.add_subplot(spec2[n,m]) for n in range(2) for m in range(num_cells)]).reshape(2,num_cells)
for n, cell in enumerate(cells):
    crange1 = np.max(np.abs(sta_HF[cell]))
    im1 = axs1[0,n].imshow(sta_HF[cell], 'RdBu_r', vmin=-crange1, vmax=crange1)
    r2_up = np.corrcoef(sta_HF[cell].flatten(),sta_FM[cell].flatten())[0,1]**2
    axs1[0,n].set_title('Unit: {}, $r^2$={:.02f}'.format(n+1,r2_up), fontsize=fontsize)
    crange2 = np.max(np.abs(sta_FM[cell]))
    im2 = axs1[1,n].imshow(sta_FM[cell],'RdBu_r', vmin=-crange2, vmax=crange2)

axs1[0,0].set_ylabel('Head \n fixed', fontsize=fontsize)
axs1[1,0].set_ylabel('Freely \n moving', fontsize=fontsize)

cbar2 = fig1.colorbar(im2, ax=axs1.ravel().tolist())
# cbar2.set_ticks([-crange2, crange2])
# cbar2.set_ticklabels(['Dark', 'Light'])
# cbar2.ax.tick_params(labelsize=fontsize)

for ax in axs1.flat:
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)
        ax.spines[axis].set_visible(True)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
# fig1.savefig(fig_dir/'HF_FM_RFComparison.png', facecolor='white', transparent=True, bbox_inches='tight')

In [ ]:
sf = 4
FM_sta_up = np.zeros((GLM_Vis['sta_all'].shape[0], GLM_Vis['sta_all'].shape[1],sf*GLM_Vis['sta_all'].shape[-2], sf*GLM_Vis['sta_all'].shape[-1]))
HF_sta_up = np.zeros((GLM_HF['sta_all'].shape[0], GLM_HF['sta_all'].shape[1], sf*(GLM_HF['sta_all'].shape[-2]), sf*(GLM_HF['sta_all'].shape[-1])))
for n in range(GLM_HF['sta_all'].shape[0]):
    for t in range(GLM_HF['sta_all'].shape[1]):
        HF_sta_up[n, t] = cv2.resize(GLM_HF['sta_all'][n, t], (sf*(GLM_HF['sta_all'].shape[-1]), sf*(GLM_HF['sta_all'].shape[-2])))
for n in range(GLM_Vis['sta_all'].shape[0]):
    for t in range(GLM_Vis['sta_all'].shape[1]):
        FM_sta_up[n, t] = cv2.resize(GLM_Vis['sta_all'][n, t], (sf*GLM_Vis['sta_all'].shape[-1], sf*GLM_Vis['sta_all'].shape[-2]))


In [ ]:
r2_HFFM_sta = np.zeros(model_nsp.shape[-1])
for celln in np.arange(model_nsp.shape[-1]):
    r2_HFFM_sta[celln] = np.corrcoef(sta_HF[celln].flatten(),sta_FM[celln].flatten())[0,1]

In [ ]:
np.argmax(r2_HFFM_sta)

In [ ]:
thresh_fr = 1
r2_thresh=.05
# r2_sigP = np.nanmean(r2_Mot,axis=0)

r2_sigV = np.nanmean(r2_Vis,axis=0)
r2_sig2 = (r2_sigV>r2_thresh) & (avg_fr>thresh_fr)
r2_idx = np.where(r2_sig2)[0]
np.sum(r2_sig2)

In [ ]:

hbins=.05
fontsize=24
fig, ax = plt.subplots(figsize=(7,7))
count,edges = np.histogram(r2_HFFM_sta,bins=np.arange(-.6,.8,hbins))
edges_mid = np.array([(edges[i]+edges[i+1])/2 for i in range(len(edges)-1)])
ax.bar(edges_mid, count/len(r2_HFFM_sta),color='k',width=hbins, alpha=.75)
ax.set_xlabel('RF Correlation',fontsize=fontsize+5)
ax.set_ylabel('Fraction of Units',fontsize=fontsize+5)
ax.set_yticks(np.arange(-.6,.8,.05))
ax.set_yticklabels(np.round(np.arange(-.6,.8,.05),decimals=3),fontsize=fontsize-2)
ax.set_xticks(np.arange(-.6,.8,.2))
ax.set_xticklabels(np.round(np.arange(-.6, .8, .2),decimals=1), fontsize=fontsize-2)
# ax.set_xlim(0,.8)
ax.set_ylim(0,.25)
# fig.savefig(fig_dir/'HFFM_RF_CCHist.pdf', facecolor='white', transparent=True, bbox_inches='tight')


In [ ]:

cells = [25,117,49,70,90] # 
# cells = np.arange(params['Ncells'])
# cells = [12,20,25,68] #np.arange(18)
# cells = [43,48,49,51,61,68,70,73,79,86,90,93,99] #np.arange(100)

num_cells = len(cells)
fontsize=24
# fig_dir_presentations = check_path(fig_dir,'PaperFigs') #UT_Austin_Talk
xcut1=5
xcut2=-5
ycut1=5
ycut2=-5
# fig1, axs1 = plt.subplots(2,2,figsize=(10,5),)
fig1 = plt.figure(constrained_layout=False, figsize=(int(5*num_cells),5))
spec2 = gridspec.GridSpec(ncols=num_cells, nrows=2, figure=fig1, wspace=.1,hspace=.05)
axs1 = np.array([fig1.add_subplot(spec2[n,m]) for n in range(2) for m in range(num_cells)]).reshape(2,num_cells)
for n, cell in enumerate(cells):
    r2_up = np.corrcoef(sta_HF[cell,xcut1:xcut2,ycut1:ycut2].flatten(),sta_FM[cell,xcut1:xcut2,ycut1:ycut2].flatten())[0,1]
    crange2 = np.max(np.abs(FM_sta_up[cell,0,int(sf*xcut1):int(sf*xcut2),int(sf*ycut1):int(sf*ycut2)]))
    im2 = axs1[0,n].imshow(FM_sta_up[cell, 0,int(sf*xcut1):int(sf*xcut2),int(sf*ycut1):int(sf*ycut2)],'RdBu_r', vmin=-crange2, vmax=crange2)
    crange1 = np.max(np.abs(HF_sta_up[cell,0,int(sf*xcut1):int(sf*xcut2),int(sf*ycut1):int(sf*ycut2)]))
    im1 = axs1[1,n].imshow(HF_sta_up[cell, 0,int(sf*xcut1):int(sf*xcut2),int(sf*ycut1):int(sf*ycut2)], 'RdBu_r', vmin=-crange1, vmax=crange1)
    axs1[0,n].set_title('Unit: {}, CC={:.02f}'.format(n+1,r2_up), fontsize=fontsize)

axs1[1,0].set_ylabel('Head \n fixed', fontsize=fontsize)
axs1[0,0].set_ylabel('Freely \n moving', fontsize=fontsize)

cbar2 = fig1.colorbar(im1, ax=axs1.ravel().tolist())
cbar2.set_ticks([-crange1, crange1])
cbar2.set_ticklabels(['Dark', 'Light'])
cbar2.ax.tick_params(labelsize=fontsize)

for ax in axs1.flat:
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)
        ax.spines[axis].set_visible(True)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])    
# fig1.savefig(fig_dir/'HF_FM_RF_CCComparison.png', facecolor='white', transparent=True, bbox_inches='tight')

In [ ]:
# cells = [25,70,90,117] # 49
cells = [12,20,25,68] #np.arange(18)
num_cells = len(cells)

fig1 = plt.figure(constrained_layout=False, figsize=(int(5*params['nt_glm_lag']),int(3*num_cells)))
spec2 = gridspec.GridSpec(ncols=params['nt_glm_lag'], nrows=num_cells, figure=fig1, wspace=.05,hspace=.1)
axs1 = np.array([fig1.add_subplot(spec2[n,m]) for n in range(num_cells) for m in range(params['nt_glm_lag'])]).reshape(num_cells,params['nt_glm_lag'])
for n, cell in enumerate(cells):
    crange2 = np.max(np.abs(FM_sta_up[cell]))
    for lag in range(params['nt_glm_lag']):
        im2 = axs1[n,lag].imshow(FM_sta_up[cell, lag],'RdBu_r', vmin=-crange2, vmax=crange2)
        axs1[0,lag].set_title('Lag:{:03d} ms'.format(int(1000*params['lag_list'][lag]*params['model_dt'])),fontsize=fontsize)
    axs1[n,0].set_ylabel('Unit {}'.format(n+1),fontsize=fontsize)
cbar2 = fig.colorbar(im2, ax=axs1.ravel().tolist())
cbar2.set_ticks([-crange2, crange2])
cbar2.set_ticklabels(['Dark', 'Light'])
cbar2.ax.tick_params(labelsize=fontsize)

for ax in axs1.flat:
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)
        ax.spines[axis].set_visible(True)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])

# fig1.savefig(fig_dir/'Example_FM_RFs.png', facecolor='white', transparent=True, bbox_inches='tight')
# print('Saved to: {}'.format(FigPath_SFN/'FM_RFs.png'))

In [ ]:
GLM_HF['r2_test'][r2_HFFM_sta>.09].shape[0],GLM_HF['r2_test'].shape[0]

In [ ]:
fig, ax = plt.subplots()
im=ax.scatter(GLM_HF['r2_test'][r2_HFFM_sta>.09],GLM_Vis['r2_test'][r2_HFFM_sta>.09],c=r2_HFFM_sta[r2_HFFM_sta>.09],cmap='cool')
ax.set_xlabel('Head-Fixed $r^2$')
ax.set_ylabel('Free-Moving $r^2$')
cbar = add_colorbar(im)

In [ ]:
##### Plot All Cells #####
cells = np.argpartition(r2_HFFM_sta, -127)[-127:]
num_cells = len(cells)
fontsize=24
# fig_dir_presentations = check_path(fig_dir,'PaperFigs') #UT_Austin_Talk

# fig1, axs1 = plt.subplots(2,2,figsize=(10,5),)
fig1 = plt.figure(constrained_layout=False, figsize=(int(5*num_cells),5))
spec2 = gridspec.GridSpec(ncols=num_cells, nrows=2, figure=fig1, wspace=.1,hspace=.05)
axs1 = np.array([fig1.add_subplot(spec2[n,m]) for n in range(2) for m in range(num_cells)]).reshape(2,num_cells)
for n, cell in enumerate(cells):
    r2_up = np.corrcoef(sta_HF[cell,xcut1:xcut2,ycut1:ycut2].flatten(),sta_FM[cell,xcut1:xcut2,ycut1:ycut2].flatten())[0,1]
    crange2 = np.max(np.abs(FM_sta_up[cell,0,int(sf*xcut1):int(sf*xcut2),int(sf*ycut1):int(sf*ycut2)]))
    im2 = axs1[0,n].imshow(FM_sta_up[cell, 0,int(sf*xcut1):int(sf*xcut2),int(sf*ycut1):int(sf*ycut2)],'RdBu_r', vmin=-crange2, vmax=crange2)
    crange1 = np.max(np.abs(HF_sta_up[cell,0,int(sf*xcut1):int(sf*xcut2),int(sf*ycut1):int(sf*ycut2)]))
    im1 = axs1[1,n].imshow(HF_sta_up[cell, 0,int(sf*xcut1):int(sf*xcut2),int(sf*ycut1):int(sf*ycut2)], 'RdBu_r', vmin=-crange1, vmax=crange1)
    axs1[0,n].set_title('Unit: {}, CC={:.02f}'.format(n+1,r2_up), fontsize=fontsize)

axs1[1,0].set_ylabel('Head \n fixed', fontsize=fontsize)
axs1[0,0].set_ylabel('Freely \n moving', fontsize=fontsize)

cbar2 = fig1.colorbar(im2, ax=axs1.ravel().tolist())
cbar2.set_ticks([-crange2, crange2])
cbar2.set_ticklabels(['Dark', 'Light'])
cbar2.ax.tick_params(labelsize=fontsize)

for ax in axs1.flat:
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)
        ax.spines[axis].set_visible(True)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])  
plt.tight_layout()  
plt.close()
fig1.savefig(params['fig_dir']/'HF_FM_RFComparison_sorted.pdf', facecolor='white', transparent=True, bbox_inches='tight')

In [ ]:
params['fig_dir']

### HF predicts FM

In [ ]:
args = arg_parser(jupyter=True)
MovModel = 1
Kfold = 0
args['free_move'] = True
if args['free_move']:
    stim_type = 'fm1'
else:
    stim_type = 'hf1_wn'  # 'fm1' #
date_exp = 0
dates_all = ['070921/J553RT' ,'101521/J559NC','102821/J570LT','110421/J569LT'] # '102621/J558NC' '062921/G6HCK1ALTRN',
args['date_ani'] = dates_all[date_exp]
# params,file_dict,exp = load_params(MovModel,Kfold,args,debug=True)

# data, train_idx_list, test_idx_list = load_train_test(file_dict, **params)

# ##### Set Train Test Splits #####
# Kfold = 0
# train_idx = train_idx_list[Kfold]
# test_idx = test_idx_list[Kfold]
# data = load_Kfold_data(data,train_idx,test_idx,params)
# locals().update(data)

# Initialize movement combinations
titles = np.array(['Theta', 'Phi', 'Roll', 'Pitch'])


mod_titles = ['Mot','Vis','Add','Mul']
mod_clrs = ["#136f63","#e0ca3c","#f34213","#80a4ed"]
ModelList_FM = []
ModelList_HF = []
xtr_all, xtrm_all, xte_all, xtem_all, ytr_all, yte_all, shift_in_tr_all, shift_in_te_all,Ncells_tot = [],[],[],[],[],[],[],[],[]
for d,date_ani in enumerate(dates_all):
    ##### Free-Moving #####
    args['free_move']=True
    args['date_ani'] = date_ani
    args['NoL1']=False
    args['Nepochs'] = 5000
    params,file_dict,exp = load_params(MovModel,Kfold,args,debug=True)
    date_ani2 = params['save_dir_fm'].parent.parent.parent/date_ani
    data, train_idx_list, test_idx_list = load_train_test(file_dict, **params)
    ##### Set Train Test Splits #####
    train_idx = train_idx_list[Kfold]
    test_idx = test_idx_list[Kfold]
    data = load_Kfold_data(data,train_idx,test_idx,params)
    params, xtr, xtrm, xte, xtem, ytr, yte, shift_in_tr, shift_in_te, input_size, output_size, model_type, meanbias, model_move = load_GLM_data(data, params, train_idx, test_idx)
    Ncells_tot.append(params['Ncells'])
    print('Model: {}, LinMix: {}, move_features: {}, Ncells: {}, train_shifter: {}'.format(params['MovModel'],params['LinMix'],params['move_features'],params['Ncells'],params['train_shifter']))
    # xtr_all.append(xtr)
    # xtrm_all.append(xtrm)
    xte_all.append(xte)
    # xtem_all.append(xtem)
    # ytr_all.append(ytr)
    yte_all.append(yte)
    # shift_in_tr_all.append(shift_in_tr)
    # shift_in_te_all.append(shift_in_te)
    if args['NoL1'] == True: 
        L1 = 'NoL1'
        mod_name = '*Vis_{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    else: 
        L1 = 'withL1'
        mod_name = '*Vis_{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    ModelList = np.array(sorted(list(date_ani2.rglob(mod_name+'.h5'))))[0]
    ModelList_FM.append(ModelList)
    ModelList = np.array(sorted(list(date_ani2.rglob(mod_name+'.h5'))))[1]
    ModelList_HF.append(ModelList)
ModelList_FM = np.stack(ModelList_FM)
ModelList_HF = np.stack(ModelList_HF)
# xte_all = nn.utils.rnn.pad_sequence(xte_all).permute(0,2,1)
# xte_all = xte_all.reshape(xte_all.shape[0],-1)
# yte_all = torch.hstack([F.pad(yte_all[n],(0,0,0,xte_all.shape[0]-yte_all[n].shape[0]),'constant',0) for n in range(len(yte_all))])


date_exp = 0
modfm,modhf = ModelList_FM[date_exp],ModelList_HF[date_exp]
# for n,(modfm,modhf) in enumerate(zip(ModelList_FM,ModelList_HF)):
GLM_FM = ioh5.load(modfm)
GLM_HF = ioh5.load(modhf)
W_FM = torch.from_numpy(GLM_FM['Cell_NN.0.weight'].copy().astype(np.float32))
W_HF = torch.from_numpy(GLM_HF['Cell_NN.0.weight'].copy().astype(np.float32))
bias_FM = torch.from_numpy(GLM_FM['Cell_NN.0.bias'].copy().astype(np.float32))
bias_HF = torch.from_numpy(GLM_HF['Cell_NN.0.bias'].copy().astype(np.float32))
# W_all.append(np.stack((W_FM,W_HF)))
# bias_all.append(np.stack((bias_FM,bias_HF)))
# W_all = np.concatenate(W_all,axis=1)
input_size = xte_all[date_exp].shape[-1]
output_size = yte_all[date_exp].shape[-1]

In [ ]:

date_exp = 0
dates_all = ['070921/J553RT' ,'101521/J559NC','102821/J570LT','110421/J569LT'] # '102621/J558NC' '062921/G6HCK1ALTRN',
args['date_ani'] = dates_all[date_exp]
params,file_dict,exp = load_params(MovModel,Kfold,args,debug=True)
data, train_idx_list, test_idx_list = load_train_test(file_dict, **params)
##### Set Train Test Splits #####
train_idx = train_idx_list[Kfold]
test_idx = test_idx_list[Kfold]
data = load_Kfold_data(data,train_idx,test_idx,params)
params, xtr, xtrm, xte, xtem, ytr, yte, shift_in_tr, shift_in_te, input_size, output_size, model_type, meanbias, model_move = load_GLM_data(data, params, train_idx, test_idx)
print('Model: {}, LinMix: {}, move_features: {}, Ncells: {}, train_shifter: {}'.format(params['MovModel'],params['LinMix'],params['move_features'],params['Ncells'],params['train_shifter']))
modfm,modhf = ModelList_FM[date_exp],ModelList_HF[date_exp]
model_info = params['date_ani2'] +'_'+'_'.join(modfm.name.split('_')[2:-1])

# for n,(modfm,modhf) in enumerate(zip(ModelList_FM,ModelList_HF)):
GLM_FM = ioh5.load(modfm)
GLM_HF = ioh5.load(modhf)
sta_FM = GLM_FM['Cell_NN.0.weight'].reshape((GLM_FM['Cell_NN.0.weight'].shape[0],params_FM['nt_glm_lag'],)+params_FM['nks'])
sta_HF = GLM_HF['Cell_NN.0.weight'].reshape((GLM_HF['Cell_NN.0.weight'].shape[0],params_FM['nt_glm_lag'],)+params_FM['nks'])
W_FM = torch.from_numpy(GLM_FM['Cell_NN.0.weight'].copy().astype(np.float32))
W_HF = torch.from_numpy(GLM_HF['Cell_NN.0.weight'].copy().astype(np.float32))
bias_FM = torch.from_numpy(GLM_FM['Cell_NN.0.bias'].copy().astype(np.float32))
bias_HF = torch.from_numpy(GLM_HF['Cell_NN.0.bias'].copy().astype(np.float32))
# W_all.append(np.stack((W_FM,W_HF)))
# bias_all.append(np.stack((bias_FM,bias_HF)))
# W_all = np.concatenate(W_all,axis=1)
input_size = xte_all[date_exp].shape[-1]
output_size = yte_all[date_exp].shape[-1]

r2_HFFM_sta = np.zeros(params_FM['Ncells'])
for celln in np.arange(params_FM['Ncells']):
    r2_HFFM_sta[celln] = np.corrcoef(sta_HF[celln].flatten(),sta_FM[celln].flatten())[0,1]

In [ ]:
l1 = LinVisNetwork(input_size,output_size,
                    move_features=params['move_features'],
                    train_shifter=params['train_shifter'], shift_hidden=50,
                    LinMix=params['LinMix'], device=device,)
l1.Cell_NN[0].weight.data = W_HF
l1.Cell_NN[0].bias.data = bias_FM
l1.to(device)
pred = l1(xte).cpu().detach().numpy()
actual = yte.cpu().detach().numpy()

In [ ]:
actual2_sm = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=0, arr=actual)[params['bin_length']:-params['bin_length']].T
pred2_sm = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=0, arr=pred)[params['bin_length']:-params['bin_length']].T
r2 = np.array([(np.corrcoef(pred2_sm[celln],actual2_sm[celln])[0, 1])**2 for celln in range(pred2_sm.shape[0])])
sorted_r2 = np.argsort(r2)[::-1] ##### Sorted indecies for R2
sorted_r2

In [ ]:
save_pdf = True
pdf_name = params_FM['fig_dir']/ ('HF_predicts_HF'+model_info+'2.pdf')
if save_pdf:
    pdf = PdfPages(pdf_name)
    fig, ax = plt.subplots(figsize=(5,5))
    lim0=0
    dlim = .2
    xlab = 'cc'
    count,edges = np.histogram(np.abs(r2_HFFM_sta),bins=np.arange(lim0,1,hbins))
    edges_mid = np.array([(edges[i]+edges[i+1])/2 for i in range(len(edges)-1)])
    ax.bar(edges_mid, count/len(r2_HFFM_sta),color=mod_clrs[n],width=hbins, alpha=1)
    ax.set_xticks(np.arange(lim0,1.2,dlim))
    ax.set_xticklabels(np.round(np.arange(lim0,1.2,dlim),decimals=1))
    ax.set_title('HF/FM CC')
    ax.set_xlabel(xlab)
    plt.tight_layout()
    pdf.savefig()
    plt.close()
for celln in tqdm(sorted_r2):
    fig = plt.figure(constrained_layout=False, figsize=(15,4))
    gs0 = gridspec.GridSpec(ncols=4, nrows=2, figure=fig,wspace=.5,hspace=.5)
    axs1 = np.array([fig.add_subplot(gs0[n,:1]) for n in range(2)])
    axs2 = fig.add_subplot(gs0[:,1:]) 

    ax = axs1[0]
    crange = np.max(np.abs(sta_FM[celln]))
    im = ax.imshow(sta_FM[celln,0],'RdBu_r',vmin=-crange,vmax=crange)
    cbar = add_colorbar(im)
    ax.set_title('FM RF')
    ax.axis('off')

    ax = axs1[1]
    crange = np.max(np.abs(sta_HF[celln]))
    im = ax.imshow(sta_HF[celln,0],'RdBu_r',vmin=-crange,vmax=crange)
    cbar = add_colorbar(im)
    ax.set_title('HF RF')
    ax.axis('off')

    ax = axs2
    ax.plot(actual2_sm[celln],c='k')
    ax.plot(pred2_sm[celln],c='r')
    ax.set_title('Cell:{}, $r^2$:{:.02f}, MSE:{:.02f}'.format(celln,r2[celln],np.mean((actual2_sm[celln]-pred2_sm[celln])**2)))
    # plt.tight_layout()

    if save_pdf:
        pdf.savefig()
        plt.close()
if save_pdf:
    pdf.close()


In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))
axs[0].scatter(r2_HM_FM[0,inds],r2_HM_FM[1,inds])
axs[0].set_xlabel('HF_W w/ HF_data $r^2$')
axs[0].set_ylabel('FM_W w/ HF_data $r^2$')
axs[0].set_ylim(-.05,.5)

axs[1].scatter(r2_HM_FM2[0,inds],r2_HM_FM2[1,inds])
axs[1].set_xlabel('FM_W w/ FM_data $r^2$')
axs[1].set_ylabel('HF_W w/ FM_data $r^2$')
axs[1].set_ylim(-.05,.5)
plt.tight_layout()
# fig.savefig(fig_dir/'SwappingWeights.png', facecolor='white', transparent=True, bbox_inches='tight')

# Shifter Network


## Save best shifter

In [ ]:
args = arg_parser(jupyter=True)
MovModel = 1
Kfold = 0
args['free_move'] = True
fm_dir = 'fm1' if args['prey_cap']==False else 'fm1_prey'
if args['free_move']:
    stim_type = fm_dir
else:
    stim_type = 'hf1_wn'  # 'fm1' #
    
dates_all = ['062921/G6HCK1ALTRN', '070921/J553RT' ,'101521/J559NC','102821/J570LT','110421/J569LT'] # '102621/J558NC' 
args['date_ani'] = dates_all[3]
# args['date_ani'] = '062921/G6HCK1ALTRN'    
# args['date_ani'] = '070921/J553RT'    
# args['date_ani'] = '101521/J559NC'
# args['date_ani'] = '102621/J558NC'
# args['date_ani'] = '110421/J569LT'
# yaml_path = sorted(list((Path(args['save_dir']).expanduser() / date_ani / stim_type / 'GLM_Network_NL/').rglob('*.yaml')))
# args['shiftn']=7
args['train_shifter']=True
args['NoL1'] = True
args['do_norm']=True
params_m1,file_dict,exp = load_params(MovModel,Kfold,args,debug=True)

data, train_idx_list, test_idx_list = load_train_test(file_dict, **params_m1)
ModelList = sorted(list(exp.save_dir.rglob('*_best.h5')))

if args['NoL1'] == True: 
    L1 = 'NoL1'
    ModelList = [ModelList[n] for n in range(len(ModelList)) if ('NoL1' in ModelList[n].as_posix()) &('Shifter' not in ModelList[n].as_posix())]
else: 
    L1 = 'withL1'
    ModelList = [ModelList[n] for n in range(len(ModelList)) if ('NoL1' not in ModelList[n].as_posix()) &('Shifter' not in ModelList[n].as_posix())]

##### Set Train Test Splits #####
Kfold = 0
train_idx = train_idx_list[Kfold]
test_idx = test_idx_list[Kfold]
data = load_Kfold_data(data,train_idx,test_idx,params_m1)
locals().update(data)

# Initialize movement combinations
titles = np.array(['Theta', 'Phi', 'Roll', 'Pitch'])

params, xtr, xtrm, xte, xtem, ytr, yte, shift_in_tr, shift_in_te, input_size, output_size, model_type, meanbias, model_move = load_GLM_data(data, params_m1, train_idx, test_idx)
print('Model: {}, LinMix: {}, move_features: {}, Ncells: {}, train_shifter: {}'.format(params['MovModel'],params['LinMix'],params['move_features'],params['Ncells'],params['train_shifter']))


In [ ]:
mean_thresh = np.nanmean(data['model_nsp']/params['model_dt'],axis=0)<1
f25,l75=int((data['model_nsp'].shape[0])*.5),int((data['model_nsp'].shape[0])*.5)
scaled_fr = (np.nanmean(data['model_nsp'][:f25], axis=0)/np.nanstd(data['model_nsp'][:f25], axis=0) - np.nanmean(data['model_nsp'][l75:], axis=0)/np.nanstd(data['model_nsp'][l75:], axis=0))/params['model_dt']
bad_cells = np.where((mean_thresh | (np.abs(scaled_fr)>4)))[0]
np.where(mean_thresh), np.where(np.abs(scaled_fr) > 4)


data['model_nsp'] = np.delete(data['model_nsp'], bad_cells, axis=1)
data['unit_nums'] = np.delete(data['unit_nums'], bad_cells, axis=0)


In [ ]:
(params['save_dir_hf']/'bad_cells.npy').exists()


In [ ]:


bin_length=40
smooth_nsp = np.apply_along_axis(lambda m: np.convolve(m, np.ones(bin_length), mode='same')/(bin_length * params['model_dt']), axis=0, arr=data['model_nsp'])[bin_length:-bin_length].T
smooth_mean = np.apply_along_axis(lambda m: np.convolve(m, np.ones(bin_length), mode='same')/(bin_length * params['model_dt']), axis=0, arr=data['model_nsp'][:,mean_thresh])[bin_length:-bin_length].T

In [ ]:
celln = 64
count[0],np.sum(count[1:]), np.sum(smooth_nsp[celln]==0),np.sum(smooth_nsp[celln]!=0)

In [ ]:
(count,bins,im) = plt.hist(smooth_nsp[42],np.arange(0,100))

In [ ]:
celln = bad_cells
fig,axs = plt.subplots(len(bad_cells),1,figsize=(10,len(bad_cells)*4))

for celln,ax in zip(bad_cells,axs):
    ax.plot(smooth_nsp[celln].T)
    ax.set_title('Celln:{}'.format(celln))

plt.tight_layout()
plt.show()

In [ ]:
save_dir_fm = params['save_dir_fm'] /'GLM_Network/MovModel1/version_0'
save_datafile = save_dir_fm/'GLM_{}_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_Kfold{:02d}_best.h5'.format(model_type, int(params['model_dt']*1000), params['nt_glm_lag'], params['MovModel'], 2000,Kfold)
GLM_CV = ioh5.load(save_datafile)


In [ ]:
list(save_dir_fm.rglob('*VisShifter*_best.h5'))

In [ ]:
plt.plot(np.nanmean(GLM_CV['loss_regcv'][0,:],axis=-1))
best_shifter = np.nanargmin(np.nanmean(GLM_CV['loss_regcv'][0,:],axis=-1))
best_shifter

In [ ]:
##### Load single shifter network and shift videos #####
reg_alph=a=0; l=best_shifter
print(best_shifter)
Kfold=0
shift_in=3
shift_hidden=50
shift_out=3
save_model_fm = params['save_model_shift'].parent.parent.parent / fm_dir / 'models/Shifter'

shifter_nn = nn.Sequential(
                nn.Linear(shift_in,shift_hidden),
                nn.Softplus(),
                nn.Linear(shift_hidden, shift_out))#.to(device)
model_name = 'GLM_{}_WC{}_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_alph{}_lam{}_Kfold{:01d}.pth'.format('Pytorch_VisShifter_NoL1','UC',int(.05*1000), 1, MovModel, 2000,a,l,Kfold)
# model_name = 'GLMShifter_WCUC_dt050_T01_MovModel1_NB5000_alph0_lam7.pth'
checkpoint = torch.load(save_model_fm/model_name)
pretrained_dict = {'.'.join(k.split('.')[1:]): v for k, v in checkpoint['model_state_dict'].items() if 'shift' in k}
shifter_nn.load_state_dict(pretrained_dict)

shift_out = shifter_nn(torch.from_numpy(model_move[:,(0,1,3)].astype(np.float32)))#.to(device))
shift = Affine(angle=torch.clamp(shift_out[:,-1],min=-45,max=45),translation=torch.clamp(shift_out[:,:2],min=-20,max=20))
vid_tensor=torch.from_numpy(data['model_vid_sm'][:,np.newaxis].astype(np.float32))#.to(device)
model_vid_sm_shift = shift(vid_tensor.contiguous()).detach().numpy().squeeze()
model_vid_sm_shift2 = {'model_vid_sm_shift':model_vid_sm_shift}
shift_vid_file = params['save_dir']/'ModelWC_shifted_dt{:03d}_MovModel{:d}.h5'.format(int(params['model_dt']*1000), 1)
ioh5.save(shift_vid_file,model_vid_sm_shift2)
print(shift_vid_file)

In [ ]:
t = 1000
fig, axs = plt.subplots(1,2, figsize=(10,5))
axs[0].imshow(data['model_vid_sm'][t], cmap='gray')
axs[1].imshow(model_vid_sm_shift[t], cmap='gray')
print(model_move[t,(0,1,3)],shift_out[t])

In [ ]:
sta_all =checkpoint['model_state_dict']['Cell_NN.0.weight'].reshape((params['Ncells'],params['nt_glm_lag'],)+params['nks']).cpu().detach().numpy()
celln = 0
fig,ax = plt.subplots()
crange = np.max(np.abs(sta_all[celln,0]))
im = ax.imshow(sta_all[celln,0],'RdBu_r',vmin=-crange,vmax=crange)
cbar = add_colorbar(im)
ax.axis('off')

## HF & FM Move Checks

In [ ]:
hftofm_imu_file = '/home/seuss/Research/SensoryMotorPred_Data/data/070921/J553RT/fm1/110521_J569LT_control_Rig2_hf_to_fm_transfer_imu.nc'
imu_data = xr.open_dataset(hftofm_imu_file)
accT = imu_data.IMU_data.sample # imu timestamps
acc_chans = imu_data.IMU_data # imu dample data
# raw gyro values
gx = np.array(acc_chans.sel(channel='gyro_x_raw'))
gy = np.array(acc_chans.sel(channel='gyro_y_raw'))
gz = np.array(acc_chans.sel(channel='gyro_z_raw'))
# gyro values in degrees
gx_deg = np.array(acc_chans.sel(channel='gyro_x'))
gy_deg = np.array(acc_chans.sel(channel='gyro_y'))
gz_deg = np.array(acc_chans.sel(channel='gyro_z'))
# pitch and roll in deg
medfiltbins = 11
groll = medfilt(np.array(acc_chans.sel(channel='roll')),medfiltbins)
gpitch = medfilt(np.array(acc_chans.sel(channel='pitch')),medfiltbins)
eye_file = '/home/seuss/Goeppert/freely_moving_ephys/ephys_recordings/070921/J553RT/hf1_wn/070921_J553RT_control_Rig2_hf1_wn_REYE.nc'
eye_data = xr.open_dataset(eye_file)

eye_params = eye_data['REYE_ellipse_params']

th = np.array((eye_params.sel(ellipse_params = 'theta')*180/np.pi))#-np.nanmean(eye_params.sel(ellipse_params = 'theta')))*180/3.14159)
phi = np.array((eye_params.sel(ellipse_params = 'phi')*180/np.pi))#-np.nanmean(eye_params.sel(ellipse_params = 'phi')))*180/3.14159)


In [ ]:
fmimu_file = list((params['data_dir'].parent / stim_type).glob('*imu.nc'))[0]
imu_data = xr.open_dataset(fmimu_file)
accT = imu_data.IMU_data.sample # imu timestamps
acc_chans = imu_data.IMU_data # imu dample data
# raw gyro values
gx = np.array(acc_chans.sel(channel='gyro_x_raw'))
gy = np.array(acc_chans.sel(channel='gyro_y_raw'))
gz = np.array(acc_chans.sel(channel='gyro_z_raw'))
# gyro values in degrees
gx_deg = np.array(acc_chans.sel(channel='gyro_x'))
gy_deg = np.array(acc_chans.sel(channel='gyro_y'))
gz_deg = np.array(acc_chans.sel(channel='gyro_z'))
# pitch and roll in deg
groll = medfilt(np.array(acc_chans.sel(channel='roll')),11)
gpitch = medfilt(np.array(acc_chans.sel(channel='pitch')),11)

eye_file = list((Path('~/Goeppert/freely_moving_ephys/ephys_recordings/').expanduser() / args['date_ani'] / stim_type).glob('*REYE.nc'))[0]
eye_data = xr.open_dataset(eye_file)
eyeT = eye_data.timestamps.copy()
eye_params = eye_data['REYE_ellipse_params']

th = np.array((eye_params.sel(ellipse_params = 'theta'))*180/np.pi)#-np.nanmean(eye_params.sel(ellipse_params = 'theta')))*180/3.14159)
phi = np.array((eye_params.sel(ellipse_params = 'phi'))*180/np.pi)#-np.nanmean(eye_params.sel(ellipse_params = 'phi')))*180/3.14159)
# fm_eye = {'avg_th': np.nanmean(th),'avg_phi': np.nanmean(phi)}
# ioh5.save(save_dir/'FM_AvgEye_dt{:03d}.h5'.format(int(params['model_dt']*1000)),fm_eye)
FM_move_avg = np.zeros((2,4))
FM_move_avg[:,0] = np.array([np.nanmean(th),np.nanstd(th)])
FM_move_avg[:,1] = np.array([np.nanmean(phi),np.nanstd(phi)])
FM_move_avg[:,2] = np.array([np.nanmean(groll),np.nanstd(groll)])
FM_move_avg[:,3] = np.array([np.nanmean(gpitch),np.nanstd(gpitch)])
# np.save(params['save_dir']/'FM_MovAvg_dt{:03d}.npy'.format(int(params['model_dt']*1000)),FM_move_avg)

In [ ]:
plt.hist(model_th,bins=100,alpha=.5)
plt.hist(model_phi,bins=100,alpha=.5)
plt.hist(model_roll,bins=100,alpha=.5)
plt.hist(model_pitch,bins=100,alpha=.5)

plt.show()

## Plot shift matricies

In [ ]:
reg_alph=a=0; l=13
GLM_CV = {}
GLM_CV['loss_regcv']      = np.zeros((params['nalph'], params['nlam'], output_size))
GLM_CV['pred_cv']         = np.zeros((params['nalph'], params['nlam'], output_size, xte.shape[0]), dtype=np.float32)
GLM_CV['out_cv']          = np.zeros((params['nalph'], params['nlam'], output_size, xtr.shape[0]), dtype=np.float32)
for l in np.arange(params['nlam']):
    l1 = LinVisNetwork(input_size,output_size,
                        reg_alph=params['alphas'][a],reg_alphm=params['alphas_m'][a],move_features=params['move_features'],
                        train_shifter=params['train_shifter'], shift_hidden=50,
                        LinMix=params['LinMix'], device=device,).to(device)
    model_name = 'GLM_{}_WC{}_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_alph{}_lam{}_Kfold{:01d}.pth'.format('Pytorch_VisShifter_NoL1','UC',int(params['model_dt']*1000), params['nt_glm_lag'], MovModel, 2000,a,l,Kfold)
    # model_name = 'GLMShifter_WC{}_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_pretrain_xyz.pth'.format(WC_type,int(params['model_dt']*1000), params['nt_glm_lag'] MovModel, Nbatches)
    checkpoint = torch.load(params['save_model_shift']/model_name)
    l1.load_state_dict(checkpoint['model_state_dict'])
    if (a==0) & (l==0):
        for name, p in l1.named_parameters():
            GLM_CV[name] = np.zeros((params['nalph'],params['nlam'],) + p.shape, dtype=np.float32)
    for name, p in l1.named_parameters():
        GLM_CV[name][a,l] = p.clone().cpu().detach().numpy()
    pred = l1(xte, xtem, shift_in_te)
    GLM_CV['loss_regcv'][a, l] = l1.loss(pred, yte).cpu().detach().numpy()
    GLM_CV['pred_cv'][a, l] = pred.detach().cpu().numpy().squeeze().T
    out = l1(xtr, xtrm, shift_in_tr)
    GLM_CV['out_cv'][a, l] = out.detach().cpu().numpy().squeeze().T
malph, mlam, cellnum = np.where(GLM_CV['loss_regcv'] == np.nanmin(GLM_CV['loss_regcv'], axis=(0, 1), keepdims=True))
cellnum, m_cinds = np.unique(cellnum, return_index=True)
malph = malph[m_cinds]
mlam = mlam[m_cinds]
sortinds = cellnum.argsort()
cellnum = cellnum[sortinds]
malph = malph[sortinds]
mlam = mlam[sortinds]

GLM_Data = {}
for key in GLM_CV.keys():
    if 'shifter' in key:
        GLM_Data[key] = GLM_CV[key][malph,mlam]  
        # GLM_Data[key] = GLM_CV[key][0,23]
    else:
        GLM_Data[key] = GLM_CV[key][malph,mlam,cellnum]
        # GLM_Data[key] = GLM_CV[key][0,23,:]
GLM_Data['loss_regcv'] = GLM_CV['loss_regcv']
sp_smooth = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=0, arr=data['test_nsp'])[params['bin_length']:-params['bin_length']]
pred_smooth = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=1, arr=GLM_CV['pred_cv'][malph,mlam,cellnum])[:,params['bin_length']:-params['bin_length']].T
# pred_smooth = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=1, arr=GLM_CV['pred_cv'][0,23,:])[:,params['bin_length']:-params['bin_length']].T
GLM_Data['r2_test']= np.array([(np.corrcoef(sp_smooth[:,celln],pred_smooth[:,celln])[0, 1])**2 for celln in range(params['Ncells'])])
save_datafile = params['save_dir']/'GLM_{}_VisMov_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_Kfold{:02d}.h5'.format(model_type, int(params['model_dt']*1000), params['nt_glm_lag'], params['MovModel'], params['Nepochs'],Kfold)
# ioh5.save(save_datafile, GLM_Data)

In [ ]:
celln = 0
plt.plot(sp_smooth[:,celln])
plt.plot(pred_smooth[:,celln])
GLM_Data['r2_test'][celln]

In [ ]:
t = 1000
fig, axs = plt.subplots(1,2, figsize=(10,5))
axs[0].imshow(data['model_vid_sm'][t], cmap='gray')
axs[1].imshow(model_vid_sm_shift[t], cmap='gray')
print(shift_out[t])

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(10,5))
axs[0].plot(np.nanmean(GLM_Data['loss_regcv'][0,:],axis=-1))
axs[1].plot(GLM_Data['r2_test'])

np.nanargmin(np.nanmean(GLM_Data['loss_regcv'][0,:],axis=-1)), np.nanargmax(GLM_Data['r2_test'])


In [ ]:
best_shift = 'GLM_{}_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_Kfold{:01d}.pth'.format('Pytorch_BestShift',int(params['model_dt']*1000), params['nt_glm_lag'], MovModel, 5000,Kfold)
GLM_shift = ioh5.load('/home/seuss/Research/SensoryMotorPred_Data/data/070921/J553RT/fm1/GLM_Pytorch_VisShifter_NoL1_VisMov_dt050_T01_MovModel1_NB5000_Kfold00.h5')

In [ ]:
sta_all = GLM_Data['Cell_NN.0.weight'].reshape((params['Ncells'],params['nt_glm_lag'],)+params['nks'])
celln = 20
fig,ax = plt.subplots()
crange = np.max(np.abs(sta_all[celln,0]))
im = ax.imshow(sta_all[celln,0],'RdBu_r',vmin=-crange,vmax=crange)
cbar = add_colorbar(im)
ax.axis('off')

In [ ]:
a=0;l = 15
l1 = LinVisNetwork(input_size,output_size,
                        reg_alph=params['alphas'][a],reg_alphm=params['alphas_m'][a],move_features=params['move_features'],
                        train_shifter=params['train_shifter'], shift_hidden=50,
                        LinMix=params['LinMix'], device=device,).to(device)
model_name = 'GLM_{}_WC{}_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_alph{}_lam{}_Kfold{:01d}.pth'.format('Pytorch_VisShifter_NoL1','UC',int(params['model_dt']*1000), params['nt_glm_lag'], MovModel, 2000,a,l,Kfold)
# model_name = 'GLMShifter_WC{}_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_pretrain_xyz.pth'.format(WC_type,int(params['model_dt']*1000), params['nt_glm_lag'] MovModel, Nbatches)
checkpoint = torch.load(params['save_model_shift']/model_name)
l1.load_state_dict(checkpoint['model_state_dict'])
shift_out = shifter_nn(torch.from_numpy(model_move[:,(0,1,3)].astype(np.float32)))#.to(device))
shift_out[:,-1] = torch.clamp(shift_out[:,-1],min=-45,max=45)
shift_out[:,:2] = torch.clamp(shift_out[:,:2],min=-20,max=20)
shift = Affine(angle=shift_out[:,-1],translation=shift_out[:,:2])
vid_tensor=torch.from_numpy(data['model_vid_sm'][:,np.newaxis].astype(np.float32))#.to(device)
model_vid_sm_shift = shift(vid_tensor.contiguous()).detach().numpy().squeeze()



In [ ]:
FM_move_avg

In [ ]:
# ang_sweepx,ang_sweepy,ang_sweepz = np.meshgrid(np.arange(-40,40),np.arange(-40,40),np.arange(-40,40),sparse=False,indexing='ij')
FM_move_avg = np.load(params['save_dir_fm']/'FM_MovAvg_dt{:03d}.npy'.format(int(params['model_dt']*1000)))
th_range = 40/FM_move_avg[1, 0]
phi_range = 40/FM_move_avg[1, 1]
pitch_range = 40/FM_move_avg[1, 2]


In [ ]:
l1.cpu()
FM_move_avg = np.load(params['save_dir_fm']/'FM_MovAvg_dt{:03d}.npy'.format(int(params['model_dt']*1000)))
th_range = 40/FM_move_avg[1,0]
phi_range = 40/FM_move_avg[1,1]
pitch_range = 40/FM_move_avg[1,2]
ang_sweepx,ang_sweepy,ang_sweepz = np.meshgrid(np.linspace(-th_range,th_range,81),np.linspace(-phi_range,phi_range,81),np.linspace(-pitch_range,pitch_range,81),sparse=False,indexing='ij')
# ang_sweepx,ang_sweepy,ang_sweepz = np.meshgrid(np.arange(-40,40),np.arange(-40,40),np.arange(-40,40),sparse=False,indexing='ij')

shift_mat = np.zeros((3,) + ang_sweepx.shape)
for i in range(ang_sweepx.shape[0]):
    for j in range(ang_sweepy.shape[1]):
        ang_sweep = torch.from_numpy(np.vstack((ang_sweepx[i,j,:],ang_sweepy[i,j,:],ang_sweepz[i,j,:])).astype(np.float32).T)
        shift_vec = shifter_nn(ang_sweep).detach().cpu().numpy()
        shift_mat[0,i,j] = shift_vec[:,0] #np.clip(shift_vec[:,0],a_min=-15,a_max=15)
        shift_mat[1,i,j] = shift_vec[:,1] #np.clip(shift_vec[:,1],a_min=-15,a_max=15)
        shift_mat[2,i,j] = shift_vec[:,2] #np.clip(shift_vec[:,2],a_min=-30,a_max=30)

    
fig, ax = plt.subplots(1,4,figsize=(20,5))
shift_titles = [r'$dx$',r'$dy$',r'$d\alpha,\phi=0$',r'$d\alpha,\theta=0$']
ticks=np.arange(0,90,20)
ticklabels=np.arange(-40,50,20)
shift_matshow=np.stack((shift_mat[0,:,:,40].T,shift_mat[1,:,:,40].T,shift_mat[2,:,40,:].T,shift_mat[2,40,:,:].T))
crange_list = np.stack((np.max(np.abs(shift_mat[:2])),np.max(np.abs(shift_mat[:2])), np.max(np.abs(shift_mat[2])), np.max(np.abs(shift_mat[2]))))
for n in range(4):
    im1=ax[n].imshow(shift_matshow[n],vmin=-crange_list[n], vmax=crange_list[n], origin='lower', cmap='RdBu_r')
    cbar1 = add_colorbar(im1)
    ax[n].set_xticks(ticks)
    ax[n].set_xticklabels(ticklabels)
    ax[n].set_yticks(ticks)
    ax[n].set_yticklabels(ticklabels)
    ax[n].set_xlabel(r'$\theta$')
    ax[n].set_ylabel(r'$\phi$')
    ax[n].set_title(shift_titles[n])

plt.tight_layout()


## Shifter Training

In [ ]:
l = 15
track_all = True
args['Nepochs'] = 2000
params, file_dict, exp = load_params(1,Kfold,args)
# params['lambdas'] = np.hstack((0, np.logspace(-2, 3, 20)))
data, train_idx_list, test_idx_list = load_train_test(file_dict, **params)
train_idx = train_idx_list[Kfold]
test_idx = test_idx_list[Kfold]
data = load_Kfold_data(data,train_idx,test_idx,params)

params, xtr, xtrm, xte, xtem, ytr, yte, shift_in_tr, shift_in_te, input_size, output_size, model_type, meanbias, model_move = load_GLM_data(data, params, train_idx, test_idx)
print('Model: {}, LinMix: {}, move_features: {}'.format(params['MovModel'],params['LinMix'], params['move_features']))

l1,optimizer,scheduler = get_model(input_size, output_size, meanbias, params['MovModel'], device, l, 0, params)

vloss_trace, tloss_trace, l1, optimizer, scheduler, Epoch_GLM = train_model(xtr,xte,xtrm,xtem,shift_in_tr,shift_in_te,ytr,yte,params['Nepochs'],l1,optimizer,scheduler,pbar=None,track_all=track_all)
pred = l1(xte, xtem, shift_in_te)
loss_regcv = l1.loss(pred, yte).cpu().detach().numpy()
pred_cv = pred.detach().cpu().numpy().squeeze().T
out = l1(xtr, xtrm, shift_in_tr)
out_cv = out.detach().cpu().numpy().squeeze().T
sp_smooth = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=0, arr=data['test_nsp'])[params['bin_length']:-params['bin_length']]
pred_smooth = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=1, arr=pred_cv)[:,params['bin_length']:-params['bin_length']].T
out_smooth = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=1, arr=out_cv)[:,params['bin_length']:-params['bin_length']].T
r2_test = np.array([(np.corrcoef(sp_smooth[:,celln],pred_smooth[:,celln])[0, 1])**2 for celln in range(params['Ncells'])])


In [ ]:
plt.plot(r2_test)

In [ ]:
celln = np.nanargmax(r2_test)
plt.plot(sp_smooth[:,celln])
plt.plot(pred_smooth[:,celln])
celln,r2_test[celln]

In [ ]:
celln = 39
rf_all = l1.Cell_NN[0].weight.cpu().detach().numpy().reshape((params['Ncells'],params['nt_glm_lag'],)+params['nks'])
crange = np.max(np.abs(rf_all[celln]))
fig, axs = plt.subplots(figsize=(5,5))
ax = axs
im = ax.imshow(rf_all[celln,0],'RdBu_r',vmin=-crange,vmax=crange)
cbar = add_colorbar(im)
ax.set_title('FM STA')

In [ ]:
Epoch_GLM.keys(),Epoch_GLM['shifter_nn.0.weight'].shape

In [ ]:
# plt.plot(np.linalg.norm(Epoch_GLM['Cell_NN.0.weight'],axis=(1,2)))
# plt.plot(np.linalg.norm(Epoch_GLM['Cell_NN.0.bias'],axis=1))
plt.plot(np.linalg.norm(Epoch_GLM['shifter_nn.0.weight'],axis=(1,2)))
# plt.plot(np.linalg.norm(Epoch_GLM['shifter_nn.0.bias'],axis=(1)))
# plt.plot(np.linalg.norm(Epoch_GLM['shifter_nn.2.weight'],axis=(1,2)))
# plt.plot(np.linalg.norm(Epoch_GLM['shifter_nn.2.bias'],axis=(1)))

## Animation world cam

In [ ]:
@ray.remote
def make_plt_im(t, model_vid_sm, model_vid_sm_shift, pbar: ActorHandle,):  #
    fig, axs = plt.subplots(1,2, figsize=(10,5))
    axs[0].imshow(model_vid_sm[t], cmap='gray')
    axs[0].set_title('Uncorrected, Frame={}'.format(t))
    axs[1].imshow(model_vid_sm_shift[t], cmap='gray')
    axs[1].set_title('Corrected, Frame={}'.format(t))
    for ax in axs:
        ax.axis('off')
    plt.tight_layout()

    width, height = fig.get_size_inches() * fig.get_dpi()
    fig.canvas.draw()       # draw the canvas, cache the renderer
    images = np.frombuffer(fig.canvas.tostring_rgb(),
                        dtype='uint8').reshape(int(height), int(width), 3)
    
    plt.close()
    pbar.update.remote(1)
    return images


In [ ]:
mpl.use('agg')

start = time.time()
##### initialize time points for animation and progressbar #####
t = 0
time_range = np.arange(10000)# model_vid_sm_shift.shape[0])
num_ticks = np.size(time_range)
pb = ProgressBar(num_ticks)
actor = pb.actor

##### Put large arrays into shared memory #####
model_vid_sm_shfit_r = ray.put(model_vid_sm_shift)
model_vid_sm_r = ray.put(model_vid_sm)

##### Loop over parameters appending process ids #####
result_ids = []
for t in time_range:
    result_ids.append(make_plt_im.remote(t, model_vid_sm_r, model_vid_sm_shfit_r, actor))

##### pring progressbar and get results #####
pb.print_until_done()
results_p = ray.get(result_ids)
images = np.stack([results_p[i] for i in range(len(results_p))])

##### Make video with opencv #####
aniname = date_ani2+ '_WorldCamCorrection_dt{:03d}.mp4'.format(int(.05*1000))



vid_name = fig_dir / aniname
FPS = int(1/params['model_dt'])
out = cv2.VideoWriter(vid_name.as_posix(), cv2.VideoWriter_fourcc(*'mp4v'), FPS, (images.shape[-2], images.shape[-3]))

for fm in tqdm(range(images.shape[0])):
    out.write(cv2.cvtColor(images[fm], cv2.COLOR_BGR2RGB))
out.release()
print('Making Animation {}: {}'.format(aniname, time.time()-start))
del results_p, pb, model_vid_sm_r, model_vid_sm_shfit_r
gc.collect()


In [ ]:

##### Make video with opencv #####
aniname = date_ani2 + '_WorldCamCorrection_dt{:03d}.mp4'.format(int(.05*1000))



vid_name = fig_dir / aniname
FPS = int(1/params['model_dt'])
out = cv2.VideoWriter(vid_name.as_posix(), cv2.VideoWriter_fourcc(*'mp4v'), FPS, (images.shape[-2], images.shape[-3]))

for fm in tqdm(range(images.shape[0])):
    out.write(cv2.cvtColor(images[fm], cv2.COLOR_BGR2RGB))
out.release()
print('Making Animation {}: {}'.format(aniname, time.time()-start))
del results_p, pb, model_vid_sm_r, model_vid_sm_shfit_r
gc.collect()

# Paper Figures

In [ ]:
paper_fig_dir = (params['fig_dir'].parent.parent.parent / 'PaperFigs')
paper_fig_dir.mkdir(parents=True, exist_ok=True)
titles = [r'$\theta$',r'$\phi$',r'$\rho$',r'$\omega$']
move_clrs = ['blue','orange','green','red']
q_clrs = ["#f72585","#7209b7","#3f37c9","#4cc9f0"]
mod_clrs = ["#003d36", "#a38d00", "#ff3b05", "#70a0ff"]#["#136f63", "#e0ca3c", "#f34213", "#80a4ed"]


In [ ]:
celltype_dir = params['data_dir'].parent.parent.parent.parent / 'batch_files/120221_hffm'
celltype_file = celltype_dir / 'pooled_ephys_population_update_120621.pickle'
celltype = pickle.load(open(celltype_file,'rb'))

Kfold = 0
args['free_move']=True
mod_titles = ['Mot','Vis','Add','Mul','HF']
mod_clrs = ["#136f63","#e0ca3c","#f34213","#80a4ed"]
dates_all = ['070921/J553RT' ,'101521/J559NC','102821/J570LT','110421/J569LT'] # '102621/J558NC' 
ModelList_all,test_std,tuning_sig_all,NCells_all,bad_cells_all,test_nsp_all,hf_nsp_all = [],[], [], [], [], [], []
for d,date_ani in enumerate(dates_all):
    args['free_move'] = True
    args['date_ani'] = date_ani
    args['NoL1']=False
    args['Nepochs'] = 10000
    params,_,_ = load_params(1,Kfold,args,debug=True)
    params['nt_glm_lag']=3
    date_ani_dir = params['save_dir'].parent.parent.parent/date_ani
    if args['NoL1'] == True: 
        L1 = 'NoL1'
        mod_name = '*{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    else: 
        L1 = 'withL1'
        mod_name = '*{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    ModelList = np.array(sorted(list(set(date_ani_dir.rglob(mod_name+'.h5')) - set(date_ani_dir.rglob('*VisNoShifter'+mod_name+'.h5')))))
    ModelList_all.append(ModelList)
    bad_cells = np.load(params['save_dir_hf']/'bad_cells.npy')
    bad_cells_all.append(bad_cells)
    data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')
    test_nsp_all.append(data['test_nsp'])
    test_std.append(np.var(data['test_nsp'],axis=0))
    tuning_sig_all.append(tuning_sig2)
    NCells_all.append(len(tuning_sig2))
    args['free_move'] = False
    params,_,_ = load_params(0,Kfold,args,debug=True)
    data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')
    hf_nsp_all.append(data['test_nsp'])

ModelList_all = np.stack(ModelList_all)
test_std=np.hstack(test_std)
tuning_sig_all = np.hstack(tuning_sig_all)


In [ ]:

sf=4
params['nt_glm_lag']=3
date_ani2 = [dates_all[n].replace('/','_') for n in range(len(dates_all))]
All_data = {date_ani2[n]:{} for n in range(len(date_ani2))}
for da, date_ani in enumerate(date_ani2):
    All_data[date_ani]['actual_meanfr'] = np.mean(test_nsp_all[da],axis=0)/params['model_dt']
    All_data[date_ani]['actual_smooth'] = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=0, arr=test_nsp_all[da])[params['bin_length']:-params['bin_length']]
    All_data[date_ani]['actual_HF_smooth'] = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=0, arr=hf_nsp_all[da])[params['bin_length']:-params['bin_length']]
    for ModelRun,fmod in enumerate(ModelList_all[da]):
        GLM_Data = ioh5.load(fmod)
        All_data[date_ani][mod_titles[ModelRun]+'_pred_smooth'] = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=1, arr=GLM_Data['pred_cv'].copy())[:,params['bin_length']:-params['bin_length']].T
        All_data[date_ani][mod_titles[ModelRun]+'_r2_test'] = GLM_Data['r2_test'].copy()
        All_data[date_ani][mod_titles[ModelRun]+'_vloss_all'] = GLM_Data['vloss_trace_all'].copy()
        All_data[date_ani][mod_titles[ModelRun]+'_vloss_trace'] = np.nanmean(GLM_Data['vloss_trace_all'][:,-10:],axis=1)/np.nanvar(test_nsp_all[da],axis=0)
        if (ModelRun == 1) | (ModelRun == 4):
            All_data[date_ani][mod_titles[ModelRun]+'_loss_regcv'] = GLM_Data['loss_regcv'].copy()
            RF_all = GLM_Data['Cell_NN.0.weight'].reshape((GLM_Data['Cell_NN.0.weight'].shape[0],params['nt_glm_lag'],)+params['nks'])
            RF_all_up = np.zeros((RF_all.shape[:2] + ( sf*(RF_all.shape[-2]), sf*(RF_all.shape[-1]))))
            for celln in range(RF_all.shape[0]):
                for t in range(RF_all.shape[1]):
                    RF_all_up[celln,t] = cv2.resize(RF_all[celln,t], (sf*(RF_all.shape[-1]), sf*(RF_all.shape[-2])))
            All_data[date_ani][mod_titles[ModelRun]+'_rf_up'] = RF_all_up
            All_data[date_ani][mod_titles[ModelRun]+'_rf_all'] = RF_all
        if (ModelRun == 2) | (ModelRun==3):
            All_data[date_ani][mod_titles[ModelRun]+'_moveW'] = GLM_Data['posNN.Layer0.weight'].copy()
        if (ModelRun == 4):
            r2_FMHF_RF = np.zeros(All_data[date_ani][mod_titles[1]+'_rf_all'].shape[0])
            for celln in np.arange(All_data[date_ani][mod_titles[1]+'_rf_all'].shape[0]):
                r2_FMHF_RF[celln] = np.corrcoef(All_data[date_ani][mod_titles[1]+'_rf_all'][celln,2].flatten(),All_data[date_ani][mod_titles[-1]+'_rf_all'][celln,2].flatten())[0,1]
            All_data[date_ani][mod_titles[ModelRun]+'_FMHF_r2'] = r2_FMHF_RF

celltypes_all = []
for n,date_exp in enumerate([2,4,7,9]):
    celltype2 = celltype[celltype['session']==celltype['session'].unique()[date_exp]].reset_index(drop=True)
    celltype2 = celltype2.drop(bad_cells_all[n])
    celltypes_all.append([celltype2['exc_or_inh'] == 'exc',celltype2['hf1_wn_depth_from_layer5']])
celltypes_all = np.hstack(celltypes_all)

da = 0
fontsize=24
args['free_move']=True
args['date_ani'] = dates_all[da]
params,_,_ = load_params(0,Kfold,args,debug=True)
data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')


## Figure 2

### Shifter Matricies

In [ ]:
save_dir_fm = params['save_dir_fm'] /'GLM_Network/MovModel1/version_0'

save_datafile = save_dir_fm/'GLM_Pytorch_VisShifter_NoL1_dt050_T01_MovModel1_NB5000_Kfold00.h5'
GLM_CV = ioh5.load(save_datafile)

best_shifter = np.nanargmin(np.nanmean(GLM_CV['loss_regcv'][0,:],axis=-1))

##### Load single shifter network and shift videos #####
reg_alph=a=0; l=best_shifter
Kfold=0
shift_in=3
shift_hidden=20
shift_out=3
save_model_fm =params['save_dir'] / 'models/Shifter'

shifter_nn = nn.Sequential(
                nn.Linear(shift_in,shift_hidden),
                nn.Softplus(),
                nn.Linear(shift_hidden, shift_out))#.to(device)
model_name = 'GLM_{}_WC{}_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_alph{}_lam{}_Kfold{:01d}.pth'.format('Pytorch_VisShifter_NoL1','UC',int(.05*1000), 1, MovModel, 5000,a,l,Kfold)
# model_name = 'GLMShifter_WCUC_dt050_T01_MovModel1_NB5000_alph0_lam7.pth'
checkpoint = torch.load(save_model_fm/model_name)
pretrained_dict = {'.'.join(k.split('.')[1:]): v for k, v in checkpoint['model_state_dict'].items() if 'shift' in k}
shifter_nn.load_state_dict(pretrained_dict)

FM_move_avg = np.load(params['save_dir_fm']/'FM_MovAvg_dt{:03d}.npy'.format(int(params['model_dt']*1000)))
th_range = 40/FM_move_avg[1,0]
phi_range = 40/FM_move_avg[1,1]
pitch_range = 40/FM_move_avg[1,2]
ang_sweepx,ang_sweepy,ang_sweepz = np.meshgrid(np.linspace(-th_range,th_range,81),np.linspace(-phi_range,phi_range,81),np.linspace(-pitch_range,pitch_range,81),sparse=False,indexing='ij')
# ang_sweepx,ang_sweepy,ang_sweepz = np.meshgrid(np.arange(-40,40),np.arange(-40,40),np.arange(-40,40),sparse=False,indexing='ij')

shift_mat = np.zeros((3,) + ang_sweepx.shape)
for i in range(ang_sweepx.shape[0]):
    for j in range(ang_sweepy.shape[1]):
        ang_sweep = torch.from_numpy(np.vstack((ang_sweepx[i,j,:],ang_sweepy[i,j,:],ang_sweepz[i,j,:])).astype(np.float32).T)
        shift_vec = shifter_nn(ang_sweep).detach().cpu().numpy()
        shift_mat[0,i,j] = shift_vec[:,0] #np.clip(shift_vec[:,0],a_min=-15,a_max=15)
        shift_mat[1,i,j] = shift_vec[:,1] #np.clip(shift_vec[:,1],a_min=-15,a_max=15)
        shift_mat[2,i,j] = shift_vec[:,2] #np.clip(shift_vec[:,2],a_min=-30,a_max=30)

    
fig, axs = plt.subplots(1,3,figsize=(20,5))
shift_titles = [r'$dx$',r'$dy$',r'$d\alpha,\phi=0$',r'$d\alpha,\theta=0$']
cbar_label = ['pixels','pixels','deg','deg']
ticks=np.arange(0,90,20)
ticklabels=np.arange(-40,50,20)
shift_matshow=np.stack((shift_mat[0,:,:,40].T,shift_mat[1,:,:,40].T,shift_mat[2,:,40,:].T,shift_mat[2,40,:,:].T))
crange_list = np.stack((np.max(np.abs(shift_mat[:2])),np.max(np.abs(shift_mat[:2])), np.max(np.abs(shift_mat[2])), np.max(np.abs(shift_mat[2]))))
for n in range(2):
    ax = axs[n]
    im1=ax.imshow(shift_matshow[n],vmin=-crange_list[n], vmax=crange_list[n], origin='lower', cmap='RdBu_r')
    cbar1 = add_colorbar(im1)
    cbar1.set_label(cbar_label[n],fontsize=fontsize)
    ax.set_xticks(ticks)
    ax.set_xticklabels(ticklabels)
    ax.set_yticks(ticks)
    ax.set_yticklabels(ticklabels)
    ax.set_xlabel(r'$\theta$',fontsize=fontsize)
    ax.set_ylabel(r'$\phi$',fontsize=fontsize)
    ax.set_title(shift_titles[n],fontsize=fontsize)
ax = axs[2]
ax.plot(FM_move_avg[1, 2]*np.linspace(-pitch_range,pitch_range,81), shift_mat[2, 40, 40, :], 'k')
ax.set_xlabel(r'$\omega$',fontsize=fontsize)
ax.set_ylabel(r'$d\alpha$',fontsize=fontsize)
ax.set_xticks(ticklabels)
ax.set_xticklabels(ticklabels)
ax.set_yticks([-40,-20,0,20,40])
ax.set_yticklabels([-40, -20, 0, 20, 40])
ax.axis('square')
ax.set_ylim(-45, 45)


plt.tight_layout()


In [ ]:
FM_move_avg[1, 2]*np.linspace(-pitch_range, pitch_range, 81)


### No Shifter RFs

In [ ]:

Kfold = 0
args['free_move']=True
mod_titles = ['Mot','Vis','Add','Mul','HF']
mod_clrs = ["#136f63","#e0ca3c","#f34213","#80a4ed"]
dates_all = ['070921/J553RT' ,'101521/J559NC','102821/J570LT','110421/J569LT'] # '102621/J558NC' 
ModelList_NoSh,NCells_all,bad_cells_all,test_nsp_all = [],[],[],[]
for d,date_ani in enumerate(dates_all):
    args['free_move'] = True
    args['date_ani'] = date_ani
    args['NoL1']=False
    args['Nepochs'] = 10000
    params,_,_ = load_params(1,Kfold,args,debug=True)
    params['nt_glm_lag']=3
    date_ani_dir = params['save_dir'].parent.parent.parent/date_ani
    if args['NoL1'] == True: 
        L1 = 'NoL1'
        mod_name = '*VisNoShifter_{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    else: 
        L1 = 'withL1'
        mod_name = '*VisNoShifter_{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    ModelList = np.array(sorted(list(date_ani_dir.rglob(mod_name+'.h5'))))
    ModelList_NoSh.append(ModelList)
    bad_cells = np.load(params['save_dir_hf']/'bad_cells.npy')
    bad_cells_all.append(bad_cells)
    data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')
    test_nsp_all.append(data['test_nsp'])
    NCells_all.append(data['test_nsp'].shape[-1])
ModelList_NoSh = np.stack(ModelList_NoSh)


In [ ]:
NoSh_titles=['VisNoSh']
sf=4
params['nt_glm_lag']=3
date_ani2 = [dates_all[n].replace('/','_') for n in range(len(dates_all))]
NoSh_data = {date_ani2[n]:{} for n in range(len(date_ani2))}
for da, date_ani in enumerate(date_ani2):
    NoSh_data[date_ani]['actual_meanfr'] = np.mean(test_nsp_all[da],axis=0)/params['model_dt']
    NoSh_data[date_ani]['actual_smooth'] = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=0, arr=test_nsp_all[da])[params['bin_length']:-params['bin_length']]
    # NoSh_data[date_ani]['actual_HF_smooth'] = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=0, arr=hf_nsp_all[da])[params['bin_length']:-params['bin_length']]
    for ModelRun,fmod in enumerate(ModelList_NoSh[da]):
        GLM_Data = ioh5.load(fmod)
        NoSh_data[date_ani][NoSh_titles[ModelRun]+'_pred_smooth'] = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=1, arr=GLM_Data['pred_cv'].copy())[:,params['bin_length']:-params['bin_length']].T
        NoSh_data[date_ani][NoSh_titles[ModelRun]+'_r2_test'] = GLM_Data['r2_test'].copy()
        NoSh_data[date_ani][NoSh_titles[ModelRun]+'_vloss_all'] = GLM_Data['vloss_trace_all'].copy()
        NoSh_data[date_ani][NoSh_titles[ModelRun]+'_vloss_trace'] = np.nanmean(GLM_Data['vloss_trace_all'][:,-10:],axis=1)/np.nanvar(test_nsp_all[da],axis=0)
    
        NoSh_data[date_ani][NoSh_titles[ModelRun]+'_loss_regcv'] = GLM_Data['loss_regcv'].copy()
        RF_all = GLM_Data['Cell_NN.0.weight'].reshape((GLM_Data['Cell_NN.0.weight'].shape[0],params['nt_glm_lag'],)+params['nks'])
        RF_all_up = np.zeros((RF_all.shape[:2] + ( sf*(RF_all.shape[-2]), sf*(RF_all.shape[-1]))))
        for celln in range(RF_all.shape[0]):
            for t in range(RF_all.shape[1]):
                RF_all_up[celln,t] = cv2.resize(RF_all[celln,t], (sf*(RF_all.shape[-1]), sf*(RF_all.shape[-2])))
        NoSh_data[date_ani][NoSh_titles[ModelRun]+'_rf_up'] = RF_all_up
        NoSh_data[date_ani][NoSh_titles[ModelRun]+'_rf_all'] = RF_all

celltypes_all = []
for n,date_exp in enumerate([2,4,7,9]):
    celltype2 = celltype[celltype['session']==celltype['session'].unique()[date_exp]].reset_index(drop=True)
    celltype2 = celltype2.drop(bad_cells_all[n])
    celltypes_all.append([celltype2['exc_or_inh'] == 'exc',celltype2['hf1_wn_depth_from_layer5']])
celltypes_all = np.hstack(celltypes_all)

da = 0
fontsize=24
args['free_move']=True
args['date_ani'] = dates_all[da]
params,_,_ = load_params(0,Kfold,args,debug=True)
data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')


In [ ]:
# cells = [25,70,90,117] # 49
cells = [22]#,25,73,74] #np.arange(18)
num_cells = len(cells)

fig1 = plt.figure(constrained_layout=False, figsize=(int(5*params['nt_glm_lag']),int(3*(num_cells*2))))
spec2 = gridspec.GridSpec(ncols=1, nrows=num_cells*2, figure=fig1, wspace=.05,hspace=.1)
axs1 = np.array([fig1.add_subplot(spec2[n,m]) for n in range(num_cells*2) for m in range(1)]).reshape(num_cells*2,1)
lag = 2
for n, cell in enumerate(cells):
    crange2 = np.max(np.abs(NoSh_data[date_ani2[da]]['VisNoSh_rf_up'][cell]))
    im2 = axs1[n,0].imshow(NoSh_data[date_ani2[da]]['VisNoSh_rf_up'][cell,lag],'RdBu_r', vmin=-crange2, vmax=crange2)
    axs1[n,0].set_ylabel('Shift Off',fontsize=fontsize)
    crange2 = np.max(np.abs(All_data[date_ani2[da]]['Vis_rf_up'][cell]))
    im2 = axs1[n+1,0].imshow(All_data[date_ani2[da]]['Vis_rf_up'][cell,lag],'RdBu_r', vmin=-crange2, vmax=crange2)
    axs1[n+1,0].set_ylabel('Shift On',fontsize=fontsize)
cbar2 = fig1.colorbar(im2, ax=axs1.ravel().tolist())
cbar2.set_ticks([-crange2, crange2])
cbar2.set_ticklabels(['Dark', 'Light'])
cbar2.ax.tick_params(labelsize=fontsize)

for ax in axs1.flat:
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)
        ax.spines[axis].set_visible(True)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])

# fig1.savefig(paper_fig_dir/('NoShifter_FM_RFs_'+model_info+'.pdf'), facecolor='white', transparent=True, bbox_inches='tight',dpi=300)
# print('Saved to: {}'.format(FigPath_SFN/'FM_RFs.png'))

### R2 for Shift/Non-shift

In [ ]:

args['NoL1'] = False
if args['NoL1'] == True: 
    L1 = 'NoL1'
    mod_name = '*Vis_{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_shuffled_best'.format(L1,int(params_FM['model_dt']*1000), 1, params_FM['Nepochs'],Kfold)
else: 
    L1 = 'withL1'
    mod_name = '*Vis_{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_shuffled_best'.format(L1,int(params_FM['model_dt']*1000), 1, params_FM['Nepochs'],Kfold)
ModelList = sorted(list(exp.save_dir.rglob(mod_name+'.h5')))


GLM_Shuff = ioh5.load(ModelList[0])
malph = 0*np.ones(params_FM['Ncells'],dtype=int)
mlam_FM_NoSh = np.nanargmin(GLM_FM_NoSh['loss_regcv'][0],axis=0) #11*np.ones(params['Ncells'],dtype=int)# 
mlam_FM = np.nanargmin(GLM_FM['loss_regcv'][0],axis=0) #11*np.ones(params['Ncells'],dtype=int)# 
cellnum = np.arange(params_FM['Ncells'])
r2_all = np.stack((GLM_Shuff['r2_test'],GLM_FM_NoSh['r2_test'][malph,mlam_FM_NoSh,cellnum],GLM_FM['r2_test'][malph,mlam_FM,cellnum]))

In [ ]:
hbins = .02
sta_titles = ['Shuffle','Shifter Off','Shifter On']
mod_clrs = ['#ff0000',"#084c61","#db504a"]
move_clrs = ['blue','orange','green','red']
lim0 = 0
lim1 = .7
dlim = .2
xlab = r'$r^2$'
fig1, axs = plt.subplots(figsize=(5,5)) 
for n in range(r2_all.shape[0]):
    ax = axs
    count,edges = np.histogram(r2_all[n],bins=np.arange(lim0,lim1,hbins))
    edges_mid = np.array([(edges[i]+edges[i+1])/2 for i in range(len(edges)-1)])
    ax.bar(edges_mid, count/len(r2_all[n]),color=mod_clrs[n],width=hbins, alpha=.75,label=sta_titles[n])
    ax.set_xticks(np.arange(lim0,lim1,dlim))
    ax.set_xticklabels(np.round(np.arange(lim0,lim1,dlim),decimals=1))
    ax.set_xlabel(xlab)
    ax.axvline(x=np.nanmean(r2_all,axis=1)[n],c=mod_clrs[n],ls='--')
ax.legend(labelcolor='linecolor', fontsize=12, handlelength=0, handletextpad=0, fancybox=True)

### Full Figure

In [ ]:
vals_Vis = np.hstack([All_data[date_ani2[da]]['Vis_r2_test'] for da in range(len(dates_all))])
vals_VisNoSh = np.hstack([NoSh_data[date_ani2[da]]['VisNoSh_r2_test'] for da in range(len(dates_all))])
r2_all = np.stack((vals_VisNoSh,vals_Vis))

fig1 = plt.figure(constrained_layout=True, figsize=(25,8))
gs0 = gridspec.GridSpec(ncols=6, nrows=2, figure=fig1,wspace=.4,hspace=.5)
lag_ls = [-50,0,50]
########## Fig 2B ########## 
da=0
cells = [22,34,42,101] #np.arange(18)
gs00 = gridspec.GridSpecFromSubplotSpec(len(cells), 3, subplot_spec=gs0[:,:2],wspace=.05,hspace=.1)
axs1 = np.array([fig1.add_subplot(gs00[n,m]) for n in range(len(cells)) for m in range(3)]).reshape(len(cells),3)

for n, cell in enumerate(cells):
    crange = np.max(np.abs(All_data[date_ani2[da]]['Vis_rf_up'][cell]))
    for lag in range(params['nt_glm_lag']):
        ax = axs1[n,lag]
        im = ax.imshow(All_data[date_ani2[da]]['Vis_rf_up'][cell,lag],'RdBu_r',vmin=-crange,vmax=crange)
        axs1[0,lag].set_title('Lag={}ms'.format(lag_ls[lag]))
        # cbar = add_colorbar(im)
        # ax.axis('off')
    axs1[n,0].set_ylabel('Unit {}'.format(n+1),fontsize=fontsize)
cbar2 = fig1.colorbar(im, ax=axs1.ravel().tolist())
cbar2.set_ticks([-crange+.0015, crange])
# cbar2.set_ticks([])
# axs1[0,2].text(5,,'Light',rotation=90)
cbar2.set_ticklabels(['Dark', 'Light'])
cbar2.ax.tick_params(labelsize=fontsize, rotation=90)

for ax in axs1.flat:
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)
        ax.spines[axis].set_visible(True)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])

########## Fig 2C ########## 
gs01 = gridspec.GridSpecFromSubplotSpec(1, 3, subplot_spec=gs0[0,2:],wspace=.5,hspace=.5)
axs2 = np.array([fig1.add_subplot(gs01[n,m]) for n in range(1) for m in range(3)]).reshape(1,3)

shift_titles = [r'$dx$',r'$dy$',r'$d\alpha,\phi=0$',r'$d\alpha,\theta=0$']
cbar_label = ['pixels','pixels','deg','deg']
ticks=np.arange(0,90,40)
ticklabels=np.arange(-40,50,40)
shift_matshow=np.stack((shift_mat[0,:,:,40].T,shift_mat[1,:,:,40].T,shift_mat[2,:,40,:].T,shift_mat[2,40,:,:].T))
crange_list = np.stack((np.max(np.abs(shift_mat[:2])),np.max(np.abs(shift_mat[:2])), np.max(np.abs(shift_mat[2])), np.max(np.abs(shift_mat[2]))))
for n in range(2):
    ax = axs2[0,n]
    im1=ax.imshow(shift_matshow[n],vmin=-crange_list[n], vmax=crange_list[n], origin='lower', cmap='RdBu_r')
    cbar1 = add_colorbar(im1)
    cbar1.set_label(cbar_label[n],fontsize=fontsize)
    ax.set_xticks(ticks)
    ax.set_xticklabels(ticklabels)
    ax.set_yticks(ticks)
    ax.set_yticklabels(ticklabels)
    ax.set_xlabel(r'$\theta$',fontsize=fontsize)
    ax.set_ylabel(r'$\phi$',fontsize=fontsize)
    ax.set_title(shift_titles[n],fontsize=fontsize)
ax = axs2[0,2]
ax.plot(FM_move_avg[1, 2]*np.linspace(-pitch_range,pitch_range,81), shift_mat[2, 40, 40, :], 'k')
ax.set_xlabel(r'$\omega$',fontsize=fontsize)
ax.set_ylabel(r'$d\alpha$',fontsize=fontsize)
ax.set_xticks(ticklabels)
ax.set_xticklabels(ticklabels)
ax.set_yticks([-40,0,40])
ax.set_yticklabels([-40,0,40])
ax.axis('square')
ax.set_ylim(-45, 45)


########## Fig 2D ########## 
gs02 = gridspec.GridSpecFromSubplotSpec(2, 2, subplot_spec=gs0[1,2:4],wspace=.1,hspace=.5)
axs3 = np.array([fig1.add_subplot(gs02[n,m]) for n in range(2) for m in range(1)])
lag = 1
da = 3
cells = [9]
for n, cell in enumerate(cells):
    ax = axs3[n]
    crange2 = np.max(np.abs(NoSh_data[date_ani2[da]]['VisNoSh_rf_up'][cell]))
    im2 = ax.imshow(NoSh_data[date_ani2[da]]['VisNoSh_rf_up'][cell,lag],'RdBu_r', vmin=-crange2, vmax=crange2)
    ax.set_title('Shifter Off',fontsize=fontsize)
    ax = axs3[n+1]
    crange2 = np.max(np.abs(All_data[date_ani2[da]]['Vis_rf_up'][cell]))
    im2 = ax.imshow(All_data[date_ani2[da]]['Vis_rf_up'][cell,lag],'RdBu_r', vmin=-crange2, vmax=crange2)
    ax.set_title('Shifter On',fontsize=fontsize)

for ax in axs3.flat:
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)
        ax.spines[axis].set_visible(True)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])



########## Fig 2E ########## 
# gs03 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs0[1,3:4],wspace=.05,hspace=.5)
axs4 = fig1.add_subplot(gs02[:, 1])

ax = axs4
vis_th = r2_all[1] > .05
ax.scatter(r2_all[1][vis_th], r2_all[0][vis_th],c='k')
ax.plot(np.linspace(0, .6, 100), np.linspace(0, .6, 100), 'k--')
ax.set_xlabel(r'Shifter On $r^2$',fontsize=fontsize)
ax.set_ylabel(r'Shifter Off $r^2$',fontsize=fontsize)
ax.set_xticks([0,.2,.4,.6])
ax.set_xticklabels([0,.2,.4,.6])
ax.set_yticks([0, .2, .4, .6])
ax.set_yticklabels([0, .2, .4, .6])
ax.axis('square')

gs03 = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs0[1,4:],wspace=.1,hspace=.4)
axs5 = np.array([fig1.add_subplot(gs03[n,m]) for n in range(1) for m in range(2)])


fig1.savefig(paper_fig_dir/('Figure2_draft.pdf'), facecolor='white', transparent=True, bbox_inches='tight',dpi=300)


In [ ]:
# np.where(r2_all[0]>r2_all[1])


In [ ]:
plt.scatter(r2_all[1][vis_th],r2_all[0][vis_th])
plt.plot(np.linspace(0,.6,100),np.linspace(0,.6,100),'k--')

In [ ]:
fig, axs4 = plt.subplots(figsize=(5,5))
hbins = .02
sta_titles = ['Shifter Off','Shifter On']
NoShift_clrs = ["#050505", "#C41508"]
move_clrs = ['blue','orange','green','red']
lim0 = 0
lim1 = .7
dlim = .2
xlab = r'$r^2$'
for n in range(r2_all.shape[0]):
    ax = axs4
    count,edges = np.histogram(r2_all[n],bins=np.arange(lim0,lim1,hbins))
    edges_mid = np.array([(edges[i]+edges[i+1])/2 for i in range(len(edges)-1)])
    ax.bar(edges_mid, count/len(r2_all[n]),color=NoShift_clrs[n],width=hbins, alpha=.5,label=sta_titles[n])
    ax.set_xticks(np.arange(lim0,lim1,dlim))
    ax.set_xticklabels(np.round(np.arange(lim0,lim1,dlim),decimals=1))
    ax.set_xlabel(xlab,fontsize=fontsize)
    ax.axvline(x=np.nanmean(r2_all[n]), c=NoShift_clrs[n], ls='--')
ax.legend(labelcolor='linecolor', fontsize=14, handlelength=0, handletextpad=0, fancybox=True)
plt.tight_layout()


In [ ]:
celln = 12
np.argsort(NoSh_data[date_ani2[da]]['VisNoSh_r2_test'])[::-1]
plt.plot(NoSh_data[date_ani2[da]]['actual_smooth'][:,celln])
plt.plot(NoSh_data[date_ani2[da]]['VisNoSh_pred_smooth'][:,celln])
plt.plot(All_data[date_ani2[da]]['Vis_pred_smooth'][:,celln])

NoSh_data[date_ani2[da]]['VisNoSh_r2_test'][celln],All_data[date_ani2[da]]['Vis_r2_test'][celln]

## Figure 3

### Full Figure

In [ ]:
args = arg_parser(jupyter=True)
MovModel = 1
Kfold = 0
args['free_move'] = True
if args['free_move']:
    stim_type = 'fm1'
else:
    stim_type = 'hf1_wn'  # 'fm1' #

dates_all = ['070921/J553RT' ,'101521/J559NC','102821/J570LT','110421/J569LT'] # '102621/J558NC' '062921/G6HCK1ALTRN',
args['date_ani'] = dates_all[0]
args['train_shifter']=False
args['NoL1'] = False
args['Nepochs'] = 5000
params,file_dict,exp = load_params(MovModel,Kfold,args,debug=True)
params['nt_glm_lag']=1
data, train_idx_list, test_idx_list = load_train_test(file_dict, **params)
##### Set Train Test Splits #####
Kfold = 0
train_idx = train_idx_list[Kfold]
test_idx = test_idx_list[Kfold]
data = load_Kfold_data(data,train_idx,test_idx,params)
params, xtr, xtrm, xte, xtem, ytr, yte, shift_in_tr, shift_in_te, input_size, output_size, model_type, meanbias, model_move = load_GLM_data(data, params, train_idx, test_idx)
print('Model: {}, LinMix: {}, move_features: {}, Ncells: {}, train_shifter: {}'.format(params['MovModel'],params['LinMix'],params['move_features'],params['Ncells'],params['train_shifter']))

mod_titles = ['Mot','Vis','Add','Mul']
mod_clrs = ["#136f63","#e0ca3c","#f34213","#80a4ed"]
dates_all = ['070921/J553RT' ,'101521/J559NC','102821/J570LT','110421/J569LT'] # '102621/J558NC' 
ModelList_FM = []
ModelList_HF = []
for d,date_ani in enumerate(dates_all):
    ##### Free-Moving #####
    args['free_move']=True
    args['date_ani'] = date_ani
    args['NoL1']=False
    args['Nepochs'] = 10000
    date_ani2 = params['save_dir_fm'].parent.parent.parent/date_ani
    if args['NoL1'] == True: 
        L1 = 'NoL1'
        mod_name = '*Vis_{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    else: 
        L1 = 'withL1'
        mod_name = '*Vis_{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    ModelList = np.array(sorted(list(date_ani2.rglob(mod_name+'.h5'))))[0]
    ModelList_FM.append(ModelList)
    ModelList = np.array(sorted(list(date_ani2.rglob(mod_name+'.h5'))))[1]
    ModelList_HF.append(ModelList)
    
ModelList_FM = np.stack(ModelList_FM)
ModelList_HF = np.stack(ModelList_HF)

    
RF_all = []
r2_all = []
for n,(modfm,modhf) in enumerate(zip(ModelList_FM,ModelList_HF)):
    GLM_FM = ioh5.load(modfm)
    GLM_HF = ioh5.load(modhf)
    r2_all.append(np.stack((GLM_FM['r2_test'],GLM_HF['r2_test'])))
    RF_FM = GLM_FM['Cell_NN.0.weight'].reshape((GLM_FM['Cell_NN.0.weight'].shape[0],params['nt_glm_lag'],)+params['nks'])
    RF_HF = GLM_HF['Cell_NN.0.weight'].reshape((GLM_HF['Cell_NN.0.weight'].shape[0],params['nt_glm_lag'],)+params['nks'])
    RF_all.append(np.stack((RF_FM,RF_HF)))
RF_all = np.concatenate(RF_all,axis=1)

sf=4
RF_all_up = np.zeros((RF_all.shape[:3] + ( sf*(RF_all.shape[-2]), sf*(RF_all.shape[-1]))))
for n in range(RF_all.shape[0]):
    for celln in range(RF_all.shape[1]):
        for t in range(RF_all.shape[2]):
            RF_all_up[n,celln,t] = cv2.resize(RF_all[n,celln,t], (sf*(RF_all.shape[-1]), sf*(RF_all.shape[-2])))
r2_FMHF_RF = np.zeros(RF_all.shape[1])
for celln in np.arange(RF_all.shape[1]):
    r2_FMHF_RF[celln] = np.corrcoef(RF_all[0,celln,0].flatten(),RF_all[1,celln,0].flatten())[0,1]

In [ ]:
RF_all = []
r2_all = []
NCells_all = []
for n,(modfm,modhf) in enumerate(zip(ModelList_FM,ModelList_HF)):
    GLM_FM = ioh5.load(modfm)
    GLM_HF = ioh5.load(modhf)
    r2_all.append(np.stack((GLM_FM['r2_test'],GLM_HF['r2_test'])))
    RF_FM = GLM_FM['Cell_NN.0.weight'].reshape((GLM_FM['Cell_NN.0.weight'].shape[0],params['nt_glm_lag'],)+params['nks'])
    RF_HF = GLM_HF['Cell_NN.0.weight'].reshape((GLM_HF['Cell_NN.0.weight'].shape[0],params['nt_glm_lag'],)+params['nks'])
    RF_all.append(np.stack((RF_FM,RF_HF)))
    NCells_all.append(len(GLM_FM['r2_test']))
RF_all = np.concatenate(RF_all,axis=1)
r2_all = np.concatenate(r2_all,axis=1)

In [ ]:
scat_clr = ["#f9c80e","#f86624","#43bccd","#662e9b"] # 
scat_clr_all = np.hstack([NCells_all[n]*[scat_clr[n]] for n in range(len(scat_clr))])
clrs_all, labels = pd.factorize(scat_clr_all)
cmap = mpl.colors.ListedColormap(scat_clr)
fig,ax = plt.subplots(figsize=(8,8))
im = ax.scatter(r2_all[1],r2_all[0],c=clrs_all,cmap=cmap)
ax.set_xlabel('$r^2$ HF')
ax.set_ylabel('$r^2$ FM')
cbar = add_colorbar(im)
cbar.set_ticks(np.arange(.4,3.25,.75))
cbar.set_ticklabels(dates_all)
cbar.ax.tick_params(labelsize=fontsize)
# fig.savefig(paper_fig_dir/'R2_all_HF_FM.png', facecolor='white', transparent=True, bbox_inches='tight')

In [ ]:
celln = 6
sorted_r2 = np.argsort(vals_FMHF)[::-1]  # Sorted indecies for R2
vals_FMHF[celln], sorted_r2


In [ ]:
fontsize=24
da=0
vals_FMHF  = np.hstack([All_data[date_ani2[da]]['HF_FMHF_r2']  for da in range(len(dates_all))])
RF_all = np.stack((np.vstack([All_data[date_ani2[da]]['Vis_rf_up'] for da in range(len(dates_all))]),np.vstack([All_data[date_ani2[da]]['HF_rf_up'] for da in range(len(dates_all))])))
sorted_r2 = np.argsort(vals_FMHF)[::-1] ##### Sorted indecies for R2
fig1 = plt.figure(constrained_layout=False, figsize=(30,5))
gs0 = gridspec.GridSpec(ncols=4, nrows=2, figure=fig1,wspace=.2,hspace=.3)
lag_ls = [-50,0,50]

########## Fig 3A ########## 
cells = [53,61,101,244] # 
num_cells = len(cells)

gs00 = gridspec.GridSpecFromSubplotSpec(2, num_cells, subplot_spec=gs0[:,:3],wspace=.2,hspace=.05)
axs1 = np.array([fig1.add_subplot(gs00[n,m]) for n in range(2) for m in range(num_cells)]).reshape(2,num_cells)
xcut1=5
xcut2=-5
ycut1=5
ycut2=-5
for n, cell in enumerate(cells):
    
    crange2 = np.max(np.abs(RF_all[0,cell,2,int(sf*xcut1):int(sf*xcut2),int(sf*ycut1):int(sf*ycut2)]))
    im2 = axs1[0,n].imshow(RF_all[0,cell, 2,int(sf*xcut1):int(sf*xcut2),int(sf*ycut1):int(sf*ycut2)],'RdBu_r', vmin=-crange2, vmax=crange2)
    crange1 = np.max(np.abs(RF_all[1,cell,2,int(sf*xcut1):int(sf*xcut2),int(sf*ycut1):int(sf*ycut2)]))
    im1 = axs1[1,n].imshow(RF_all[1,cell, 2,int(sf*xcut1):int(sf*xcut2),int(sf*ycut1):int(sf*ycut2)], 'RdBu_r', vmin=-crange1, vmax=crange1)
    axs1[0,n].set_title('Unit: {}, cc={:.02f}'.format(n+1,vals_FMHF[cell]), fontsize=fontsize)

axs1[1,0].set_ylabel('Head \n fixed', fontsize=fontsize)
axs1[0,0].set_ylabel('Freely \n moving', fontsize=fontsize)

cbar2 = fig1.colorbar(im1, ax=axs1.ravel().tolist())
cbar2.set_ticks([-crange1, crange1])
cbar2.set_ticklabels(['Dark', 'Light'])
cbar2.ax.tick_params(labelsize=fontsize)

for ax in axs1.flat:
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)
        ax.spines[axis].set_visible(True)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])   

########## Fig 3B ########## 
gs01 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs0[:,3:],wspace=.05,hspace=.1)
axs2 = fig1.add_subplot(gs01[0,0])
vals_Vis = np.hstack([All_data[date_ani2[da]]['Vis_r2_test'] for da in range(len(dates_all))])
vals_HF  = np.hstack([All_data[date_ani2[da]]['HF_r2_test'] for da in range(len(dates_all))])
# vals_FMHF_th = (vals_Vis>.05) & (vals_HF>.05)

hbins=.05
lim0 = 0
lim1 = .75
dlim = .2
ylim = .4
dylim = .1
count,edges = np.histogram(np.abs(vals_FMHF),bins=np.arange(lim0,lim1,hbins))
edges_mid = np.array([(edges[i]+edges[i+1])/2 for i in range(len(edges)-1)])
axs2.bar(edges_mid, count/len(vals_FMHF),color='k',width=hbins, alpha=.75)
axs2.set_xlabel('RF Correlation',fontsize=fontsize+5)
axs2.set_ylabel('Fraction of Units',fontsize=fontsize+5)
axs2.set_yticks(np.arange(lim0,ylim,dylim))
axs2.set_yticklabels(np.round(np.arange(lim0,ylim,dylim),decimals=3),fontsize=fontsize-2)
axs2.set_xticks(np.arange(lim0,lim1,dlim))
axs2.set_xticklabels(np.round(np.arange(lim0,lim1,dlim),decimals=1), fontsize=fontsize-2)
# axs2.set_xlim(0,.8)
# axs2.set_ylim(0,.25)

plt.tight_layout()
plt.show()
fig1.savefig(paper_fig_dir/'Figure3_draft.pdf', facecolor='white', transparent=True, bbox_inches='tight')


## Figure 4

In [ ]:
celltype_dir = params['data_dir'].parent.parent.parent.parent / 'batch_files/120221_hffm'
celltype_file = celltype_dir / 'pooled_ephys_population_update_120621.pickle'
celltype = pickle.load(open(celltype_file,'rb'))

Kfold = 0
args['free_move']=True
mod_titles = ['Mot','Vis','Add','Mul']
mod_clrs = ["#136f63","#e0ca3c","#f34213","#80a4ed"]
dates_all = ['070921/J553RT' ,'101521/J559NC','102821/J570LT','110421/J569LT'] # '102621/J558NC' 
ModelList_all = []
test_std =[]
tuning_sig_all = []
NCells_all = []
bad_cells_all = []
test_nsp_all = []
for d,date_ani in enumerate(dates_all):
    args['date_ani'] = date_ani
    args['NoL1']=False
    args['Nepochs'] = 10000
    params,_,_ = load_params(0,Kfold,args,debug=True)
    params['nt_glm_lag']=3
    date_ani_dir = params['save_dir'].parent.parent.parent/date_ani
    if args['NoL1'] == True: 
        L1 = 'NoL1'
        mod_name = '*{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    else: 
        L1 = 'withL1'
        mod_name = '*{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    ModelList = np.array(sorted(list(set(date_ani_dir.rglob(mod_name+'.h5')) - set(date_ani_dir.rglob('*VisNoShifter'+mod_name+'.h5')))))
    ModelList_all.append(ModelList)
    bad_cells = np.load(params['save_dir_hf']/'bad_cells.npy')
    bad_cells_all.append(bad_cells)
    data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')
    test_nsp_all.append(data['test_nsp'])
    test_std.append(np.var(data['test_nsp'],axis=0))
    tuning_sig_all.append(tuning_sig2)
    NCells_all.append(len(tuning_sig2))
ModelList_all = np.stack(ModelList_all)
test_std=np.hstack(test_std)
tuning_sig_all = np.hstack(tuning_sig_all)



In [ ]:
date_ani2 = [dates_all[n].replace('/','_') for n in range(len(dates_all))]
All_data = {date_ani2[n]:{} for n in range(len(date_ani2))}
for da, date_ani in enumerate(date_ani2):
    All_data[date_ani]['actual_meanfr'] = np.mean(test_nsp_all[da],axis=0)/params['model_dt']
    All_data[date_ani]['actual_smooth'] = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=0, arr=test_nsp_all[da])[params['bin_length']:-params['bin_length']]
    for modeln,fmod in enumerate(ModelList_all[da][:-1]):
        GLM_Data = ioh5.load(fmod)
        All_data[date_ani][mod_titles[modeln]+'_pred_smooth'] = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=1, arr=GLM_Data['pred_cv'].copy())[:,params['bin_length']:-params['bin_length']].T
        All_data[date_ani][mod_titles[modeln]+'_r2_test'] = GLM_Data['r2_test'].copy()
        All_data[date_ani][mod_titles[modeln]+'_vloss_trace'] = np.nanmean(GLM_Data['vloss_trace_all'][:,-10:],axis=1)/np.nanvar(test_nsp_all[da],axis=0)
celltypes_all = []
for n,date_exp in enumerate([2,4,7,9]):
    celltype2 = celltype[celltype['session']==celltype['session'].unique()[date_exp]].reset_index(drop=True)
    celltype2 = celltype2.drop(bad_cells_all[n])
    celltypes_all.append([celltype2['exc_or_inh'] == 'exc',celltype2['hf1_wn_depth_from_layer5']])
celltypes_all = np.hstack(celltypes_all)

In [ ]:
da = 1
fontsize=24
args['date_ani'] = dates_all[da]
params,_,_ = load_params(0,Kfold,args,debug=True)
data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')


In [ ]:
titles = [r'$\theta$',r'$\phi$',r'$\rho$',r'$\omega$']
celln = 52#106
modN = 1
da = 1
anglim = 70
quartiles = np.arange(0,1.25,.25)
spike_percentiles = np.arange(0,1.25,.25)
spike_percentiles[-1]=.99
spk_percentile2 = np.arange(.125,1.125,.25)
ncells=All_data[date_ani2[da]][mod_titles[1]+'_pred_smooth'].shape[-1]
predcell = All_data[date_ani2[da]][mod_titles[1]+'_pred_smooth'][:,celln]#/params['model_dt']
nspcell = All_data[date_ani2[da]]['actual_smooth'][:,celln]#/params['model_dt']
pred_range = np.quantile(predcell,[.1,.9])
test_nsp_range = np.quantile(nspcell,[.01,1])
pred_rangelin = np.quantile(predcell,spike_percentiles)
xbin_pts = np.quantile(predcell,spk_percentile2)
stat_bins = len(pred_rangelin)

fig = plt.figure(constrained_layout=False, figsize=(18,15))
gs0 = gridspec.GridSpec(ncols=4, nrows=2, figure=fig,wspace=.6,hspace=.3)
########## Fig 4A ########## 
gs00 = gridspec.GridSpecFromSubplotSpec(2, len(titles), subplot_spec=gs0[0,:4],wspace=.5,hspace=.7)
axs1 = np.array([fig.add_subplot(gs00[n,m]) for n in range(1) for m in range(len(titles))]).reshape(1,len(titles))
axs1b = np.array([fig.add_subplot(gs00[n,m]) for n in range(1,2) for m in range(len(titles))]).reshape(1,len(titles))

top_yaxs = np.max(ax_ylims[celln])+2*np.max(tuning_stds[celln])
traces = np.zeros((ncells,len(titles),len(quartiles)-1,stat_bins-1))
traces_mean = np.zeros((ncells,len(titles),stat_bins-1))
edges_all = np.zeros((ncells,len(titles),len(quartiles)-1,stat_bins-1))
for i,modeln in enumerate(range(len(titles))):
    metric = move_data[params['bin_length']:-params['bin_length'],modeln]
    nranges = np.quantile(metric,quartiles)
    stat_range, edges, _ = binned_statistic(metric,nsp_raw[params['bin_length']:-params['bin_length'],celln],statistic='mean',bins=nranges)
    edge_mids = np.quantile(metric,spk_percentile2)#
    for m in range(len(nranges)-1):
        axs1[0,modeln].axvspan(nranges[m], nranges[m+1],ymin=0,ymax=1,alpha=0.8, color=q_clrs[m],zorder=0)
    axs1[0,modeln].plot(edge_mids,stat_range/params['model_dt'],'.-', ms=20, lw=4,c='k')
    axs1[0,modeln].set_ylim(bottom=0,top=np.max(ax_ylims,axis=1)[celln]+2*np.nanmax(tuning_stds,axis=(1,2))[celln])
    xlim_range = np.max(np.abs([nranges[0],nranges[-1]]))
    axs1[0,modeln].set_xlim(-xlim_range,xlim_range)
    axs1[0,modeln].set_xlabel('angle (deg)',fontsize=fontsize)
    axs1[0,modeln].set_ylabel('sp/sec',fontsize=fontsize)
    axs1[0,modeln].set_title(titles[modeln],fontsize=fontsize)

    
    stat_all, edges, _ = binned_statistic(predcell,nspcell,statistic='mean',bins=pred_rangelin)
    edge_mids = xbin_pts 
    traces_mean[celln,modeln]=stat_all
    max_fr = np.max(stat_all)
    for n in range(len(nranges)-1):
        ind = np.where(((metric<=nranges[n+1])&(metric>nranges[n])))[0]
        pred = predcell[ind]
        sp = nspcell[ind]
        stat_range, edges, _ = binned_statistic(pred, sp, statistic='mean',bins=pred_rangelin)
        edge_mids = xbin_pts
        traces[celln,modeln,n]=stat_range
        edges_all[celln,modeln,n]=edge_mids
        axs1b[0,modeln].plot(edge_mids, stat_range,'.-', c=q_clrs[n],label='{:.02f} : {:.02f}'.format(nranges[n],nranges[n+1]),lw=4,ms=20,alpha=.9)
        axs1b[0,modeln].set_xlabel('predicted (sp/sec)',fontsize=fontsize)
        axs1b[0,modeln].set_ylabel('actual (sp/sec)',fontsize=fontsize)
    
    lim_max = np.nanmax(np.hstack((edge_mids,traces[celln,modeln].flatten())))+.5*np.std(edges)
    lim_min = np.nanmin(np.hstack((edge_mids,traces[celln,modeln].flatten())))-.5*np.std(edges)
    lims = (0, lim_max) if (lim_min)<0 else (lim_min,lim_max) 
    axs1b[0,modeln].plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
    axs1b[0,modeln].plot(edge_mids, stat_all,'.-', c='k', lw=5, ms=20, label='All_data', alpha=.8)
    axs1b[0,modeln].set_xticks(np.arange(0,lims[1],int(lims[1]/3)))
    axs1b[0,modeln].set_yticks(np.arange(0,lims[1],int(lims[1]/3)))
    axs1b[0,modeln].set_xticklabels(np.arange(0,lims[1],int(lims[1]/3)))
    axs1b[0,modeln].set_yticklabels(np.arange(0,lims[1],int(lims[1]/3)))
    axs1b[0,modeln].axis('square')
    axs1b[0,modeln].set(xlim=lims, ylim=lims)
    axs1b[0,modeln].set_ylim(bottom=0)
    axs1b[0, modeln].set_xlim(left=0)

########## Fig 4B ########## 
gs01 = gridspec.GridSpecFromSubplotSpec(2,1, subplot_spec=gs0[1,:2],wspace=.05,hspace=.8)
axs2 = np.array([fig.add_subplot(gs01[n,m]) for n in range(1) for m in range(1)]).reshape(1,1)
axs2b = np.array([fig.add_subplot(gs01[n,m]) for n in range(1,2) for m in range(1)]).reshape(1,1)

cmap = mpl.colors.ListedColormap(q_clrs)
##### Plotting Position Scatter #####
celln = 52#106
da = 1
t = np.argmin(np.abs((np.arange(All_data[date_ani2[da]]['actual_smooth'].shape[0])*params['model_dt'])-0)) # 200 seconds in #6000
dt = int(100/params['model_dt']) #All_data[date_ani2[da]]['actual_smooth'].shape[0] # dt in seconds converted to time points
pred_time = (np.arange(All_data[date_ani2[da]]['actual_smooth'].shape[0])*params['model_dt'])[0:np.abs(dt)]
ax = axs2[0,0]
metric = move_data[params['bin_length']:-params['bin_length'],3]
ax.scatter(pred_time, metric[t:t+dt],s=10,c=np.digitize(metric,nranges)[t:t+dt],cmap=cmap)
ax.set_ylabel(r'$\omega$ (deg)',fontsize=fontsize)
ax.set_xlabel('time (s)',fontsize=fontsize)
ax.set_ylim([-anglim,anglim])

# celln = 18#106
# da = 0
# t = 6000 #np.argmin(np.abs(pred_time-375)) # T seconds in 
# dt = 3000 #All_data[date_ani2[da]]['actual_smooth'].shape[0]
##### Plotting Firing Rate #####
pred_time = (np.arange(All_data[date_ani2[da]]['actual_smooth'].shape[0])*params['model_dt'])[0:np.abs(dt)]
ax = axs2b[0,0]
zorder = np.argsort([All_data[date_ani2[da]][mod_titles[modN]+'_r2_test'][celln] for modN in range(len(mod_titles))])+1
ax.plot(pred_time,All_data[date_ani2[da]]['actual_smooth'][t:t+dt,celln],c='k',lw=3)
for modN in range(len(mod_titles)):
    ax.plot(pred_time,All_data[date_ani2[da]][mod_titles[modN]+'_pred_smooth'][t:t+dt,celln],c=mod_clrs[modN],lw=3,
            label='{} $r^2$: {:.02f}'.format(mod_titles[modN],All_data[date_ani2[da]][mod_titles[modN]+'_r2_test'][celln]),zorder=zorder[n])
ax.legend(labelcolor='linecolor', fontsize=14, handlelength=0, handletextpad=0,loc='upper right',ncol=4, bbox_to_anchor=(.9, 1.3))
ax.set_xlabel('time (s)',fontsize=fontsize)
ax.set_ylabel('sp/sec',fontsize=fontsize)

########## Fig 4c ########## 
fr_th = 1 # hz
vals_Mot = np.hstack([All_data[date_ani2[da]]['Mot_r2_test'] for da in range(len(dates_all))])
vals_Vis = np.hstack([All_data[date_ani2[da]]['Vis_r2_test'] for da in range(len(dates_all))])
vals_Add = np.hstack([All_data[date_ani2[da]]['Add_r2_test'] for da in range(len(dates_all))])
vals_Mul = np.hstack([All_data[date_ani2[da]]['Mul_r2_test'] for da in range(len(dates_all))])
gs02 = gridspec.GridSpecFromSubplotSpec(2,2, subplot_spec=gs0[1,2:4],wspace=.8,hspace=.8)
axs3a = np.array([fig.add_subplot(gs02[n,m]) for n in range(2) for m in range(2)]).reshape(2,2)

tuning_idx = np.where(tuning_sig_all)[0]
lims = (0, .65)
crange=500
##### Vis vs Mot #####
ax = axs3a[0,0]
im=ax.scatter(vals_Vis,vals_Mot,c=celltypes_all[1],cmap='jet',s=20,vmin=-crange,vmax=crange)
ax.plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
ax.set_xlabel(r'$r^2_{Vis}$',fontsize=fontsize)
ax.set_ylabel(r'$r^2_{Pos}$',fontsize=fontsize)
ax.set_yticks(np.arange(0,lims[1],.2))
ax.set_yticklabels(np.round(np.arange(0,lims[1],.2),decimals=1))
ax.set_ylim(0,lims[1])
ax.set_xlim(0,lims[1])
ax.set_xticks(np.arange(0,lims[1],.2))
ax.set_xticklabels(np.round(np.arange(0,lims[1],.2),decimals=1))
ax.axis('square')
cbar = add_colorbar(im)
cbar.set_ticks([-crange,0,crange])
cbar.set_ticklabels([-crange,0,crange])
cbar.ax.tick_params(labelsize=fontsize-10)
cbar.set_label(r'depth $(\mu m)$')

##### Vis vs Add/Mul #####
cmap_mod = mpl.colors.ListedColormap(mod_clrs[2:])
vals_AM = np.max(np.stack((vals_Add,vals_Mul)),axis=0)
AvM = vals_Add<vals_Mul
ax = axs3a[0,1]
scatter=ax.scatter(vals_Vis,vals_AM,30,c=AvM,cmap=cmap_mod)
ax.plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
ax.set_xlabel(r'$r^2_{Vis}$',fontsize=fontsize)
ax.set_ylabel(r'$r^2$',fontsize=fontsize)
ax.set_yticks(np.arange(0,lims[1],.2))
ax.set_yticklabels(np.round(np.arange(0,lims[1],.2),decimals=1))
ax.set_xticks(np.arange(0,lims[1],.2))
ax.set_xticklabels(np.round(np.arange(0,lims[1],.2),decimals=1))
ax.axis('square')
ax.set_ylim(-.05, lims[1])
ax.set_xlim(-.05, lims[1])
# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*scatter.legend_elements(),fontsize=14,markerscale=0, handlelength=0, handletextpad=0,loc="lower right")
legend1.get_texts()[0].set_text('Add')
legend1.get_texts()[0].set_color(mod_clrs[2])
legend1.get_texts()[1].set_text('Mul')
legend1.get_texts()[1].set_color(mod_clrs[3])

move_r2_th = (vals_Mot>.05) # & (vals_Add>vals_Vis) & (vals_Mul>vals_Vis)
vals_Mot = vals_Mot[move_r2_th]
vals_Vis = vals_Vis[move_r2_th]
vals_Add = vals_Add[move_r2_th]
vals_Mul = vals_Mul[move_r2_th]
celltypes_all2 = celltypes_all[:,move_r2_th]

##### R2 Explained Variance #####
diff = vals_Mul-vals_Add
increase = vals_Vis
th = increase>0

NMul = np.sum(diff[th]>0)
NAdd = np.sum(diff[th]<0)
ax=axs3a[1,0]
im = ax.scatter(diff[th],increase[th],c=celltypes_all2[1,th],cmap='jet',s=20,vmin=-500,vmax=500)
ax.axvline(x=0,c='k')
ax.set_ylabel(r'$r^2_{Vis}$',fontsize=fontsize)
ax.set_xlabel(r'$r^2_{Mul}-r^2_{Add}$',fontsize=fontsize)
ax.set_title(r'$N_{{Add}}$:{}, $N_{{Mul}}$:{}'.format(NAdd,NMul),fontsize=fontsize)
ax.set_ylim(-.05,.6)
ax.set_xlim(-.075,.075)
cbar = add_colorbar(im)
cbar.set_ticks([-crange,0,crange])
cbar.set_ticklabels([-crange,0,crange])
cbar.ax.tick_params(labelsize=fontsize-10)
cbar.set_label(r'depth $(\mu m)$')

vals_Mot = np.hstack([All_data[date_ani2[da]]['Mot_r2_test'] for da in range(len(dates_all))])
vals_Vis = np.hstack([All_data[date_ani2[da]]['Vis_r2_test'] for da in range(len(dates_all))])
vals_Add = np.hstack([All_data[date_ani2[da]]['Add_r2_test'] for da in range(len(dates_all))])
vals_Mul = np.hstack([All_data[date_ani2[da]]['Mul_r2_test'] for da in range(len(dates_all))])
move_r2_th = (vals_Mot>.05) & (vals_Vis>.05) # & (vals_Mul>vals_Vis)
vals_Mot = vals_Mot[move_r2_th]
vals_Vis = vals_Vis[move_r2_th]
vals_Add = vals_Add[move_r2_th]
vals_Mul = vals_Mul[move_r2_th]
celltypes_all2 = celltypes_all[:,move_r2_th]
ax=axs3a[1,1]
hbins=.005
count,edges = np.histogram(vals_Mul-vals_Add,bins=np.arange(-.05,.05,hbins))
edges_mid = np.array([(edges[i]+edges[i+1])/2 for i in range(len(edges)-1)])
ax.bar(edges_mid, count/len(vals_Mul),color='#0A2239',width=hbins, alpha=1)
ax.axvline(x=np.nanmean(vals_Mul-vals_Add),c=mod_clrs[3],ls='--',lw=2)

ax.set_xticklabels(np.round([-.05,0,.05],decimals=2))
# ax.set_title(mod_titles[n])
ax.set_xlabel(r'$r^2_{Mul}-r^2_{Add}$',fontsize=fontsize)
ax.set_xticks([-.05, 0, .05])
ax.set_xlim([-.05, .05])
plt.show()
gs0.tight_layout(fig)

fig.savefig(paper_fig_dir/'Fig4_draft_Cell52_101521.pdf', facecolor='white', transparent=True, bbox_inches='tight')


In [ ]:
fig = plt.figure(constrained_layout=False, figsize=(15,12))
gs0 = gridspec.GridSpec(ncols=2, nrows=2, figure=fig,wspace=.5,hspace=.5)
########## Fig 4c ########## 
fr_th = 1 # hz
vals_Mot = np.hstack([All_data[date_ani2[da]]['Mot_r2_test'] for da in range(len(dates_all))])
vals_Vis = np.hstack([All_data[date_ani2[da]]['Vis_r2_test'] for da in range(len(dates_all))])
vals_Add = np.hstack([All_data[date_ani2[da]]['Add_r2_test'] for da in range(len(dates_all))])
vals_Mul = np.hstack([All_data[date_ani2[da]]['Mul_r2_test'] for da in range(len(dates_all))])

axs3a = np.array([fig.add_subplot(gs0[n,m]) for n in range(2) for m in range(2)]).reshape(2,2)

tuning_idx = np.where(tuning_sig_all)[0]
lims = (0, .65)
crange=500
##### Vis vs Mot #####
ax = axs3a[0,0]
im=ax.scatter(vals_Vis,vals_Mot,c=celltypes_all[1],cmap='jet',s=20,vmin=-crange,vmax=crange)
ax.plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
ax.set_xlabel(r'$r^2_{Vis}$',fontsize=fontsize)
ax.set_ylabel(r'$r^2_{Pos}$',fontsize=fontsize)
ax.set_yticks(np.arange(0,lims[1],.2))
ax.set_yticklabels(np.round(np.arange(0,lims[1],.2),decimals=1))
ax.set_ylim(0,lims[1])
ax.set_xlim(0,lims[1])
ax.set_xticks(np.arange(0,lims[1],.2))
ax.set_xticklabels(np.round(np.arange(0,lims[1],.2),decimals=1))
ax.axis('square')
cbar = add_colorbar(im)
cbar.set_ticks([-crange,0,crange])
cbar.set_ticklabels([-crange,0,crange])
cbar.ax.tick_params(labelsize=fontsize-10)
cbar.set_label(r'depth $(\mu m)$')

##### Vis vs Add/Mul #####
cmap_mod = mpl.colors.ListedColormap(mod_clrs[2:])
vals_AM = np.max(np.stack((vals_Add,vals_Mul)),axis=0)
AvM = vals_Add<vals_Mul
ax = axs3a[0,1]
scatter=ax.scatter(vals_Vis,vals_AM,30,c=AvM,cmap=cmap_mod)
ax.plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
ax.set_xlabel(r'$r^2_{Vis}$',fontsize=fontsize)
ax.set_ylabel(r'$r^2$',fontsize=fontsize)
ax.set_yticks(np.arange(0,lims[1],.2))
ax.set_yticklabels(np.round(np.arange(0,lims[1],.2),decimals=1))
ax.set_ylim(0,lims[1])
ax.set_xlim(0,lims[1])
ax.set_xticks(np.arange(0,lims[1],.2))
ax.set_xticklabels(np.round(np.arange(0,lims[1],.2),decimals=1))
ax.axis('square')
# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*scatter.legend_elements(),fontsize=14,markerscale=0, handlelength=0, handletextpad=0,loc="best")
legend1.get_texts()[0].set_text('Add')
legend1.get_texts()[0].set_color(mod_clrs[2])
legend1.get_texts()[1].set_text('Mul')
legend1.get_texts()[1].set_color(mod_clrs[3])


move_r2_th = (vals_Mot>.05)# & (vals_Add>vals_Vis) & (vals_Mul>vals_Vis)
vals_Mot = vals_Mot[move_r2_th]
vals_Vis = vals_Vis[move_r2_th]
vals_Add = vals_Add[move_r2_th]
vals_Mul = vals_Mul[move_r2_th]
celltypes_all2 = celltypes_all[:,move_r2_th]

##### R2 Explained Variance #####
diff = vals_Mul-vals_Add
increase = vals_Vis
th = increase>0

NMul = np.sum(diff[th]>0)
NAdd = np.sum(diff[th]<0)
ax=axs3a[1,0]
im = ax.scatter(diff[th],increase[th],c=celltypes_all2[1,th],cmap='jet',s=20,vmin=-500,vmax=500)
ax.axvline(x=0,c='k')
ax.set_ylabel(r'$r^2_{Vis}$',fontsize=fontsize)
ax.set_xlabel(r'$r^2_{Mul}-r^2_{Add}$',fontsize=fontsize)
ax.set_title(r'$N_{{Add}}$:{}, $N_{{Mul}}$:{}'.format(NAdd,NMul),fontsize=fontsize)
ax.set_ylim(-.05,.6)
ax.set_xlim(-.075,.075)
cbar = add_colorbar(im)
cbar.set_ticks([-crange,0,crange])
cbar.set_ticklabels([-crange,0,crange])
cbar.ax.tick_params(labelsize=fontsize-10)
cbar.set_label(r'depth $(\mu m)$')


vals_Mot = np.hstack([All_data[date_ani2[da]]['Mot_r2_test'] for da in range(len(dates_all))])
vals_Vis = np.hstack([All_data[date_ani2[da]]['Vis_r2_test'] for da in range(len(dates_all))])
vals_Add = np.hstack([All_data[date_ani2[da]]['Add_r2_test'] for da in range(len(dates_all))])
vals_Mul = np.hstack([All_data[date_ani2[da]]['Mul_r2_test'] for da in range(len(dates_all))])
move_r2_th = (vals_Mot>.05) & (vals_Vis>.05) # & (vals_Mul>vals_Vis)
vals_Mot = vals_Mot[move_r2_th]
vals_Vis = vals_Vis[move_r2_th]
vals_Add = vals_Add[move_r2_th]
vals_Mul = vals_Mul[move_r2_th]
celltypes_all2 = celltypes_all[:,move_r2_th]
ax=axs3a[1,1]
hbins=.005
count,edges = np.histogram(vals_Mul-vals_Add,bins=np.arange(-.05,.05,hbins))
edges_mid = np.array([(edges[i]+edges[i+1])/2 for i in range(len(edges)-1)])
ax.bar(edges_mid, count/len(vals_Mul),color='#0A2239',width=hbins, alpha=1)
ax.axvline(x=np.nanmean(vals_Mul-vals_Add),c=mod_clrs[3],ls='--',lw=2)
# ax.set_xticks(np.arange(0,1.2,.2))
# ax.set_xticklabels(np.round(np.arange(0,1.2,.2),decimals=1))
# ax.set_title(mod_titles[n])
ax.set_xlabel(r'$r^2_{Mul}-r^2_{Add}$',fontsize=fontsize)

fig.savefig(paper_fig_dir/'Scatter_comparision.pdf', facecolor='white', transparent=True, bbox_inches='tight')


In [ ]:
(vals_Mul-vals_Add).shape

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))
ax=axs[0]
ax.scatter(vals_Mot+vals_Vis,vals_Add)
ax.plot(np.linspace(0,.7),np.linspace(0,.7),'k--',zorder=0)
ax.set_xlabel(r'$r^2_{Mot}+r^2_{Vis}$')
ax.set_ylabel(r'$r^2_{Add}$')
ax=axs[1]
ax.scatter(vals_Mot+vals_Vis,vals_Mul)
ax.plot(np.linspace(0,.7),np.linspace(0,.7),'k--',zorder=0)
ax.set_xlabel(r'$r^2_{Mot}+r^2_{Vis}$')
ax.set_ylabel(r'$r^2_{Mul}$')
plt.tight_layout()
# fig.savefig(paper_fig_dir/'SumR2_Vs_Joint.png', facecolor='white', transparent=True, bbox_inches='tight')

## Other Plots

### Comparison of all Plots

In [ ]:
celltype_dir = params['data_dir'].parent.parent.parent.parent / 'batch_files/120221_hffm'
celltype_file = celltype_dir / 'pooled_ephys_population_update_120621.pickle'
celltype = pickle.load(open(celltype_file,'rb'))

Kfold = 0
args['free_move']=True
mod_titles = ['Mot','Vis','Add','Mul','HF']
dates_all = ['070921/J553RT' ,'101521/J559NC','102821/J570LT','110421/J569LT'] # '102621/J558NC' 
ModelList_all,test_std,tuning_sig_all,NCells_all,bad_cells_all,test_nsp_all,hf_nsp_all = [],[], [], [], [], [], []
for d,date_ani in enumerate(dates_all):
    args['free_move'] = True
    args['date_ani'] = date_ani
    args['NoL1']=False
    args['Nepochs'] = 10000
    params,_,_ = load_params(1,Kfold,args,debug=True)
    params['nt_glm_lag'] = 3
    date_ani_dir = params['save_dir'].parent.parent.parent/date_ani
    if args['NoL1'] == True: 
        L1 = 'NoL1'
        mod_name = '*{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    else: 
        L1 = 'withL1'
        mod_name = '*{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    ModelList = np.array(sorted(list(set(date_ani_dir.rglob(mod_name+'.h5')) - set(date_ani_dir.rglob('*VisNoShifter'+mod_name+'.h5')))))
    ModelList_all.append(ModelList)
    bad_cells = np.load(params['save_dir_hf']/'bad_cells.npy')
    bad_cells_all.append(bad_cells)
    data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')
    test_nsp_all.append(data['test_nsp'])
    test_std.append(np.var(data['test_nsp'],axis=0))
    tuning_sig_all.append(tuning_sig2)
    NCells_all.append(len(tuning_sig2))
    args['free_move'] = False
    params,_,_ = load_params(0,Kfold,args,debug=True)
    data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')
    hf_nsp_all.append(data['test_nsp'])

ModelList_all = np.stack(ModelList_all)
test_std=np.hstack(test_std)
tuning_sig_all = np.hstack(tuning_sig_all)


In [ ]:
sf=4
date_ani2 = [dates_all[n].replace('/','_') for n in range(len(dates_all))]
All_data = {date_ani2[n]:{} for n in range(len(date_ani2))}
for da, date_ani in enumerate(date_ani2):
    All_data[date_ani]['actual_meanfr'] = np.mean(test_nsp_all[da],axis=0)/params['model_dt']
    All_data[date_ani]['actual_smooth'] = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=0, arr=test_nsp_all[da])[params['bin_length']:-params['bin_length']]
    All_data[date_ani]['actual_HF_smooth'] = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=0, arr=hf_nsp_all[da])[params['bin_length']:-params['bin_length']]
    for ModelRun,fmod in enumerate(ModelList_all[da]):
        GLM_Data = ioh5.load(fmod)
        All_data[date_ani][mod_titles[ModelRun]+'_pred_smooth'] = np.apply_along_axis(lambda m: np.convolve(m, np.ones(params['bin_length']), mode='same')/(params['bin_length'] * params['model_dt']), axis=1, arr=GLM_Data['pred_cv'].copy())[:,params['bin_length']:-params['bin_length']].T
        All_data[date_ani][mod_titles[ModelRun]+'_r2_test'] = GLM_Data['r2_test'].copy()
        All_data[date_ani][mod_titles[ModelRun]+'_vloss_all'] = GLM_Data['vloss_trace_all'].copy()
        All_data[date_ani][mod_titles[ModelRun]+'_vloss_trace'] = np.nanmean(GLM_Data['vloss_trace_all'][:,-10:],axis=1)/np.nanvar(test_nsp_all[da],axis=0)
        if (ModelRun == 1) | (ModelRun == 4):
            All_data[date_ani][mod_titles[ModelRun]+'_loss_regcv'] = GLM_Data['loss_regcv'].copy()
            RF_all = GLM_Data['Cell_NN.0.weight'].reshape((GLM_Data['Cell_NN.0.weight'].shape[0],params['nt_glm_lag'],)+params['nks'])
            RF_all_up = np.zeros((RF_all.shape[:2] + ( sf*(RF_all.shape[-2]), sf*(RF_all.shape[-1]))))
            for celln in range(RF_all.shape[0]):
                for t in range(RF_all.shape[1]):
                    RF_all_up[celln,t] = cv2.resize(RF_all[celln,t], (sf*(RF_all.shape[-1]), sf*(RF_all.shape[-2])))
            All_data[date_ani][mod_titles[ModelRun]+'_rf_up'] = RF_all_up
            All_data[date_ani][mod_titles[ModelRun]+'_rf_all'] = RF_all
        if (ModelRun == 2) | (ModelRun==3):
            All_data[date_ani][mod_titles[ModelRun]+'_moveW'] = GLM_Data['posNN.Layer0.weight'].copy()
        if (ModelRun == 4):
            r2_FMHF_RF = np.zeros(All_data[date_ani][mod_titles[1]+'_rf_all'].shape[0])
            for celln in np.arange(All_data[date_ani][mod_titles[1]+'_rf_all'].shape[0]):
                r2_FMHF_RF[celln] = np.corrcoef(All_data[date_ani][mod_titles[1]+'_rf_all'][celln,2].flatten(),All_data[date_ani][mod_titles[-1]+'_rf_all'][celln,2].flatten())[0,1]
            All_data[date_ani][mod_titles[ModelRun]+'_FMHF_r2'] = r2_FMHF_RF

celltypes_all = []
for n,date_exp in enumerate([2,4,7,9]):
    celltype2 = celltype[celltype['session']==celltype['session'].unique()[date_exp]].reset_index(drop=True)
    celltype2 = celltype2.drop(bad_cells_all[n])
    celltypes_all.append([celltype2['exc_or_inh'] == 'exc',celltype2['hf1_wn_depth_from_layer5']])
celltypes_all = np.hstack(celltypes_all)


In [ ]:
save_pdf = True
anglim = 65
fontsize=24
titles = [r'$\theta$',r'$\phi$',r'$\rho$',r'$\omega$']
quartiles = np.arange(0,1.25,.25)
spike_percentiles = np.arange(0,1.25,.25)
spike_percentiles[-1]=.99
spk_percentile2 = np.arange(.125,1.125,.25)
paper_fig_dir = (params['fig_dir'].parent.parent.parent / 'PaperFigs')
paper_fig_dir.mkdir(parents=True, exist_ok=True)
pdf_name = paper_fig_dir/ ('MoViAdMuHF_AllCells_CellSummary.pdf')
totcelln = 0
if save_pdf:
    pdf = PdfPages(pdf_name)
    fig = plt.figure(constrained_layout=False, figsize=(30,10))
    gs0 = gridspec.GridSpec(nrows=2, ncols=5, figure=fig,wspace=.8,hspace=.3)

    ########## Histograms of R2 ##########
    axs0a = np.array([fig.add_subplot(gs0[0, n]) for n in range(gs0.ncols)])
    axs0b = np.array([fig.add_subplot(gs0[1, n]) for n in range(gs0.ncols)])

    vals_Mot = np.hstack([All_data[date_ani2[da]]['Mot_r2_test'] for da in range(len(dates_all))])
    vals_Vis = np.hstack([All_data[date_ani2[da]]['Vis_r2_test'] for da in range(len(dates_all))])
    vals_Add = np.hstack([All_data[date_ani2[da]]['Add_r2_test'] for da in range(len(dates_all))])
    vals_Mul = np.hstack([All_data[date_ani2[da]]['Mul_r2_test'] for da in range(len(dates_all))])
    vals_HF  = np.hstack([All_data[date_ani2[da]]['HF_r2_test'] for da in range(len(dates_all))])
    vals_FMHF  = np.hstack([All_data[date_ani2[da]]['HF_FMHF_r2']  for da in range(len(dates_all))])

    r2_all = np.stack((vals_Mot,vals_Vis,vals_Add,vals_Mul,vals_HF))
    hbins=.05
    for n in range(len(mod_titles)):
        ax = axs0a[n]
        count,edges = np.histogram(r2_all[n],bins=np.arange(0,1,hbins))
        edges_mid = np.array([(edges[i]+edges[i+1])/2 for i in range(len(edges)-1)])
        ax.bar(edges_mid, count/len(r2_all[n]),color=mod_clrs[n],width=hbins, alpha=1)
        ax.set_xticks(np.arange(0,1.2,.2))
        ax.set_xticklabels(np.round(np.arange(0,1.2,.2),decimals=1))
        ax.set_title(mod_titles[n])
        ax.set_xlabel(r'$r^2$')

    ax=axs0b[-1]
    count,edges = np.histogram(np.abs(vals_FMHF),bins=np.arange(0,1,hbins))
    edges_mid = np.array([(edges[i]+edges[i+1])/2 for i in range(len(edges)-1)])
    ax.bar(edges_mid, count/len(r2_all[n]),color=mod_clrs[n],width=hbins, alpha=1)
    ax.set_xticks(np.arange(0,1.2,.2))
    ax.set_xticklabels(np.round(np.arange(0,1.2,.2),decimals=1))
    ax.set_title('FM_HF_CC')
    ax.set_xlabel('cc')
    ########## Scatter of all R2 ########## 

    tuning_idx = np.where(tuning_sig_all)[0]
    lims = (0, .65)
    crange=500
    ##### Vis vs Mot #####
    ax = axs0b[0]
    im=ax.scatter(vals_Vis,vals_Mot,c=celltypes_all[1],cmap='jet',s=20,vmin=-crange,vmax=crange)
    ax.plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
    ax.set_xlabel('Visual: $r^2$',fontsize=fontsize)
    ax.set_ylabel('Position: $r^2$',fontsize=fontsize)
    ax.set_yticks(np.arange(0,lims[1],.2))
    ax.set_yticklabels(np.round(np.arange(0,lims[1],.2),decimals=1))
    ax.set_ylim(0,lims[1])
    ax.set_xlim(0,lims[1])
    ax.set_xticks(np.arange(0,lims[1],.2))
    ax.set_xticklabels(np.round(np.arange(0,lims[1],.2),decimals=1))
    ax.axis('equal')
    cbar = add_colorbar(im)
    cbar.set_ticks([-crange,0,crange])
    cbar.set_ticklabels([-crange,0,crange])
    cbar.ax.tick_params(labelsize=fontsize-10)
    cbar.set_label(r'depth $(\mu m)$')

    ##### Vis vs Add/Mul #####
    cmap_mod = mpl.colors.ListedColormap(mod_clrs[2:4])
    vals_AM = np.max(np.stack((vals_Add,vals_Mul)),axis=0)
    AvM = vals_Add<vals_Mul
    ax = axs0b[1]
    scatter=ax.scatter(vals_Vis,vals_AM,30,c=AvM,cmap=cmap_mod)
    ax.plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
    ax.set_xlabel('Visual: $r^2$',fontsize=fontsize)
    ax.set_ylabel('$r^2$',fontsize=fontsize)
    ax.set_yticks(np.arange(0,lims[1],.2))
    ax.set_yticklabels(np.round(np.arange(0,lims[1],.2),decimals=1))
    ax.set_ylim(0,lims[1])
    ax.set_xlim(0,lims[1])
    ax.set_xticks(np.arange(0,lims[1],.2))
    ax.set_xticklabels(np.round(np.arange(0,lims[1],.2),decimals=1))
    ax.axis('equal')
    # produce a legend with the unique colors from the scatter
    legend1 = ax.legend(*scatter.legend_elements(),fontsize=14,markerscale=0, handlelength=0, handletextpad=0,loc="best")
    legend1.get_texts()[0].set_text('Add') 
    legend1.get_texts()[0].set_color(mod_clrs[2])
    legend1.get_texts()[1].set_text('Mul')
    legend1.get_texts()[1].set_color(mod_clrs[3])

    vloss_Mot = np.hstack([All_data[date_ani2[da]]['Mot_vloss_trace'] for da in range(len(dates_all))])
    vloss_Vis = np.hstack([All_data[date_ani2[da]]['Vis_vloss_trace'] for da in range(len(dates_all))])
    vloss_Add = np.hstack([All_data[date_ani2[da]]['Add_vloss_trace'] for da in range(len(dates_all))])
    vloss_Mul = np.hstack([All_data[date_ani2[da]]['Mul_vloss_trace'] for da in range(len(dates_all))])

    ##### MSE #####
    diff = vloss_Add-vloss_Mul
    increase = 1-vloss_Vis
    th = increase>-0

    NMul = np.sum(diff[th]>0)
    NAdd = np.sum(diff[th]<0)
    ax=axs0b[2]
    im=ax.scatter(diff[th],increase[th],c=celltypes_all[1,th],cmap='jet',s=20,vmin=-500,vmax=500)
    ax.axvline(x=0,c='k')
    ax.set_ylabel(r'$1-(MSE_{Vis}/Var)$',fontsize=fontsize)
    ax.set_xlabel('MSE: Add-Mul',fontsize=fontsize)
    ax.set_title(r'$N_{{Add}}$:{}, $N_{{Mul}}$:{}'.format(NAdd,NMul),fontsize=fontsize)
    ax.set_ylim(-.05,.25)
    ax.set_xlim(-.02,.02)
    cbar = add_colorbar(im)
    cbar.set_ticks([-crange,0,crange])
    cbar.set_ticklabels([-crange,0,crange])
    cbar.ax.tick_params(labelsize=fontsize-10)
    cbar.set_label(r'depth $(\mu m)$')

    ##### R2 Explained Variance #####
    diff = vals_Mul-vals_Add
    increase = 1-vloss_Vis
    th = increase>-0

    NMul = np.sum(diff[th]>0)
    NAdd = np.sum(diff[th]<0)
    ax=axs0b[3]
    im = ax.scatter(diff[th],increase[th],c=celltypes_all[1,th],cmap='jet',s=20,vmin=-500,vmax=500)
    ax.axvline(x=0,c='k')
    ax.set_ylabel(r'$1-(MSE_{Vis}/Var)$',fontsize=fontsize)
    ax.set_xlabel('R2: Mul-Add',fontsize=fontsize)
    ax.set_title(r'$N_{{Add}}$:{}, $N_{{Mul}}$:{}'.format(NAdd,NMul),fontsize=fontsize)
    ax.set_ylim(-.05,.25)
    ax.set_xlim(-.075,.075)
    cbar = add_colorbar(im)
    cbar.set_ticks([-crange,0,crange])
    cbar.set_ticklabels([-crange,0,crange])
    cbar.ax.tick_params(labelsize=fontsize-10)
    cbar.set_label(r'depth $(\mu m)$')

    plt.tight_layout()
    pdf.savefig()
    plt.close()
for da in tqdm(np.arange(len(date_ani2))):
    args['date_ani'] = dates_all[da]
    args['free_move'] = True
    params,_,_ = load_params(0,Kfold,args,debug=True)
    data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')
    ncells=All_data[date_ani2[da]][mod_titles[0]+'_pred_smooth'].shape[-1]
    for celln in tqdm(np.arange(ncells)):
        predcell = All_data[date_ani2[da]][mod_titles[1]+'_pred_smooth'][:,celln]#/params['model_dt']
        nspcell = All_data[date_ani2[da]]['actual_smooth'][:,celln]#/params['model_dt']
        pred_range = np.quantile(predcell,[.1,.9])
        test_nsp_range = np.quantile(nspcell,[.01,1])
        pred_rangelin = np.quantile(predcell,spike_percentiles)
        xbin_pts = np.quantile(predcell,spk_percentile2)
        stat_bins = len(pred_rangelin)
        fig = plt.figure(constrained_layout=False, figsize=(30,25))
        gs0 = gridspec.GridSpec(nrows=3, ncols=4, figure=fig,wspace=.5,hspace=.3)

        ########## Plotting FM/HF RFs ##########
        gs00a = gridspec.GridSpecFromSubplotSpec(2, 5, subplot_spec=gs0[:1,:4],wspace=.5,hspace=.5)
        axs0a = np.array([fig.add_subplot(gs00a[0, n]) for n in range(gs00a.ncols)])
        axs0b = np.array([fig.add_subplot(gs00a[1, n]) for n in range(gs00a.ncols)])

        ax = axs0a[0]
        crange = np.max(np.abs(All_data[date_ani2[da]]['Vis_rf_up'][celln]))
        im = ax.imshow(All_data[date_ani2[da]]['Vis_rf_up'][celln,0],'RdBu_r',vmin=-crange,vmax=crange)
        cbar = add_colorbar(im)
        ax.axis('off')
        ax.set_title('FM RF')

        ax = axs0a[1]
        ax.plot(All_data[date_ani2[da]]['actual_smooth'][:,celln])
        ax.plot(All_data[date_ani2[da]]['Vis_pred_smooth'][:,celln])
        # ax.set_title('$r^2$: {:.03f}'.format(r2_FM[celln]))

        ax = axs0a[2]
        ax.plot(All_data[date_ani2[da]]['Vis_loss_regcv'][0,:,celln])
        ax.set_ylabel('MSE')
        ax.set_xlabel(r'$\lambda_n$')
        ax.set_title('Best lam:{}'.format(np.nanargmin(All_data[date_ani2[da]]['Vis_loss_regcv'][0,:,celln])))

        ax =axs0a[3]
        for modN in range(len(mod_titles)-1):
            ax.plot(All_data[date_ani2[da]][mod_titles[modN]+'_vloss_all'][celln],c=mod_clrs[modN])
        ax.set_ylabel('MSE')
        ax.set_xlabel('batches')

        ########### Head Fixed plots ###########
        ax = axs0b[0]
        crange = np.max(np.abs(All_data[date_ani2[da]]['HF_rf_up'][celln]))
        im = ax.imshow(All_data[date_ani2[da]]['HF_rf_up'][celln,0],'RdBu_r',vmin=-crange,vmax=crange)
        cbar = add_colorbar(im)
        ax.set_title('HF RF')
        ax.axis('off')

        ax = axs0b[1]
        ax.plot(All_data[date_ani2[da]]['actual_HF_smooth'][:,celln])
        ax.plot(All_data[date_ani2[da]]['HF_pred_smooth'][:,celln])
        # ax.set_title('$r^2$: {:.03f}'.format(r2_HF[celln]))

        ax = axs0b[2]
        ax.plot(All_data[date_ani2[da]]['HF_loss_regcv'][0,:,celln])
        ax.set_ylabel('MSE')
        ax.set_xlabel(r'$\lambda_n$')
        ax.set_title('Best lam:{}'.format(np.nanargmin(All_data[date_ani2[da]]['HF_loss_regcv'][0,:,celln])))

        ax =axs0b[3]
        ax.plot(All_data[date_ani2[da]][mod_titles[-1]+'_vloss_all'][celln],c='k')
        ax.set_ylabel('MSE')
        ax.set_xlabel('batches')

        ######### Position Weights ##########

        w_move_Add = All_data[date_ani2[da]][mod_titles[2]+'_moveW']
        w_move_Mov = All_data[date_ani2[da]][mod_titles[3]+'_moveW']
        for modeln in range(len(titles)):
            axs0a[4].bar(modeln, w_move_Add[celln,modeln], color=move_clrs[modeln])
            axs0a[4].set_xticks(np.arange(0,len(titles)))
            axs0a[4].set_xticklabels(titles)
            axs0a[4].set_ylabel('GLM Weight')
            axs0a[4].set_title('Add Weights')

            axs0b[4].bar(modeln, w_move_Mov[celln,modeln], color=move_clrs[modeln])
            axs0b[4].set_xticks(np.arange(0,len(titles)))
            axs0b[4].set_xticklabels(titles)
            axs0b[4].set_ylabel('GLM Weight')
            axs0b[4].set_title('Mul Weights')



        ########## Fig 4A ########## 
        gs00 = gridspec.GridSpecFromSubplotSpec(2, len(titles), subplot_spec=gs0[1,:4],wspace=.5,hspace=.7)
        axs1 = np.array([fig.add_subplot(gs00[n,m]) for n in range(1) for m in range(len(titles))]).reshape(1,len(titles))
        axs1b = np.array([fig.add_subplot(gs00[n,m]) for n in range(1,2) for m in range(len(titles))]).reshape(1,len(titles))

        top_yaxs = np.max(ax_ylims[celln])+2*np.max(tuning_stds[celln])
        traces = np.zeros((ncells,len(titles),len(quartiles)-1,stat_bins-1))
        traces_mean = np.zeros((ncells,len(titles),stat_bins-1))
        edges_all = np.zeros((ncells,len(titles),len(quartiles)-1,stat_bins-1))
        for i,modeln in enumerate(range(len(titles))):
            metric = move_data[params['bin_length']:-params['bin_length'],modeln]
            nranges = np.quantile(metric,quartiles)
            stat_range, edges, _ = binned_statistic(metric,nsp_raw[params['bin_length']:-params['bin_length'],celln],statistic='mean',bins=nranges)
            edge_mids = np.quantile(metric,spk_percentile2)#
            for m in range(len(nranges)-1):
                axs1[0,modeln].axvspan(nranges[m], nranges[m+1],ymin=0,ymax=1,alpha=0.8, color=q_clrs[m],zorder=0)
            axs1[0,modeln].plot(edge_mids,stat_range/params['model_dt'],'.-', ms=20, lw=4,c=move_clrs[modeln])
            axs1[0,modeln].set_ylim(bottom=0,top=np.max(ax_ylims,axis=1)[celln]+2*np.nanmax(tuning_stds,axis=(1,2))[celln])
            xlim_range = np.max(np.abs([nranges[0],nranges[-1]]))
            axs1[0,modeln].set_xlim(-xlim_range,xlim_range)
            axs1[0,modeln].set_xlabel('angle (deg)',fontsize=fontsize)
            axs1[0,modeln].set_ylabel('sp/sec',fontsize=fontsize)
            axs1[0,modeln].set_title(titles[modeln],fontsize=fontsize)

            
            stat_all, edges, _ = binned_statistic(predcell,nspcell,statistic='mean',bins=pred_rangelin)
            edge_mids = xbin_pts 
            traces_mean[celln,modeln]=stat_all
            max_fr = np.max(stat_all)
            for n in range(len(nranges)-1):
                ind = np.where(((metric<=nranges[n+1])&(metric>nranges[n])))[0]
                pred = predcell[ind]
                sp = nspcell[ind]
                stat_range, edges, _ = binned_statistic(pred, sp, statistic='mean',bins=pred_rangelin)
                edge_mids = xbin_pts
                traces[celln,modeln,n]=stat_range
                edges_all[celln,modeln,n]=edge_mids
                axs1b[0,modeln].plot(edge_mids, stat_range,'.-', c=q_clrs[n],label='{:.02f} : {:.02f}'.format(nranges[n],nranges[n+1]),lw=4,ms=20,alpha=.9)
                axs1b[0,modeln].set_xlabel('predicted (sp/sec)',fontsize=fontsize)
                axs1b[0,modeln].set_ylabel('actual (sp/sec)',fontsize=fontsize)
            
            lim_max = np.nanmax(np.hstack((edge_mids,traces[celln,modeln].flatten())))+.5*np.std(edges)
            lim_min = np.nanmin(np.hstack((edge_mids,traces[celln,modeln].flatten())))-.5*np.std(edges)
            lims = (0, lim_max) if (lim_min)<0 else (lim_min,lim_max) 
            axs1b[0,modeln].plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
            axs1b[0,modeln].plot(edge_mids, stat_all,'.-', c='k', lw=5, ms=20, label='All_data', alpha=.8)
            # axs1b[0,modeln].axis('equal')
            axs1b[0,modeln].set(xlim=lims, ylim=lims)
            # axs1b[0,modeln].set_ylim(bottom=0)



        ########## Fig 4B ########## 
        gs01 = gridspec.GridSpecFromSubplotSpec(2,1, subplot_spec=gs0[2,:4],wspace=.05,hspace=.5)
        axs2 = np.array([fig.add_subplot(gs01[n,m]) for n in range(1) for m in range(1)]).reshape(1,1)
        axs2b = np.array([fig.add_subplot(gs01[n,m]) for n in range(1,2) for m in range(1)]).reshape(1,1)

        cmap = mpl.colors.ListedColormap(q_clrs)
        ##### Plotting Position Scatter #####
        t = np.argmin(np.abs((np.arange(All_data[date_ani2[da]]['actual_smooth'].shape[0])*params['model_dt'])-0)) # 200 seconds in #6000
        dt = int(200/params['model_dt']) #All_data[date_ani2[da]]['actual_smooth'].shape[0] # dt in seconds converted to time points
        pred_time = (np.arange(All_data[date_ani2[da]]['actual_smooth'].shape[0])*params['model_dt'])[0:np.abs(dt)]
        ax = axs2[0,0]
        metric = move_data[params['bin_length']:-params['bin_length'],3]
        ax.scatter(pred_time, metric[t:t+dt],s=10,c=np.digitize(metric,nranges)[t:t+dt],cmap=cmap)
        ax.set_ylabel(r'$\omega$ (deg)',fontsize=fontsize)
        ax.set_xlabel('time (s)',fontsize=fontsize)
        ax.set_ylim([-anglim,anglim])

        # celln = 18#106
        # da = 0
        # t = 6000 #np.argmin(np.abs(pred_time-375)) # T seconds in 
        # dt = 3000 #All_data[date_ani2[da]]['actual_smooth'].shape[0]
        ##### Plotting Firing Rate #####
        pred_time = (np.arange(All_data[date_ani2[da]]['actual_smooth'].shape[0])*params['model_dt'])[0:np.abs(dt)]
        ax = axs2b[0,0]
        zorder = np.argsort([All_data[date_ani2[da]][mod_titles[modN]+'_r2_test'][celln] for modN in range(len(mod_titles))])+1
        ax.plot(pred_time,All_data[date_ani2[da]]['actual_smooth'][t:t+dt,celln],c='k',lw=3)
        for modN in range(len(mod_titles)-1):
            ax.plot(pred_time,All_data[date_ani2[da]][mod_titles[modN]+'_pred_smooth'][t:t+dt,celln],c=mod_clrs[modN],lw=3,
                    label='{} $r^2$: {:.02f}'.format(mod_titles[modN],All_data[date_ani2[da]][mod_titles[modN]+'_r2_test'][celln]),zorder=zorder[n])
        ax.legend(labelcolor='linecolor', fontsize=14, handlelength=0, handletextpad=0,loc='upper right', bbox_to_anchor=(1.05, 1))
        ax.set_xlabel('time (s)',fontsize=fontsize)
        ax.set_ylabel('sp/sec',fontsize=fontsize)

        Mot_r2 = All_data[date_ani2[da]]['Mot_r2_test'][celln]
        Vis_r2 = All_data[date_ani2[da]]['Vis_r2_test'][celln]
        Add_r2 = All_data[date_ani2[da]]['Add_r2_test'][celln]
        Mul_r2 = All_data[date_ani2[da]]['Mul_r2_test'][celln]
        FMHF_r2 = All_data[date_ani2[da]]['HF_FMHF_r2'][celln]
        plt.suptitle('{} celln: {} ({}) \n Mot $r^2$:{:.03f}, Vis $r^2$:{:.03f}, Add $r^2$:{:.03f}, Mult $r^2$:{:.03f} \n FM_HF_CC: {:.02f}'.format(date_ani2[da],celln,totcelln,Mot_r2,Vis_r2,Add_r2,Mul_r2,FMHF_r2))
        totcelln = totcelln+1
        # fig.savefig(paper_fig_dir/'Fig4_draft_Cell52_101521.png', facecolor='white', transparent=True, bbox_inches='tight')
        # plt.tight_layout()
        if save_pdf:
            pdf.savefig()
            plt.close()
if save_pdf:
    pdf.close()


### Single Cell Comparison

In [ ]:
da = 0
args['date_ani'] = dates_all[da]
args['free_move'] = True
params,_,_ = load_params(1,Kfold,args,debug=True)
data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')
ncells=All_data[date_ani2[da]][mod_titles[0]+'_pred_smooth'].shape[-1]


In [ ]:
celln = 105
modN = 1
fontsize=24
anglim=65
titles = [r'$\theta$',r'$\phi$',r'$\rho$',r'$\omega$']
quartiles = np.arange(0,1.25,.25)
spike_percentiles = np.arange(0,1.25,.25)
spike_percentiles[-1]=.99
spk_percentile2 = np.arange(.125,1.125,.25)
predcell = All_data[date_ani2[da]][mod_titles[1]+'_pred_smooth'][:,celln]#/params['model_dt']
nspcell = All_data[date_ani2[da]]['actual_smooth'][:,celln]#/params['model_dt']
pred_range = np.quantile(predcell,[.1,.9])
test_nsp_range = np.quantile(nspcell,[.01,1])
pred_rangelin = np.quantile(predcell,spike_percentiles)
xbin_pts = np.quantile(predcell,spk_percentile2)
stat_bins = len(pred_rangelin)
fig = plt.figure(constrained_layout=False, figsize=(30,25))
gs0 = gridspec.GridSpec(nrows=3, ncols=4, figure=fig,wspace=.5,hspace=.3)

########## Plotting FM/HF RFs ##########
gs00a = gridspec.GridSpecFromSubplotSpec(2, 5, subplot_spec=gs0[:1,:4],wspace=.5,hspace=.5)
axs0a = np.array([fig.add_subplot(gs00a[0, n]) for n in range(gs00a.ncols)])
axs0b = np.array([fig.add_subplot(gs00a[1, n]) for n in range(gs00a.ncols)])

ax = axs0a[0]
crange = np.max(np.abs(All_data[date_ani2[da]]['Vis_rf_up'][celln]))
im = ax.imshow(All_data[date_ani2[da]]['Vis_rf_up'][celln,2],'RdBu_r',vmin=-crange,vmax=crange)
cbar = add_colorbar(im)
ax.axis('off')
ax.set_title('FM RF')

ax = axs0a[1]
ax.plot(All_data[date_ani2[da]]['actual_smooth'][:,celln])
ax.plot(All_data[date_ani2[da]]['Vis_pred_smooth'][:,celln])
ax.set_title('$r^2$: {:.03f}'.format(All_data[date_ani2[da]]['Vis_r2_test'][celln]))

ax = axs0a[2]
ax.plot(All_data[date_ani2[da]]['Vis_loss_regcv'][0,:,celln])
ax.set_ylabel('MSE')
ax.set_xlabel(r'$\lambda_n$')
ax.set_title('Best lam:{}'.format(np.nanargmin(All_data[date_ani2[da]]['Vis_loss_regcv'][0,:,celln])))

ax =axs0a[3]
for modN in range(len(mod_titles)-1):
    ax.plot(All_data[date_ani2[da]][mod_titles[modN]+'_vloss_all'][celln],c=mod_clrs[modN])
ax.set_ylabel('MSE')
ax.set_xlabel('batches')

########### Head Fixed plots ###########
ax = axs0b[0]
crange = np.max(np.abs(All_data[date_ani2[da]]['HF_rf_up'][celln]))
im = ax.imshow(All_data[date_ani2[da]]['HF_rf_up'][celln,2],'RdBu_r',vmin=-crange,vmax=crange)
cbar = add_colorbar(im)
ax.set_title('HF RF')
ax.axis('off')

ax = axs0b[1]
ax.plot(All_data[date_ani2[da]]['actual_HF_smooth'][:,celln])
ax.plot(All_data[date_ani2[da]]['HF_pred_smooth'][:,celln])
ax.set_title('$r^2$: {:.03f}'.format(All_data[date_ani2[da]]['HF_r2_test'][celln]))

ax = axs0b[2]
ax.plot(All_data[date_ani2[da]]['HF_loss_regcv'][0,:,celln])
ax.set_ylabel('MSE')
ax.set_xlabel(r'$\lambda_n$')
ax.set_title('Best lam:{}'.format(np.nanargmin(All_data[date_ani2[da]]['HF_loss_regcv'][0,:,celln])))

ax =axs0b[3]
ax.plot(All_data[date_ani2[da]][mod_titles[-1]+'_vloss_all'][celln],c='k')
ax.set_ylabel('MSE')
ax.set_xlabel('batches')

######### Position Weights ##########

w_move_Add = All_data[date_ani2[da]][mod_titles[2]+'_moveW']
w_move_Mov = All_data[date_ani2[da]][mod_titles[3]+'_moveW']
for modeln in range(len(titles)):
    axs0a[4].bar(modeln, w_move_Add[celln,modeln], color=move_clrs[modeln])
    axs0a[4].set_xticks(np.arange(0,len(titles)))
    axs0a[4].set_xticklabels(titles)
    axs0a[4].set_ylabel('GLM Weight')
    axs0a[4].set_title('Add Weights')

    axs0b[4].bar(modeln, w_move_Mov[celln,modeln], color=move_clrs[modeln])
    axs0b[4].set_xticks(np.arange(0,len(titles)))
    axs0b[4].set_xticklabels(titles)
    axs0b[4].set_ylabel('GLM Weight')
    axs0b[4].set_title('Mul Weights')



########## Fig 4A ########## 
gs00 = gridspec.GridSpecFromSubplotSpec(2, len(titles), subplot_spec=gs0[1,:4],wspace=.5,hspace=.7)
axs1 = np.array([fig.add_subplot(gs00[n,m]) for n in range(1) for m in range(len(titles))]).reshape(1,len(titles))
axs1b = np.array([fig.add_subplot(gs00[n,m]) for n in range(1,2) for m in range(len(titles))]).reshape(1,len(titles))

top_yaxs = np.max(ax_ylims[celln])+2*np.max(tuning_stds[celln])
traces = np.zeros((ncells,len(titles),len(quartiles)-1,stat_bins-1))
traces_mean = np.zeros((ncells,len(titles),stat_bins-1))
edges_all = np.zeros((ncells,len(titles),len(quartiles)-1,stat_bins-1))
for i,modeln in enumerate(range(len(titles))):
    metric = move_data[params['bin_length']:-params['bin_length'],modeln]
    nranges = np.quantile(metric,quartiles)
    stat_range, edges, _ = binned_statistic(metric,nsp_raw[params['bin_length']:-params['bin_length'],celln],statistic='mean',bins=nranges)
    edge_mids = np.quantile(metric,spk_percentile2)#
    for m in range(len(nranges)-1):
        axs1[0,modeln].axvspan(nranges[m], nranges[m+1],ymin=0,ymax=1,alpha=0.8, color=q_clrs[m],zorder=0)
    axs1[0,modeln].plot(edge_mids,stat_range/params['model_dt'],'.-', ms=20, lw=4,c=move_clrs[modeln])
    axs1[0,modeln].set_ylim(bottom=0,top=np.max(ax_ylims,axis=1)[celln]+2*np.nanmax(tuning_stds,axis=(1,2))[celln])
    xlim_range = np.max(np.abs([nranges[0],nranges[-1]]))
    axs1[0,modeln].set_xlim(-xlim_range,xlim_range)
    axs1[0,modeln].set_xlabel('angle (deg)',fontsize=fontsize)
    axs1[0,modeln].set_ylabel('sp/sec',fontsize=fontsize)
    axs1[0,modeln].set_title(titles[modeln],fontsize=fontsize)

    
    stat_all, edges, _ = binned_statistic(predcell,nspcell,statistic='mean',bins=pred_rangelin)
    edge_mids = xbin_pts 
    traces_mean[celln,modeln]=stat_all
    max_fr = np.max(stat_all)
    for n in range(len(nranges)-1):
        ind = np.where(((metric<=nranges[n+1])&(metric>nranges[n])))[0]
        pred = predcell[ind]
        sp = nspcell[ind]
        stat_range, edges, _ = binned_statistic(pred, sp, statistic='mean',bins=pred_rangelin)
        edge_mids = xbin_pts
        traces[celln,modeln,n]=stat_range
        edges_all[celln,modeln,n]=edge_mids
        axs1b[0,modeln].plot(edge_mids, stat_range,'.-', c=q_clrs[n],label='{:.02f} : {:.02f}'.format(nranges[n],nranges[n+1]),lw=4,ms=20,alpha=.9)
        axs1b[0,modeln].set_xlabel('predicted (sp/sec)',fontsize=fontsize)
        axs1b[0,modeln].set_ylabel('actual (sp/sec)',fontsize=fontsize)
        axs1b[0,modeln].axis('square')
    
    lim_max = np.nanmax(np.hstack((edge_mids,traces[celln,modeln].flatten())))+.5*np.std(edges)
    lim_min = np.nanmin(np.hstack((edge_mids,traces[celln,modeln].flatten())))-.5*np.std(edges)
    lims = (0, lim_max) if (lim_min)<0 else (lim_min,lim_max) 
    axs1b[0,modeln].plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
    axs1b[0,modeln].plot(edge_mids, stat_all,'.-', c='k', lw=5, ms=20, label='All_data', alpha=.8)
    # axs1b[0,modeln].axis('equal')
    axs1b[0,modeln].set(xlim=lims, ylim=lims)
    # axs1b[0,modeln].set_ylim(bottom=0)



########## Fig 4B ########## 
gs01 = gridspec.GridSpecFromSubplotSpec(2,1, subplot_spec=gs0[2,:4],wspace=.05,hspace=.5)
axs2 = np.array([fig.add_subplot(gs01[n,m]) for n in range(1) for m in range(1)]).reshape(1,1)
axs2b = np.array([fig.add_subplot(gs01[n,m]) for n in range(1,2) for m in range(1)]).reshape(1,1)

cmap = mpl.colors.ListedColormap(q_clrs)
##### Plotting Position Scatter #####
t0=300
t = np.argmin(np.abs((np.arange(All_data[date_ani2[da]]['actual_smooth'].shape[0])*params['model_dt'])-t0)) # 200 seconds in #6000
dt = int(150/params['model_dt']) # All_data[date_ani2[da]]['actual_smooth'].shape[0] # dt in seconds converted to time points
pred_time = (np.arange(All_data[date_ani2[da]]['actual_smooth'].shape[0])*params['model_dt'])[0:np.abs(dt)]
ax = axs2[0,0]
metric = move_data[params['bin_length']:-params['bin_length'],3]
ax.scatter(pred_time, metric[t:t+dt],s=10,c=np.digitize(metric,nranges)[t:t+dt],cmap=cmap)
ax.set_ylabel(r'$\omega$ (deg)',fontsize=fontsize)
ax.set_xlabel('time (s)',fontsize=fontsize)
ax.set_ylim([-anglim,anglim])

# celln = 18#106
# da = 0
# t = 6000 #np.argmin(np.abs(pred_time-375)) # T seconds in 
# dt = 3000 #All_data[date_ani2[da]]['actual_smooth'].shape[0]
##### Plotting Firing Rate #####
pred_time = (np.arange(All_data[date_ani2[da]]['actual_smooth'].shape[0])*params['model_dt'])[0:np.abs(dt)]
ax = axs2b[0,0]
zorder = np.argsort([All_data[date_ani2[da]][mod_titles[modN]+'_r2_test'][celln] for modN in range(len(mod_titles))])+1
ax.plot(pred_time,All_data[date_ani2[da]]['actual_smooth'][t:t+dt,celln],c='k',lw=3)
for modN in range(len(mod_titles)-1):
    ax.plot(pred_time,All_data[date_ani2[da]][mod_titles[modN]+'_pred_smooth'][t:t+dt,celln],c=mod_clrs[modN],lw=3,
            label='{} $r^2$: {:.02f}'.format(mod_titles[modN],All_data[date_ani2[da]][mod_titles[modN]+'_r2_test'][celln]),zorder=zorder[n])
ax.legend(labelcolor='linecolor', fontsize=14, handlelength=0, handletextpad=0,loc='upper right', bbox_to_anchor=(1.05, 1))
ax.set_xlabel('time (s)',fontsize=fontsize)
ax.set_ylabel('sp/sec',fontsize=fontsize)

Mot_r2 = All_data[date_ani2[da]]['Mot_r2_test'][celln]
Vis_r2 = All_data[date_ani2[da]]['Vis_r2_test'][celln]
Add_r2 = All_data[date_ani2[da]]['Add_r2_test'][celln]
Mul_r2 = All_data[date_ani2[da]]['Mul_r2_test'][celln]
FMHF_r2 = All_data[date_ani2[da]]['HF_FMHF_r2'][celln]
plt.suptitle('{} celln: {} ({}) \n Mot $r^2$:{:.03f}, Vis $r^2$:{:.03f}, Add $r^2$:{:.03f}, Mult $r^2$:{:.03f} \n FM_HF_CC: {:.02f}'.format(date_ani2[da],celln,celln,Mot_r2,Vis_r2,Add_r2,Mul_r2,FMHF_r2))
plt.show()

### Scatter all data

In [ ]:
celltype_dir = params['data_dir'].parent.parent.parent.parent / 'batch_files/120221_hffm'
celltype_file = celltype_dir / 'pooled_ephys_population_update_120621.pickle'
celltype = pickle.load(open(celltype_file,'rb'))
date_exp = [n for n,da in enumerate(celltype['session'].unique()) if params['date_ani2'] in da][0]

In [ ]:
Kfold = 0
args['free_move']=True
mod_titles = ['Mot','Vis','Add','Mul']
mod_clrs = ["#136f63","#e0ca3c","#f34213","#80a4ed"]
dates_all = ['070921/J553RT' ,'101521/J559NC','102821/J570LT','110421/J569LT'] # '102621/J558NC' 
ModelList_all = []
test_std =[]
tuning_sig_all = []
NCells_all = []
bad_cells_all = []
for d,date_ani in enumerate(dates_all):
    args['date_ani'] = date_ani
    args['NoL1']=False
    args['Nepochs'] = 5000
    date_ani2 = params['save_dir'].parent.parent.parent/date_ani
    if args['NoL1'] == True: 
        L1 = 'NoL1'
        mod_name = '*{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    else: 
        L1 = 'withL1'
        mod_name = '*{}_dt{:03d}_T{:02d}*_NB{}_Kfold{:02d}_best'.format(L1,int(params['model_dt']*1000), params['nt_glm_lag'], params['Nepochs'],Kfold)
    ModelList = np.array(sorted(list(date_ani2.rglob(mod_name+'.h5'))))[:-1]
    ModelList_all.append(ModelList)
    params,_,_ = load_params(0,Kfold,args,debug=True)
    bad_cells = np.load(params['save_dir_hf']/'bad_cells.npy')
    bad_cells_all.append(bad_cells)
    data,move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')
    test_std.append(np.var(data['test_nsp'],axis=0))
    tuning_sig_all.append(tuning_sig2)
    NCells_all.append(len(tuning_sig2))
ModelList_all = np.stack(ModelList_all)
test_std=np.hstack(test_std)
tuning_sig_all = np.hstack(tuning_sig_all)

celltypes_all = []
for n,date_exp in enumerate([2,4,7,9]):
    celltype2 = celltype[celltype['session']==celltype['session'].unique()[date_exp]].reset_index(drop=True)
    celltype2 = celltype2.drop(bad_cells_all[n])
    celltypes_all.append([celltype2['exc_or_inh'] == 'exc',celltype2['hf1_wn_depth_from_layer5']])
    
celltypes_all = np.hstack(celltypes_all)

In [ ]:

r2_all = []
vloss_all = []
cell_tot = 0
for modeln, mod in enumerate(mod_titles):
    r2 = []
    vloss = []
    for fmod in ModelList_all[:,modeln]:
        GLM_Data = ioh5.load(fmod)
        r2.append(GLM_Data['r2_test'])
        vloss.append(np.mean(GLM_Data['vloss_trace_all'][:,-10:],axis=1))
    r2_all.append(np.hstack(r2))
    vloss_all.append(np.hstack(vloss))
r2_all = np.stack(r2_all)
vloss_all = np.stack(vloss_all)
r2_Mot,r2_Vis,r2_Add,r2_Mul = r2_all
vloss_Mot,vloss_Vis,vloss_Add,vloss_Mul = vloss_all/test_std

In [ ]:
r2_Mot.shape,r2_Vis.shape,r2_Add.shape,r2_Mul.shape, vloss_Mot.shape,vloss_Vis.shape,vloss_Add.shape,vloss_Mul.shape

In [ ]:

a = 0; l = 0
scat_clr = ["#f9c80e","#f86624","#43bccd","#662e9b"] # 
scat_clr_all = np.hstack([NCells_all[n]*[scat_clr[n]] for n in range(len(scat_clr))])
clrs_all, labels = pd.factorize(scat_clr_all)
cmap = mpl.colors.ListedColormap(scat_clr)
# scat_clr = ["#f9c80e","#f86624","#ea3546","#662e9b","#43bccd"]
thresh_fr = 1
tuning_thresh = .2
fontsize=24
bin_length=40

vals_Mot = r2_Mot[tuning_sig_all]
vals_Vis = r2_Vis[tuning_sig_all]
vals_Add = r2_Add[tuning_sig_all]
vals_Mul = r2_Mul[tuning_sig_all]
tuning_idx = np.where(tuning_sig_all)[0]
lims = (0, .6)

fig, axs = plt.subplots(2,2,figsize=(10,10))
##### Vis vs Mot #####
ax = axs[0,0]
ax.scatter(vals_Vis,vals_Mot,30,c=clrs_all[tuning_sig_all],cmap=cmap, label='Mul')
ax.plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
ax.set_xlabel('Visual: $r^2$',fontsize=fontsize)
ax.set_ylabel('Position: $r^2$',fontsize=fontsize)
ax.set_yticks(np.arange(0,lims[1],.1))
ax.set_yticklabels(np.round(np.arange(0,lims[1],.1),decimals=1))
# for i, txt in enumerate(np.arange(len(vals_Vis))):
#     ax.annotate(tuning_idx[i], (vals_Vis[i], vals_Mot[i]),fontsize=10)

##### Vis vs Add/Mul #####
ax = axs[0,1]
ax.scatter(vals_Vis,vals_Add,30,c=mod_clrs[2], label='Add')
ax.scatter(vals_Vis,vals_Mul,30,c=mod_clrs[3], label='Mul')
ax.plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
ax.set_xlabel('Visual: $r^2$',fontsize=fontsize)
ax.set_ylabel('Visual & Position: $r^2$',fontsize=fontsize)
ax.set_yticks(np.arange(0,lims[1],.1))
ax.set_yticklabels(np.round(np.arange(0,lims[1],.1),decimals=1))
leg = ax.legend()
for handle, text in zip(leg.legendHandles, leg.get_texts()):
    text.set_color(handle.get_facecolor()[0])
# for i, txt in enumerate(np.arange(len(vals_Vis))):
#     ax.annotate(tuning_idx[i], (vals_Vis[i], vals_Add[i]),fontsize=10,c=mod_clrs[2])
#     ax.annotate(tuning_idx[i], (vals_Vis[i], vals_Mul[i]),fontsize=10,c=mod_clrs[3])

##### MSE #####
diff = vloss_Add[tuning_sig_all]-vloss_Mul[tuning_sig_all]
increase = 1-vloss_Vis[tuning_sig_all]
th = increase>-100

NMul = np.sum(diff[th]>0)
NAdd = np.sum(diff[th]<0)
ax=axs[1,0]
ax.scatter(diff[th],increase[th],c=clrs_all[tuning_sig_all][th],cmap=cmap,s=20)
ax.axvline(x=0,c='k')
ax.set_ylabel('1-(NLVis_MSE/SD)')
ax.set_xlabel('MSE: Add-Mul')
ax.set_title('NAdd:{}, NMul:{}'.format(NAdd,NMul))
ax.set_ylim(-.05,.25)
ax.set_xlim(-.02,.02)
for i, txt in enumerate(np.arange(len(tuning_idx[th]))):
    ax.annotate(tuning_idx[th][i], (diff[th][i], increase[th][i]),fontsize=10)

##### R2 Explained Variance #####
diff = r2_Mul[tuning_sig_all]-r2_Add[tuning_sig_all]
increase = 1-vloss_Vis[tuning_sig_all]
th = increase>-100

NMul = np.sum(diff[th]>0)
NAdd = np.sum(diff[th]<0)
ax=axs[1,1]
im = ax.scatter(diff[th],increase[th],c=clrs_all[tuning_sig_all][th],cmap=cmap,s=20)
ax.axvline(x=0,c='k')
ax.set_ylabel('1-(NLVis_MSE/SD)')
ax.set_xlabel('R2: Mul-Add')
ax.set_title('NAdd:{}, NMul:{}'.format(NAdd,NMul))
ax.set_ylim(-.05,.25)
for i, txt in enumerate(np.arange(len(tuning_idx[th]))):
    ax.annotate(tuning_idx[th][i], (diff[th][i], increase[th][i]),fontsize=10)

cbar_ax = fig.add_axes([1, 0.15, 0.05, 0.7])
cbar2 = fig.colorbar(im, cax=cbar_ax)
cbar2.set_ticks(np.arange(.4,3.25,.75))
cbar2.set_ticklabels(dates_all)
cbar2.ax.tick_params(labelsize=fontsize)

plt.tight_layout() 
# fig.savefig(paper_fig_dir/('ScatterComparison.png'), facecolor='white', transparent=True, bbox_inches='tight')

In [ ]:
a = 0; l = 0
scat_clr = ["#f9c80e","#f86624","#43bccd","#662e9b"] # 
scat_clr_all = np.hstack([NCells_all[n]*[scat_clr[n]] for n in range(len(scat_clr))])
clrs_all, labels = pd.factorize(scat_clr_all)
cmap = mpl.colors.ListedColormap(scat_clr)
# scat_clr = ["#f9c80e","#f86624","#ea3546","#662e9b","#43bccd"]
thresh_fr = 1
tuning_thresh = .2
fontsize=24
bin_length=40

vals_Mot = r2_Mot[tuning_sig_all]
vals_Vis = r2_Vis[tuning_sig_all]
vals_Add = r2_Add[tuning_sig_all]
vals_Mul = r2_Mul[tuning_sig_all]
tuning_idx = np.where(tuning_sig_all)[0]
lims = (0, .6)

cell_T = 0
fig, axs = plt.subplots(2,2,figsize=(10,10))
##### Vis vs Mot #####
ax = axs[0,0]
ax.scatter(vals_Vis,vals_Mot,30,c=celltypes_all[cell_T][tuning_sig_all],cmap=cmap, label='Mul')
ax.plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
ax.set_xlabel('Visual: $r^2$',fontsize=fontsize)
ax.set_ylabel('Position: $r^2$',fontsize=fontsize)
ax.set_yticks(np.arange(0,lims[1],.1))
ax.set_yticklabels(np.round(np.arange(0,lims[1],.1),decimals=1))
# for i, txt in enumerate(np.arange(len(vals_Vis))):
#     ax.annotate(tuning_idx[i], (vals_Vis[i], vals_Mot[i]),fontsize=10)

##### Vis vs Add/Mul #####
ax = axs[0,1]
ax.scatter(vals_Vis,vals_Add,30,c=mod_clrs[2], label='Add')
ax.scatter(vals_Vis,vals_Mul,30,c=mod_clrs[3], label='Mul')
ax.plot(np.linspace(lims[0],lims[1]),np.linspace(lims[0],lims[1]),'k--',zorder=0)
ax.set_xlabel('Visual: $r^2$',fontsize=fontsize)
ax.set_ylabel('Visual & Position: $r^2$',fontsize=fontsize)
ax.set_yticks(np.arange(0,lims[1],.1))
ax.set_yticklabels(np.round(np.arange(0,lims[1],.1),decimals=1))
leg = ax.legend()
for handle, text in zip(leg.legendHandles, leg.get_texts()):
    text.set_color(handle.get_facecolor()[0])
# for i, txt in enumerate(np.arange(len(vals_Vis))):
#     ax.annotate(tuning_idx[i], (vals_Vis[i], vals_Add[i]),fontsize=10,c=mod_clrs[2])
#     ax.annotate(tuning_idx[i], (vals_Vis[i], vals_Mul[i]),fontsize=10,c=mod_clrs[3])

##### MSE #####
diff = vloss_Add[tuning_sig_all]-vloss_Mul[tuning_sig_all]
increase = 1-vloss_Vis[tuning_sig_all]
th = increase>-100

NMul = np.sum(diff[th]>0)
NAdd = np.sum(diff[th]<0)
ax=axs[1,0]
ax.scatter(diff[th],increase[th],c=celltypes_all[cell_T][tuning_sig_all][th],cmap=cmap,s=20)
ax.axvline(x=0,c='k')
ax.set_ylabel('1-(NLVis_MSE/SD)')
ax.set_xlabel('MSE: Add-Mul')
ax.set_title('NAdd:{}, NMul:{}'.format(NAdd,NMul))
ax.set_ylim(-.05,.25)
ax.set_xlim(-.02,.02)
# for i, txt in enumerate(np.arange(len(tuning_idx[th]))):
#     ax.annotate(tuning_idx[th][i], (diff[th][i], increase[th][i]),fontsize=10)

##### R2 Explained Variance #####
diff = r2_Mul[tuning_sig_all]-r2_Add[tuning_sig_all]
increase = 1-vloss_Vis[tuning_sig_all]
th = increase>-100

NMul = np.sum(diff[th]>0)
NAdd = np.sum(diff[th]<0)
ax=axs[1,1]
im = ax.scatter(diff[th],increase[th],c=celltypes_all[cell_T][tuning_sig_all][th],cmap=cmap,s=20)
ax.axvline(x=0,c='k')
ax.set_ylabel('1-(NLVis_MSE/SD)')
ax.set_xlabel('R2: Mul-Add')
ax.set_title('NAdd:{}, NMul:{}'.format(NAdd,NMul))
ax.set_ylim(-.05,.25)
# for i, txt in enumerate(np.arange(len(tuning_idx[th]))):
#     ax.annotate(tuning_idx[th][i], (diff[th][i], increase[th][i]),fontsize=10)

cbar_ax = fig.add_axes([1, 0.15, 0.05, 0.7])
cbar2 = fig.colorbar(im, cax=cbar_ax)
cbar2.set_ticks([0,1])
cbar2.set_ticklabels(['inh','exc'])
# cbar2.ax.tick_params(labezlsize=fontsize)

plt.tight_layout() 
fig.savefig(paper_fig_dir/('ScatterComparison_InhExc.png'), facecolor='white', transparent=True, bbox_inches='tight')

### Raster Plots

In [ ]:

ephys_file = save_dir/'RawEphysData.h5'
ephys_data = pd.read_hdf(ephys_file)
spikeT = ephys_data['spikeT']
Tmax = 5
cell_spikes = []
for celln,spks in spikeT.iteritems():
    cell_spikes.append(spks[(spks>0) &(spks < Tmax)])


In [ ]:
fontsize=24
fig,axs = plt.subplots(figsize=(5,5))
ax = axs
ax.eventplot(cell_spikes,color='k',linelengths=.5)
ax.set_xlabel('Time (sec)',fontsize=fontsize+5)
ax.set_ylabel('Cell #',fontsize=fontsize+5)
ax.set_yticks(np.arange(0,params['Ncells'],2))
ax.set_yticklabels(np.arange(0, params['Ncells'], 2),fontsize=fontsize)
ax.set_xticks(np.arange(0, 6, 2))
ax.set_xticklabels(np.arange(0, 6, 2), fontsize=fontsize)

plt.show()
fig.savefig(fig_dir/'SpikeRaster.pdf', facecolor='white', transparent=True, bbox_inches='tight')


### WorldCam/EyeCam/IMU Example

In [ ]:
celln = 12
bin_length=40
titles = np.array(['theta', 'phi', 'rho', 'omega'])
clrs = ['blue', 'orange', 'green', 'red']
fontsize=24
fig, axs = plt.subplots(4,1,figsize=(10,5),sharex=True)
for modeln,ax in enumerate(axs):
    sp_smooth = ((np.convolve(GLM_Vis['test_nsp'][:,celln], np.ones(bin_length), 'same')) / (bin_length * params['model_dt']))[int(bin_length*1.5):-int(bin_length*1.5)]
    smooth_move = np.zeros((len(titles), sp_smooth.shape[0]))
    smooth1 = ((np.convolve(move_test[:,modeln], np.ones(bin_length), 'same')) / (bin_length * params['model_dt']))[int(bin_length*1.5):-int(bin_length*1.5)]
    smooth_move[modeln] = (smooth1 - np.nanmean(smooth1))/np.max(smooth1)
    ax.plot(np.arange(smooth_move[modeln].shape[0])*params['model_dt'], smooth_move[modeln],c=clrs[modeln], label=r'$\{}$'.format(titles[modeln]),alpha=.75)
    ax.set_yticks([])
axs[-1].tick_params(axis='x', labelsize=fontsize)
axs[-1].set_xlabel('Time (sec)',fontsize=fontsize)
fig.savefig(fig_dir/'IMU_Example.pdf', facecolor='white', transparent=True, bbox_inches='tight')


In [ ]:
WC_raw = np.load('/home/seuss/Research/SensoryMotorPred_Data/data/070921/J553RT/fm1/uncorrected_worldcam_dt050.npy')

In [ ]:
WC_raw.shape,data['model_vid_sm'].shape

In [ ]:
fig,axs = plt.subplots(1,figsize=(2,2))
# axs.imshow(WC_raw[1000], cmap='gray')
axs.imshow(model_vid_sm_shift[2000], cmap='gray')
axs.axis('off')
fig.savefig(fig_dir/'WC_DSIm.pdf', facecolor='white', transparent=True, bbox_inches='tight')

# axs[1].imshow(model_vid_sm_shift[1000])

### Position Only

In [ ]:
celln = 101
bin_length=40
Nepochs = 5000
fontsize = 24
for Kfold in range(3):
    move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')
    GLM_Mot = ioh5.load(save_dir/'GLM_{}_VisMov_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_Kfold{:02d}.h5'.format('Pytorch_Mot',int(params['model_dt']*1000), params['nt_glm_lag'], 0, Nepochs, Kfold))
    GLM_Vis = ioh5.load(save_dir/'GLM_{}_VisMov_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_Kfold{:02d}.h5'.format('Pytorch_Vis',int(params['model_dt']*1000), params['nt_glm_lag'], 1, 12000, Kfold))
    GLM_Add = ioh5.load(save_dir/'GLM_{}_VisMov_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_Kfold{:02d}.h5'.format('Pytorch_Add',int(params['model_dt']*1000), params['nt_glm_lag'], 2, Nepochs, Kfold))
    GLM_Mul = ioh5.load(save_dir/'GLM_{}_VisMov_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_Kfold{:02d}.h5'.format('Pytorch_Mul',int(params['model_dt']*1000), params['nt_glm_lag'], 2, Nepochs, Kfold))

    fig,ax = plt.subplots(figsize=(20,5))
    sp_smooth = ((np.convolve(GLM_Vis['test_nsp'][:,celln], np.ones(bin_length), 'same')) / (bin_length * params['model_dt']))[int(bin_length*1.5):-int(bin_length*1.5)]

    pred_smooth_Mot = ((np.convolve(GLM_Mot['pred_test'][:,celln], np.ones(bin_length), 'same')) / (bin_length * params['model_dt']))[int(bin_length*1.5):-int(bin_length*1.5)]
    pred_smooth_Vis = ((np.convolve(GLM_Vis['pred_test'][:,celln], np.ones(bin_length), 'same')) / (bin_length * params['model_dt']))[int(bin_length*1.5):-int(bin_length*1.5)]

    pred_smooth_Add = ((np.convolve(GLM_Add['pred_test'][:,celln], np.ones(bin_length), 'same')) / (bin_length * params['model_dt']))[int(bin_length*1.5):-int(bin_length*1.5)]
    pred_smooth_Mul = ((np.convolve(GLM_Mul['pred_test'][:,celln], np.ones(bin_length), 'same')) / (bin_length * params['model_dt']))[int(bin_length*1.5):-int(bin_length*1.5)]

    ax.plot(np.arange(len(sp_smooth))*params['model_dt'],sp_smooth, 'k', lw=2,label='Actual')
    ax.plot(np.arange(len(pred_smooth_Mot))*params['model_dt'],pred_smooth_Mot,'r', lw=2,label='Mot')
    ax.plot(np.arange(len(pred_smooth_Vis))*params['model_dt'],pred_smooth_Vis,'b', lw=2,label='Vis')
    ax.plot(np.arange(len(pred_smooth_Add))*params['model_dt'],pred_smooth_Add,'m', lw=2,label='Add')
    ax.plot(np.arange(len(pred_smooth_Mul))*params['model_dt'],pred_smooth_Mul,'g', lw=2,label='Mul')
    ax.legend(labelcolor='linecolor', fontsize=18, handlelength=0, handletextpad=0, fancybox=True)
    ax.set_xlabel('Time (s)',fontsize=fontsize)
    ax.set_ylabel('Spike Rate',fontsize=fontsize)
    r2_Mot = np.corrcoef(sp_smooth,pred_smooth_Mot)[0,1]**2
    r2_Vis = np.corrcoef(sp_smooth,pred_smooth_Vis)[0,1]**2
    r2_Add = np.corrcoef(sp_smooth,pred_smooth_Add)[0,1]**2
    r2_Mul = np.corrcoef(sp_smooth,pred_smooth_Mul)[0,1]**2
    ax.set_title('Kfold:{:02d}, Mot:{:.03}, Vis:{:.03}, Add:{:.03}, Mul:{:.03}'.format(Kfold, r2_Mot,r2_Vis,r2_Add,r2_Mul))
    fig.savefig(fig_dir/'ExamplePredFR_celln{}_Kfold{:02d}.png'.format(celln,Kfold), facecolor='white', transparent=True, bbox_inches='tight')


In [ ]:
Kfold = 2
Nepochs=5000
bin_length = 40
move_train,move_test,model_move,nsp_raw,move_data,tuning_curves,tuning_stds,tuning_curve_edges,ax_ylims,tc_mod,avg_fr,tuning_sig,tuning_sig2,tuning_idx=load_Kfold_forPlots(params, Kfold=Kfold, dataset_type='test')
GLM_Mot = ioh5.load(save_dir/'GLM_{}_VisMov_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_Kfold{:02d}.h5'.format('Pytorch_Mot',int(params['model_dt']*1000), params['nt_glm_lag'], 0, 5000, Kfold))
GLM_Vis = ioh5.load(save_dir/'GLM_{}_VisMov_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_Kfold{:02d}.h5'.format('Pytorch_Vis',int(params['model_dt']*1000), params['nt_glm_lag'], 1, 12000, Kfold))
GLM_Add = ioh5.load(save_dir/'GLM_{}_VisMov_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_Kfold{:02d}.h5'.format('Pytorch_Add',int(params['model_dt']*1000), params['nt_glm_lag'], 2, 5000, Kfold))
GLM_Mul = ioh5.load(save_dir/'GLM_{}_VisMov_dt{:03d}_T{:02d}_MovModel{:d}_NB{}_Kfold{:02d}.h5'.format('Pytorch_Mul',int(params['model_dt']*1000), params['nt_glm_lag'], 2, 5000, Kfold))


In [ ]:

celln = 101
t = 1200
dt = 2000
fontsize=24
fig,ax = plt.subplots(figsize=(15,5))
sp_smooth = ((np.convolve(GLM_Vis['test_nsp'][:,celln], np.ones(bin_length), 'same')) / (bin_length * params['model_dt']))[int(bin_length*1.5):-int(bin_length*1.5)]

pred_smooth_Mot = ((np.convolve(GLM_Mot['pred_test'][:,celln], np.ones(bin_length), 'same')) / (bin_length * params['model_dt']))[int(bin_length*1.5):-int(bin_length*1.5)]
pred_smooth_Vis = ((np.convolve(GLM_Vis['pred_test'][:,celln], np.ones(bin_length), 'same')) / (bin_length * params['model_dt']))[int(bin_length*1.5):-int(bin_length*1.5)]
pred_smooth_Add = ((np.convolve(GLM_Add['pred_test'][:,celln], np.ones(bin_length), 'same')) / (bin_length * params['model_dt']))[int(bin_length*1.5):-int(bin_length*1.5)]
pred_smooth_Mul = ((np.convolve(GLM_Mul['pred_test'][:,celln], np.ones(bin_length), 'same')) / (bin_length * params['model_dt']))[int(bin_length*1.5):-int(bin_length*1.5)]

r2_Mot = np.corrcoef(sp_smooth,pred_smooth_Mot)[0,1]**2
r2_Vis = np.corrcoef(sp_smooth,pred_smooth_Vis)[0,1]**2
r2_Add = np.corrcoef(sp_smooth,pred_smooth_Add)[0,1]**2
r2_Mul = np.corrcoef(sp_smooth,pred_smooth_Mul)[0,1]**2

ax.plot(np.arange(len(sp_smooth[t:t+dt]))*params['model_dt'],sp_smooth[t:t+dt], 'k', lw=5,label='Actual', zorder=0)
ax.plot(np.arange(len(pred_smooth_Mot[t:t+dt]))*params['model_dt'],pred_smooth_Mot[t:t+dt],'r', lw=5,label='Pos: $r^2$={:.02f}'.format(r2_Mot), zorder=4)
ax.plot(np.arange(len(pred_smooth_Vis[t:t+dt]))*params['model_dt'],pred_smooth_Vis[t:t+dt],'b', lw=5,label='Vis: $r^2$={:.02f}'.format(r2_Vis), zorder=3)
ax.plot(np.arange(len(pred_smooth_Add[t:t+dt]))*params['model_dt'],pred_smooth_Add[t:t+dt],'m', lw=5,label='Add: $r^2$={:.02f}'.format(r2_Add), zorder=2)
ax.plot(np.arange(len(pred_smooth_Mul[t:t+dt]))*params['model_dt'],pred_smooth_Mul[t:t+dt],'g', lw=5,label='Mul: $r^2$={:.02f}'.format(r2_Mul), zorder=1)
ax.legend(labelcolor='linecolor', fontsize=18, handlelength=0, handletextpad=0, fancybox=True, bbox_to_anchor=(1.01, 1))
ax.set_xlabel('Time (s)',fontsize=fontsize)
ax.set_ylabel('sp/sec',fontsize=fontsize)

fig.savefig(fig_dir/'Mul_ExamplePredFR_celln{}_Kfold{:02d}.pdf'.format(celln,Kfold), facecolor='white', transparent=True, bbox_inches='tight')